In [58]:
import pandas as pd
import numpy as np

In [59]:
df= pd.read_csv('data/activ.csv')
df.head()

,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,membership_name,membership_status
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,NaN,NaN
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,NaN,NaN
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,NaN,NaN
3,Juan Carlos Silva jaime,Juan Carlos,Silva Jaime,jcsilva@uach.mx,2025-09-28T19:16:02.033Z,usr-Fq8IHykAHIjcT1M,NaN,"0,00 US$",1,0,NaN,NaN
4,Cecilia alexandra Soto Andrade,Cecilia,Soto Andrade,sotoandrade069@gmail.com,2025-09-26T02:38:33.212Z,usr-U8jYkpT4BrtuA7k,NaN,"0,00 US$",1,0,NaN,NaN


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3696 entries, 0 to 3695
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    3422 non-null   object 
 1   first_name              2985 non-null   object 
 2   last_name               2807 non-null   object 
 3   email                   3696 non-null   object 
 4   first_seen              3696 non-null   object 
 5   user_api_id             3696 non-null   object 
 6   tags                    4 non-null      object 
 7   revenue                 3696 non-null   object 
 8   event_approved_count    3696 non-null   int64  
 9   event_checked_in_count  3696 non-null   int64  
 10  membership_name         0 non-null      float64
 11  membership_status       0 non-null      float64
dtypes: float64(2), int64(2), object(8)
memory usage: 346.6+ KB


In [61]:
# Conteo de valores vacíos en la columna 'name'
count_empty = df['name'].isna().sum()
print(f"Registros con 'name' vacío: {count_empty}")

Registros con 'name' vacío: 274


In [62]:
# Revisar emails de registros sin nombre para deducir posibles nombres
df_no_name = df[df['name'].isna() | df['name'].str.strip().eq('')]

# Extraer la parte local del email (antes del @) como posible nombre
df_no_name_sample = df_no_name[['name', 'first_name', 'email']].copy()
df_no_name_sample['email_local'] = df_no_name_sample['email'].str.split('@').str[0]

print(f"Total de registros sin nombre: {len(df_no_name_sample)}\n")
print("Primeros 20 registros con sus emails:")
print(df_no_name_sample.head(20))

# Mostrar estadísticas de los emails
print(f"\n\nTotal de emails únicos: {df_no_name_sample['email'].nunique()}")

Total de registros sin nombre: 274

Primeros 20 registros con sus emails:
    name first_name                           email           email_local
349  NaN        NaN              s.loer@outlook.com                s.loer
350  NaN        NaN               mburrola1@uach.mx             mburrola1
351  NaN        NaN             leosausix@gmail.com             leosausix
352  NaN        NaN        fernandoaginer@gmail.com        fernandoaginer
353  NaN        NaN        marlenfavela@hotmail.com          marlenfavela
356  NaN        NaN           daneliz18@hotmail.com             daneliz18
358  NaN        NaN               rocio.rojo@tec.mx            rocio.rojo
359  NaN        NaN   jberenice.mendoza@outlook.com     jberenice.mendoza
360  NaN        NaN         alejandromaju@gmail.com         alejandromaju
361  NaN        NaN        ishida.cecilia@gmail.com        ishida.cecilia
363  NaN        NaN          pablo.aguirre@meddi.mx         pablo.aguirre
364  NaN        NaN          victor.mo

In [63]:
# Crear columna email_local en el dataframe principal
df['email_local'] = df['email'].str.split('@').str[0]

In [64]:
# Limpiar la columna 'name': eliminar espacios dobles y capitalizar cada palabra
df['name'] = df['name'].str.replace(r'\s+', ' ', regex=True).str.strip().str.title()

In [65]:
import unicodedata

# Función para eliminar tildes/acentos
def remove_accents(text):
    if pd.isna(text):
        return text
    # Normalizar a NFD (descompone caracteres con tildes)
    # y eliminar marcas diacríticas
    nfd = unicodedata.normalize('NFD', text)
    return ''.join([c for c in nfd if unicodedata.category(c) != 'Mn'])

# Aplicar a la columna 'name'
df['name'] = df['name'].apply(remove_accents)

print("Tildes eliminadas exitosamente")
print(f"\nPrimeros 10 registros después de eliminar tildes:")
print(df[['name', 'first_name', 'last_name']].head(10))

Tildes eliminadas exitosamente

Primeros 10 registros después de eliminar tildes:
                                  name   first_name         last_name
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon
1                          Alex Palomo         Alex            Palomo
2                      Laura Hernandez        Laura         Hernandez
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade
5                     Stephanie Jaquez    Stephanie            Jaquez
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez
7                            Luis Diaz         Luis              Diaz
8           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez
9                          Johan Bader        Johan             Bader


In [66]:
# Normalizar todos los nombres: capitalizar cada palabra
df['name'] = df['name'].str.title()
df['first_name'] = df['first_name'].str.title()
df['last_name'] = df['last_name'].str.title()

# Actualizar también la columna full_name después de normalizar
df['full_name'] = df['name'].fillna('').str.strip().str.lower()

print("Nombres normalizados exitosamente")
print(f"\nPrimeros 10 registros después de la normalización:")
print(df[['name', 'first_name', 'last_name', 'full_name']].head(10))

Nombres normalizados exitosamente

Primeros 10 registros después de la normalización:
                                  name   first_name         last_name  \
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon   
1                          Alex Palomo         Alex            Palomo   
2                      Laura Hernandez        Laura         Hernandez   
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime   
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade   
5                     Stephanie Jaquez    Stephanie            Jaquez   
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
7                            Luis Diaz         Luis              Diaz   
8           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
9                          Johan Bader        Johan             Bader   

                             full_name  
0  cristian gian pieer sanchez martell  
1                          a

In [67]:
# Eliminar espacios dobles en todas las columnas de tipo string
df = df.apply(lambda x: x.str.replace(r'\s+', ' ', regex=True).str.strip() if x.dtype == "object" else x)

print("Espacios dobles eliminados exitosamente")
print(f"\nPrimeros 10 registros después de limpiar espacios:")
print(df[['name', 'first_name', 'last_name', 'email']].head(10))

Espacios dobles eliminados exitosamente

Primeros 10 registros después de limpiar espacios:
                                  name   first_name         last_name  \
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon   
1                          Alex Palomo         Alex            Palomo   
2                      Laura Hernandez        Laura         Hernandez   
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime   
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade   
5                     Stephanie Jaquez    Stephanie            Jaquez   
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
7                            Luis Diaz         Luis              Diaz   
8           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
9                          Johan Bader        Johan             Bader   

                                email  
0    phayromartellfalcon380@gmail.com  
1           alexpalomo@l

In [68]:
# Crear función para extraer nombre del email
def extract_name_from_email(email):
    if pd.isna(email):
        return ''
    
    # Extraer la parte local del email (antes del @)
    local_part = email.split('@')[0]
    
    # Limpiar caracteres no alfabéticos comunes
    # Mantener solo letras, espacios y puntos
    cleaned = local_part.replace('.', ' ').replace('_', ' ').replace('-', ' ')
    
    # Remover números al final (ej: maria123 -> maria)
    import re
    cleaned = re.sub(r'\d+$', '', cleaned)
    
    # Capitalizar cada palabra
    name_parts = cleaned.split()
    capitalized = ' '.join([part.capitalize() for part in name_parts if part.isalpha()])
    
    return capitalized.strip()

# Crear nueva columna con nombre extraído del email
df['name_from_email'] = df['email'].apply(extract_name_from_email)

# Mostrar registros donde 'name' está vacío pero se pudo extraer del email
mask_empty_name = df['name'].isna() | df['name'].str.strip().eq('')
df_extracted = df[mask_empty_name & (df['name_from_email'] != '')].copy()

print(f"Total de registros con nombre extraído del email: {len(df_extracted)}\n")
print("Primeros 20 registros:")
print(df_extracted[['name', 'name_from_email', 'email']].head(20))

Total de registros con nombre extraído del email: 274

Primeros 20 registros:
    name       name_from_email                           email
349  NaN                S Loer              s.loer@outlook.com
350  NaN              Mburrola               mburrola1@uach.mx
351  NaN             Leosausix             leosausix@gmail.com
352  NaN        Fernandoaginer        fernandoaginer@gmail.com
353  NaN          Marlenfavela        marlenfavela@hotmail.com
356  NaN               Daneliz           daneliz18@hotmail.com
358  NaN            Rocio Rojo               rocio.rojo@tec.mx
359  NaN     Jberenice Mendoza   jberenice.mendoza@outlook.com
360  NaN         Alejandromaju         alejandromaju@gmail.com
361  NaN        Ishida Cecilia        ishida.cecilia@gmail.com
363  NaN         Pablo Aguirre          pablo.aguirre@meddi.mx
364  NaN        Victor Morales          victor.morales@uacj.mx
365  NaN         Hectordecionr         hectordecionr@gmail.com
366  NaN  Luis Melendez Araiza  luis.mel

In [69]:
# Crear columna para identificar si first_name contiene dos nombres
df['has_two_first_names'] = df['first_name'].fillna('').str.strip().str.split().str.len() >= 2

# Crear columnas para separar los dos nombres
df['first_name_1'] = df['first_name'].fillna('').str.strip().str.split().str[0]
df['first_name_2'] = df['first_name'].fillna('').str.strip().str.split().str[1]

print(f"Registros con dos nombres: {df['has_two_first_names'].sum()}\n")
print("Primeros 20 registros con dos nombres:")
print(df[df['has_two_first_names']][['first_name', 'first_name_1', 'first_name_2', 'name']].head(20))

Registros con dos nombres: 339

Primeros 20 registros con dos nombres:
           first_name first_name_1 first_name_2  \
3         Juan Carlos         Juan       Carlos   
52     Barbara Alicia      Barbara       Alicia   
61     María Fernanda        María     Fernanda   
79      Lidia Yesenia        Lidia      Yesenia   
98      Martha Silvia       Martha       Silvia   
99      Jorge Alberto        Jorge      Alberto   
122       Mia Isabela          Mia      Isabela   
133          Xiao Yan         Xiao          Yan   
138  María De Lourdes        María           De   
141     Aldo Dionicio         Aldo     Dionicio   
147     José Humberto         José     Humberto   
152    Maria Fernanda        Maria     Fernanda   
172    Maria Cristina        Maria     Cristina   
187        Javier Noe       Javier          Noe   
189        Jesús Obed        Jesús         Obed   
198       José Carlos         José       Carlos   
205      Jesús Andrés        Jesús       Andrés   
212       A

In [70]:
# Crear columna para identificar si last_name contiene dos apellidos
df['has_two_last_names'] = df['last_name'].fillna('').str.strip().str.split().str.len() >= 2

# Crear columnas para separar los dos apellidos
df['last_name_1'] = df['last_name'].fillna('').str.strip().str.split().str[0]
df['last_name_2'] = df['last_name'].fillna('').str.strip().str.split().str[1]

print(f"Registros con dos apellidos: {df['has_two_last_names'].sum()}\n")
print("Primeros 20 registros con dos apellidos:")
print(df[df['has_two_last_names']][['last_name', 'last_name_1', 'last_name_2', 'name']].head(20))

Registros con dos apellidos: 1282

Primeros 20 registros con dos apellidos:
                     last_name last_name_1 last_name_2  \
0               Martell Falcon     Martell      Falcon   
3                  Silva Jaime       Silva       Jaime   
4                 Soto Andrade        Soto     Andrade   
6             Navarrete Jaquez   Navarrete      Jaquez   
8             Navarrete Jaquez   Navarrete      Jaquez   
13  Alejjandro Cervantes Tarín  Alejjandro   Cervantes   
15               Palomec Ortiz     Palomec       Ortiz   
17              Morales Ibarra     Morales      Ibarra   
19              Chavez Lazcano      Chavez     Lazcano   
21                Leon Mendoza        Leon     Mendoza   
22   Del Carmen Acosta Luevano         Del      Carmen   
23          David Vargas Perez       David      Vargas   
24                 Leal Chávez        Leal      Chávez   
25            Aguilar Quiñonez     Aguilar    Quiñonez   
31              Ortiz Hernádez       Ortiz    Hernádez

In [71]:
df.head(50)

,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,...,membership_status,email_local,full_name,name_from_email,has_two_first_names,first_name_1,first_name_2,has_two_last_names,last_name_1,last_name_2
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,...,NaN,phayromartellfalcon380,cristian gian pieer sanchez martell,Phayromartellfalcon,False,Phayro,NaN,True,Martell,Falcon
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,...,NaN,alexpalomo,alex palomo,Alexpalomo,False,Alex,NaN,False,Palomo,NaN
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,...,NaN,laurar1803,laura hernandez,Laurar,False,Laura,NaN,False,Hernandez,NaN
3,Juan Carlos Silva Jaime,Juan Carlos,Silva Jaime,jcsilva@uach.mx,2025-09-28T19:16:02.033Z,usr-Fq8IHykAHIjcT1M,NaN,"0,00 US$",1,0,...,NaN,jcsilva,juan carlos silva jaime,Jcsilva,True,Juan,Carlos,True,Silva,Jaime
4,Cecilia Alexandra Soto Andrade,Cecilia,Soto Andrade,sotoandrade069@gmail.com,2025-09-26T02:38:33.212Z,usr-U8jYkpT4BrtuA7k,NaN,"0,00 US$",1,0,...,NaN,sotoandrade069,cecilia alexandra soto andrade,Sotoandrade,False,Cecilia,NaN,True,Soto,Andrade
5,Stephanie Jaquez,Stephanie,Jaquez,mario.simental0208@gmail.com,2025-09-26T01:57:45.053Z,usr-ShowEBU58V81RCy,NaN,"0,00 US$",1,0,...,NaN,mario.simental0208,stephanie jaquez,Mario Simental,False,Stephanie,NaN,False,Jaquez,NaN
6,Stephanie Navarrete Jaquez,Stephanie,Navarrete Jaquez,fannyjaquez4566@gmail.com,2025-09-26T01:53:27.884Z,usr-73xJIFmdQJNY5hK,NaN,"0,00 US$",1,0,...,NaN,fannyjaquez4566,stephanie navarrete jaquez,Fannyjaquez,False,Stephanie,NaN,True,Navarrete,Jaquez
7,Luis Diaz,Luis,Diaz,mi6141526425@gmail.com,2025-09-26T01:52:16.008Z,usr-NlNR7sHw59X7mSK,NaN,"0,00 US$",1,0,...,NaN,mi6141526425,luis diaz,Mi,False,Luis,NaN,False,Diaz,NaN
8,Stephanie Navarrete Jaquez,Stephanie,Navarrete Jaquez,stephanienavarretejaquez@gmail.com,2025-09-26T01:49:09.650Z,usr-JIVumRu4rwmJR9y,NaN,"0,00 US$",1,0,...,NaN,stephanienavarretejaquez,stephanie navarrete jaquez,Stephanienavarretejaquez,False,Stephanie,NaN,True,Navarrete,Jaquez
9,Johan Bader,Johan,Bader,maxkmp22@gmail.com,2025-09-26T01:09:06.206Z,usr-W1U23T9UAnod66J,NaN,"0,00 US$",1,0,...,NaN,maxkmp22,johan bader,Maxkmp,False,Johan,NaN,False,Bader,NaN


In [72]:
# Rellenar campos vacíos de 'name' con el valor de 'name_from_email'
# Solo para registros donde 'name' está vacío y 'name_from_email' tiene un valor válido
mask_to_fill = (df['name'].isna() | df['name'].str.strip().eq('')) & (df['name_from_email'] != '')

# Contar cuántos registros se van a actualizar
count_to_update = mask_to_fill.sum()

# Realizar la actualización
df.loc[mask_to_fill, 'name'] = df.loc[mask_to_fill, 'name_from_email']

# Actualizar también full_name para estos registros
df.loc[mask_to_fill, 'full_name'] = df.loc[mask_to_fill, 'name'].str.strip().str.lower()

print(f"Se actualizaron {count_to_update} registros con nombres extraídos del email\n")

# Verificar cuántos registros quedan con 'name' vacío después de la actualización
mask_still_empty = df['name'].isna() | df['name'].str.strip().eq('')
count_still_empty = mask_still_empty.sum()

print(f"Registros con 'name' vacío antes: {count_empty}")
print(f"Registros actualizados: {count_to_update}")
print(f"Registros con 'name' vacío después: {count_still_empty}\n")

# Mostrar algunos ejemplos de los registros actualizados
print("Primeros 20 registros actualizados:")
print(df[mask_to_fill][['name', 'name_from_email', 'email', 'full_name']].head(20))

Se actualizaron 274 registros con nombres extraídos del email

Registros con 'name' vacío antes: 274
Registros actualizados: 274
Registros con 'name' vacío después: 0

Primeros 20 registros actualizados:
                     name       name_from_email  \
349                S Loer                S Loer   
350              Mburrola              Mburrola   
351             Leosausix             Leosausix   
352        Fernandoaginer        Fernandoaginer   
353          Marlenfavela          Marlenfavela   
356               Daneliz               Daneliz   
358            Rocio Rojo            Rocio Rojo   
359     Jberenice Mendoza     Jberenice Mendoza   
360         Alejandromaju         Alejandromaju   
361        Ishida Cecilia        Ishida Cecilia   
363         Pablo Aguirre         Pablo Aguirre   
364        Victor Morales        Victor Morales   
365         Hectordecionr         Hectordecionr   
366  Luis Melendez Araiza  Luis Melendez Araiza   
367             Yodeseoun      

In [73]:
import re

def infer_name_from_email(email_local):
    """
    Intenta inferir un nombre real separando palabras que están pegadas en el email.
    Retorna cadena vacía si no puede inferir con confianza o si parece un nombre genérico.
    """
    if pd.isna(email_local) or not email_local:
        return ''
    
    # Limpiar números y caracteres especiales
    cleaned = re.sub(r'[0-9._-]', ' ', email_local).strip()
    
    # Lista de palabras que indican que NO es un nombre de persona
    skip_words = [
        'contacto', 'info', 'admin', 'soporte', 'support', 'ventas', 
        'comunicacion', 'asistente', 'assistant', 'ayuda', 'help',
        'registro', 'hola', 'hello', 'test', 'prueba', 'gerencia'
    ]
    
    # Si contiene alguna palabra genérica, retornar vacío
    if any(word in cleaned.lower() for word in skip_words):
        return ''
    
    # Si es muy corto (menos de 4 caracteres), probablemente no sea un nombre completo
    if len(cleaned) < 4:
        return ''
    
    # Intentar separar palabras pegadas usando mayúsculas como delimitador
    # Ejemplo: "FernandoGiner" -> "Fernando Giner"
    separated = re.sub(r'([a-z])([A-Z])', r'\1 \2', email_local)
    separated = re.sub(r'[0-9._-]', ' ', separated).strip()
    
    # Si después de separar tiene 2 o más palabras alfabéticas, es candidato
    words = [w for w in separated.split() if w.isalpha() and len(w) >= 2]
    
    # Solo aceptar si tiene al menos 2 palabras (nombre y apellido)
    if len(words) >= 2:
        return ' '.join(words).title()
    
    return ''

# Crear nueva columna con nombres inferidos mejorados
df['name_inferred'] = df['email_local'].apply(infer_name_from_email)

# Actualizar solo los registros donde 'name' sigue vacío y se pudo inferir un nombre válido
mask_to_infer = (df['name'].isna() | df['name'].str.strip().eq('')) & (df['name_inferred'] != '')

count_inferred = mask_to_infer.sum()

# Aplicar la actualización
df.loc[mask_to_infer, 'name'] = df.loc[mask_to_infer, 'name_inferred']
df.loc[mask_to_infer, 'full_name'] = df.loc[mask_to_infer, 'name'].str.strip().str.lower()

print(f"Se infirieron {count_inferred} nombres adicionales desde el email\n")

# Mostrar ejemplos de nombres inferidos
if count_inferred > 0:
    print("Ejemplos de nombres inferidos:")
    print(df[mask_to_infer][['name', 'name_inferred', 'email', 'email_local']].head(20))
else:
    print("No se pudieron inferir nombres adicionales")

# Contar cuántos registros quedan sin nombre
mask_final_empty = df['name'].isna() | df['name'].str.strip().eq('')
count_final_empty = mask_final_empty.sum()

print(f"\nRegistros con 'name' vacío después de la inferencia: {count_final_empty}")

Se infirieron 0 nombres adicionales desde el email

No se pudieron inferir nombres adicionales

Registros con 'name' vacío después de la inferencia: 0


In [74]:
df.head()

,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,...,email_local,full_name,name_from_email,has_two_first_names,first_name_1,first_name_2,has_two_last_names,last_name_1,last_name_2,name_inferred
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,...,phayromartellfalcon380,cristian gian pieer sanchez martell,Phayromartellfalcon,False,Phayro,NaN,True,Martell,Falcon,
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,...,alexpalomo,alex palomo,Alexpalomo,False,Alex,NaN,False,Palomo,NaN,
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,...,laurar1803,laura hernandez,Laurar,False,Laura,NaN,False,Hernandez,NaN,
3,Juan Carlos Silva Jaime,Juan Carlos,Silva Jaime,jcsilva@uach.mx,2025-09-28T19:16:02.033Z,usr-Fq8IHykAHIjcT1M,NaN,"0,00 US$",1,0,...,jcsilva,juan carlos silva jaime,Jcsilva,True,Juan,Carlos,True,Silva,Jaime,
4,Cecilia Alexandra Soto Andrade,Cecilia,Soto Andrade,sotoandrade069@gmail.com,2025-09-26T02:38:33.212Z,usr-U8jYkpT4BrtuA7k,NaN,"0,00 US$",1,0,...,sotoandrade069,cecilia alexandra soto andrade,Sotoandrade,False,Cecilia,NaN,True,Soto,Andrade,


In [75]:
# Cuenta registros con 'name' vacío (NaN o solo espacios)
mask_empty_name = df['name'].isna() | df['name'].str.strip().eq('')
count_empty_name = int(mask_empty_name.sum())
print(count_empty_name)

0


In [76]:
# Identificar y analizar registros duplicados basados en 'full_name'
duplicated_names = df[df['full_name'] != '']['full_name'].value_counts()
duplicated_names = duplicated_names[duplicated_names > 1]

print(f"Total de nombres duplicados: {len(duplicated_names)}")
print(f"Total de registros afectados: {duplicated_names.sum()}\n")

# Mostrar los nombres más duplicados
print("Top 20 nombres más duplicados:")
print(duplicated_names.head(20))

# Mostrar detalles de algunos registros duplicados
if len(duplicated_names) > 0:
    print("\n" + "="*80)
    print("EJEMPLO: Primeros registros duplicados")
    print("="*80)
    
    # Tomar el primer nombre duplicado como ejemplo
    first_duplicated_name = duplicated_names.index[0]
    df_example = df[df['full_name'] == first_duplicated_name].sort_values('email')
    print(f"\nNombre: {first_duplicated_name}")
    print(f"Cantidad de duplicados: {len(df_example)}\n")
    print(df_example[['name', 'first_name', 'last_name', 'email', 'full_name']])

Total de nombres duplicados: 219
Total de registros afectados: 493

Top 20 nombres más duplicados:
full_name
contacto            6
carlos garcia       5
ventas              5
a                   5
carlos              5
alejandro           5
diego               5
alan                4
alejandro romero    4
armando             4
daniel              4
info                4
rafael gonzalez     4
andrea              3
eduardo padilla     3
jose parra          3
jaime gonzalez      3
salvador beltran    3
lizeth estrada      3
luis nevarez        3
Name: count, dtype: int64

EJEMPLO: Primeros registros duplicados

Nombre: contacto
Cantidad de duplicados: 6

          name first_name last_name                         email full_name
2037  Contacto        NaN       NaN          contacto@alsuper.com  contacto
3116  Contacto        NaN       NaN  contacto@chihuahuafutura.org  contacto
2039  Contacto        NaN       NaN            contacto@demek.com  contacto
1993  Contacto        NaN       NaN 

In [77]:
# Limpiar full_name eliminando valores genéricos que no son nombres reales
generic_values = ['contacto', 'ventas', 'a', 'info']

# Crear máscara para identificar registros con valores genéricos en full_name
mask_generic = df['full_name'].isin(generic_values)

# Contar cuántos registros se van a limpiar
count_generic = mask_generic.sum()

print(f"Registros con valores genéricos en full_name: {count_generic}\n")

# Mostrar ejemplos antes de limpiar
if count_generic > 0:
    print("Ejemplos antes de limpiar:")
    print(df[mask_generic][['name', 'full_name', 'email']].head(20))
    print("\n" + "="*80 + "\n")

# Reemplazar full_name con cadena vacía para valores genéricos
df.loc[mask_generic, 'full_name'] = ''

print(f"Se limpiaron {count_generic} registros con valores genéricos\n")

# Verificar después de la limpieza
print("Valores únicos de full_name después de limpiar:")
print(df['full_name'].value_counts().head(20))

Registros con valores genéricos en full_name: 20

Ejemplos antes de limpiar:
          name full_name                          email
1982  Contacto  contacto      contacto@grupomesteno.com
1993  Contacto  contacto           contacto@deutzmx.com
1994      Info      info         info@artecpisos.com.mx
2004    Ventas    ventas          ventas@cofiasa.com.mx
2008      Info      info                info@climas.com
2014    Ventas    ventas        ventas@grupoamex.com.mx
2018  Contacto  contacto               contacto@tr3z.mx
2021      Info      info            info@coprose.com.mx
2024    Ventas    ventas           ventas@abitat.com.mx
2035    Ventas    ventas  ventas@tacglobalsolutions.com
2037  Contacto  contacto           contacto@alsuper.com
2039  Contacto  contacto             contacto@demek.com
2051    Ventas    ventas       ventas@hotelsoberano.com
2840         A         a               a01562935@tec.mx
2854         A         a               a01569486@tec.mx
2863         A         a   

In [78]:
# Eliminar duplicados basados en 'full_name', manteniendo solo la primera ocurrencia
# Solo se eliminan duplicados donde full_name no esté vacío

# Contar duplicados antes de eliminar
duplicates_before = df[df['full_name'] != '']['full_name'].duplicated().sum()
total_before = len(df)

# Eliminar duplicados manteniendo el primer registro de cada full_name
# Los registros con full_name vacío se mantienen todos
df = df[~((df['full_name'] != '') & df['full_name'].duplicated(keep='first'))]

# Resetear el índice después de eliminar filas
df = df.reset_index(drop=True)

total_after = len(df)
duplicates_removed = total_before - total_after

print(f"Registros antes de eliminar duplicados: {total_before}")
print(f"Duplicados eliminados: {duplicates_removed}")
print(f"Registros después de eliminar duplicados: {total_after}")

# Verificar que ya no hay duplicados en full_name (excluyendo valores vacíos)
duplicates_after = df[df['full_name'] != '']['full_name'].duplicated().sum()
print(f"\nDuplicados restantes en full_name: {duplicates_after}")

# Mostrar los primeros registros del dataframe limpio
print("\nPrimeros 10 registros del dataframe limpio:")
print(df[['name', 'first_name', 'last_name', 'email', 'full_name']].head(10))

Registros antes de eliminar duplicados: 3696
Duplicados eliminados: 258
Registros después de eliminar duplicados: 3438

Duplicados restantes en full_name: 0

Primeros 10 registros del dataframe limpio:
                                  name   first_name         last_name  \
0  Cristian Gian Pieer Sanchez Martell       Phayro    Martell Falcon   
1                          Alex Palomo         Alex            Palomo   
2                      Laura Hernandez        Laura         Hernandez   
3              Juan Carlos Silva Jaime  Juan Carlos       Silva Jaime   
4       Cecilia Alexandra Soto Andrade      Cecilia      Soto Andrade   
5                     Stephanie Jaquez    Stephanie            Jaquez   
6           Stephanie Navarrete Jaquez    Stephanie  Navarrete Jaquez   
7                            Luis Diaz         Luis              Diaz   
8                          Johan Bader        Johan             Bader   
9                       Sophia Alarcon       Sophia           Alarcó

In [79]:
import sys, subprocess

# Instalación opcional y conteo de género estimado a partir del primer nombre
try:
    import gender_guesser.detector as gender
except ModuleNotFoundError:
    # Instalamos la librería si no está disponible
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gender-guesser"], check=True)
    import gender_guesser.detector as gender

detector = gender.Detector(case_sensitive=False)

def guess_gender(row):
    first = row["first_name"]
    fallback = row["name"]
    candidate = (str(first) if pd.notna(first) and str(first).strip() else str(fallback)).split()[0] if pd.notna(fallback) else ""
    raw = detector.get_gender(candidate) if candidate else "unknown"
    mapping = {
        "male": "male",
        "mostly_male": "male",
        "female": "female",
        "mostly_female": "female",
        "andy": "unknown",
        "unknown": "unknown",
    }
    return mapping.get(raw, "unknown")

df["gender_guess"] = df.apply(guess_gender, axis=1)
gender_counts = df["gender_guess"].value_counts().rename_axis("gender").reset_index(name="count")
print(gender_counts)

    gender  count
0     male   1737
1   female   1186
2  unknown    515


In [80]:
# Filtrar registros con género "unknown" y mostrar los primeros 20
df_unknown = df[df["gender_guess"] == "unknown"]
print(f"Total de nombres con género desconocido: {len(df_unknown)}\n")
print(df_unknown[["name", "first_name", "gender_guess"]].head(20))

Total de nombres con género desconocido: 515

                                    name first_name gender_guess
0    Cristian Gian Pieer Sanchez Martell     Phayro      unknown
12    Gnolaum Alejjandro Cervantes Tarin    Gnolaum      unknown
14         Yarelli Abigail Palomec Ortiz    Yarelli      unknown
18                 Sandta Chavez Lazcano     Sandta      unknown
27                                 Eneas      Eneas      unknown
31                                 Zared      Zared      unknown
39                  Aricela Perez Espino    Aricela      unknown
52                         Idaly Alvarez      Idaly      unknown
53                         Rossy Navarro      Rossy      unknown
60                     Mifam El Porvenir      Mifam      unknown
64                          Jovany Diego     Jovany      unknown
74                      Amaranda Becerra   Amaranda      unknown
86                           Aide Neaves       Aidé      unknown
95                                 Areli    

In [81]:
# Filtrar nombres femeninos y mostrar los primeros 20
df_female = df[df['gender_guess'] == 'female']
print(f"Total de nombres femeninos: {len(df_female)}\n")
print(df_female[['name', 'first_name', 'gender_guess']].head(20))

Total de nombres femeninos: 1186

                                 name      first_name gender_guess
2                     Laura Hernandez           Laura       female
4      Cecilia Alexandra Soto Andrade         Cecilia       female
5                    Stephanie Jaquez       Stephanie       female
6          Stephanie Navarrete Jaquez       Stephanie       female
9                      Sophia Alarcon          Sophia       female
16              Renata Morales Ibarra          Renata       female
20  Vanessa Del Carmen Acosta Luevano         Vanessa       female
25                          Valeria S         Valeria       female
28                       Lucia Orozco           Lucia       female
33                     Annette Aguayo         Annette       female
42            Gabriela Fuentes Garcia        Gabriela       female
45                        Yadira Pena          Yadira       female
46                      Nadia Sanchez             NaN       female
50     Barbara Alicia Bordon

In [82]:
#imprime nombres masculinos
df_male = df[df['gender_guess'] == 'male']
print(f"Total de nombres masculinos: {len(df_male)}\n")
print(df_male[['name', 'first_name', 'gender_guess']].head(20))

Total de nombres masculinos: 1737

                               name   first_name gender_guess
1                       Alex Palomo         Alex         male
3           Juan Carlos Silva Jaime  Juan Carlos         male
7                         Luis Diaz         Luis         male
8                       Johan Bader        Johan         male
10                Christopher Bravo  Christopher         male
11                 Alejandro Arroyo    Alejandro         male
13                      Max Morales          Max         male
15                    Jaime Jimenez        Jaime         male
17                      Eric Aguayo         Eric         male
19         Manuel Ivan Leon Mendoza       Manuel         male
21        Gerardo David Vargasperez      Gerardo         male
22        Daniel Arturo Leal Chavez       Daniel         male
23  Joel Alejandro Aguilar Quinonez         Joel         male
24                     Angel Medina        Angel         male
26                        Guillermo

In [83]:
# Analizar más detalladamente los casos con género unknown
print("Análisis detallado de nombres con género unknown:")
print("="*60)

# Extraer el primer nombre de first_name para los casos unknown
df_unknown_analysis = df[df["gender_guess"] == "unknown"].copy()
df_unknown_analysis['first_word_from_first_name'] = df_unknown_analysis['first_name'].str.split().str[0]
df_unknown_analysis['first_word_from_name'] = df_unknown_analysis['name'].str.split().str[0]

print(f"Total de registros con género unknown: {len(df_unknown_analysis)}")
print("\nPrimeros 30 casos con first_name extraído:")
print(df_unknown_analysis[['name', 'first_name', 'first_word_from_first_name', 'first_word_from_name']].head(30))

# Ver qué primeros nombres son más comunes en los casos unknown
print("\nPrimeros nombres más comunes en casos unknown:")
first_names_unknown = df_unknown_analysis['first_word_from_first_name'].value_counts().head(20)
print(first_names_unknown)

Análisis detallado de nombres con género unknown:
Total de registros con género unknown: 515

Primeros 30 casos con first_name extraído:
                                     name  first_name  \
0     Cristian Gian Pieer Sanchez Martell      Phayro   
12     Gnolaum Alejjandro Cervantes Tarin     Gnolaum   
14          Yarelli Abigail Palomec Ortiz     Yarelli   
18                  Sandta Chavez Lazcano      Sandta   
27                                  Eneas       Eneas   
31                                  Zared       Zared   
39                   Aricela Perez Espino     Aricela   
52                          Idaly Alvarez       Idaly   
53                          Rossy Navarro       Rossy   
60                      Mifam El Porvenir       Mifam   
64                           Jovany Diego      Jovany   
74                       Amaranda Becerra    Amaranda   
86                            Aide Neaves        Aidé   
95                                  Areli       Areli   
99      

In [84]:
# Función mejorada para inferir género con listas específicas de nombres
def improved_gender_guess(row):
    """
    Función mejorada para inferir género usando listas de nombres comunes
    y múltiples estrategias de detección
    """
    
    # Listas de nombres comunes femeninos y masculinos en español
    female_names = {
        'maria', 'ana', 'carmen', 'josefa', 'isabel', 'dolores', 'pilar', 'teresa', 
        'rosa', 'francisca', 'antonia', 'mercedes', 'elena', 'cristina', 'patricia',
        'laura', 'monica', 'sandra', 'andrea', 'silvia', 'beatriz', 'marta',
        'lucia', 'sofia', 'claudia', 'natalia', 'paula', 'sara', 'adriana',
        'valeria', 'camila', 'daniela', 'alejandra', 'carolina', 'gabriela',
        'fernanda', 'mariana', 'paola', 'veronica', 'rocio', 'lorena', 'noelia',
        'esther', 'concepcion', 'amparo', 'gloria', 'esperanza', 'remedios',
        'encarnacion', 'montserrat', 'inmaculada', 'rosario', 'soledad'
    }
    
    male_names = {
        'antonio', 'manuel', 'jose', 'francisco', 'david', 'juan', 'javier',
        'jesus', 'daniel', 'carlos', 'miguel', 'rafael', 'pedro', 'angel',
        'alejandro', 'diego', 'sergio', 'fernando', 'pablo', 'jorge', 'luis',
        'alberto', 'ivan', 'oscar', 'ruben', 'adrian', 'raul', 'gonzalo',
        'mario', 'victor', 'eduardo', 'roberto', 'marcos', 'antonio', 'emilio',
        'ricardo', 'guillermo', 'andres', 'ramon', 'nicolas', 'jaime', 'salvador',
        'cesar', 'ignacio', 'francisco', 'joaquin', 'enrique', 'agustin', 'felipe'
    }
    
    first_name = str(row.get('first_name', ''))
    name = str(row.get('name', ''))
    
    # Extraer el primer nombre de first_name o name
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower().strip()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower().strip()
    else:
        return "unknown"
    
    # Limpiar el candidato
    candidate = candidate.replace('.', '').replace(',', '')
    
    # Verificar en listas específicas primero
    if candidate in female_names:
        return "female"
    elif candidate in male_names:
        return "male"
    
    # Si no está en las listas, usar el detector original
    try:
        raw_gender = detector.get_gender(candidate)
        mapping = {
            "male": "male",
            "mostly_male": "male", 
            "female": "female",
            "mostly_female": "female",
            "andy": "unknown",
            "unknown": "unknown",
        }
        return mapping.get(raw_gender, "unknown")
    except:
        return "unknown"

# Aplicar la función mejorada solo a los registros con género unknown
print("Aplicando función mejorada de detección de género...")
mask_unknown = df["gender_guess"] == "unknown"
print(f"Registros con género unknown antes: {mask_unknown.sum()}")

# Crear nueva columna temporal para comparar
df['gender_improved'] = df.apply(improved_gender_guess, axis=1)

# Ver cuántos se pudieron mejorar
df_comparison = df[mask_unknown].copy()
df_comparison['changed'] = df_comparison['gender_guess'] != df_comparison['gender_improved']
improved_count = df_comparison['changed'].sum()

print(f"Registros que se pudieron mejorar: {improved_count}")

# Mostrar algunos ejemplos de mejoras
if improved_count > 0:
    print("\nEjemplos de mejoras:")
    examples = df_comparison[df_comparison['changed']][['name', 'first_name', 'gender_guess', 'gender_improved']].head(20)
    print(examples)
    
    # Actualizar la columna principal
    df.loc[mask_unknown, 'gender_guess'] = df.loc[mask_unknown, 'gender_improved']
    
    # Mostrar el nuevo conteo
    print("\nNuevo conteo de géneros:")
    new_counts = df["gender_guess"].value_counts()
    print(new_counts)
else:
    print("No se encontraron mejoras con la función personalizada")

# Eliminar columna temporal
df = df.drop('gender_improved', axis=1)

Aplicando función mejorada de detección de género...
Registros con género unknown antes: 515
Registros que se pudieron mejorar: 9

Ejemplos de mejoras:
                               name       first_name gender_guess  \
147                   Rocio Ramirez            Rocio      unknown   
168         Rocio Rodriguez Sanchez            Rocio      unknown   
351                      Rocio Rojo              NaN      unknown   
875        Concepcion Ramos Caldera       Concepcion      unknown   
1109           Rocio Mendez Jimenez              NaN      unknown   
2436    Rocio Adela Flores Carrillo            Rocio      unknown   
2960  Joaquin Eliezer Blanco Chacon  Joaquin Eliezer      unknown   
3333                  Rocio Infante            Rocio      unknown   
3397                 Joaquin Galvez          Joaquin      unknown   

     gender_improved  
147           female  
168           female  
351           female  
875           female  
1109          female  
2436          femal

In [85]:
# Análisis adicional para nombres que terminan en patrones específicos
def analyze_name_patterns(name):
    """
    Analiza patrones en los nombres para inferir género
    """
    if pd.isna(name) or not name.strip():
        return "unknown"
        
    name = str(name).lower().strip()
    first_word = name.split()[0] if name.split() else ""
    
    # Patrones femeninos
    female_endings = ['a', 'ia', 'ina', 'ana', 'ela', 'isa', 'ita', 'eta']
    female_patterns = ['maria', 'ana', 'ina', 'lia', 'nda', 'tra']
    
    # Patrones masculinos  
    male_endings = ['o', 'io', 'ino', 'ano', 'elo', 'ito', 'eto']
    male_patterns = ['andro', 'ando', 'ardo', 'erto', 'urdo']
    
    # Verificar terminaciones
    for ending in female_endings:
        if first_word.endswith(ending) and len(first_word) > len(ending):
            return "female"
    
    for ending in male_endings:
        if first_word.endswith(ending) and len(first_word) > len(ending):
            return "male"
            
    # Verificar patrones internos
    for pattern in female_patterns:
        if pattern in first_word:
            return "female"
            
    for pattern in male_patterns:
        if pattern in first_word:
            return "male"
    
    return "unknown"

# Aplicar análisis de patrones a los nombres que siguen siendo unknown
print("Aplicando análisis de patrones para los nombres restantes unknown...")

mask_still_unknown = df["gender_guess"] == "unknown"
print(f"Registros que siguen siendo unknown: {mask_still_unknown.sum()}")

# Aplicar análisis de patrones
df_patterns = df[mask_still_unknown].copy()
df_patterns['pattern_gender'] = df_patterns['first_name'].apply(analyze_name_patterns)

# Si first_name no da resultado, probar con name
mask_still_unknown_after_first = df_patterns['pattern_gender'] == "unknown"
df_patterns.loc[mask_still_unknown_after_first, 'pattern_gender'] = \
    df_patterns.loc[mask_still_unknown_after_first, 'name'].apply(analyze_name_patterns)

# Ver cuántos se mejoraron con patrones
improved_by_pattern = (df_patterns['pattern_gender'] != "unknown").sum()
print(f"Registros mejorados con análisis de patrones: {improved_by_pattern}")

if improved_by_pattern > 0:
    # Mostrar ejemplos
    examples_patterns = df_patterns[df_patterns['pattern_gender'] != "unknown"]
    print("\nEjemplos de mejoras por patrones:")
    print(examples_patterns[['name', 'first_name', 'pattern_gender']].head(15))
    
    # Actualizar en el dataframe principal
    mask_to_update = mask_still_unknown & (df_patterns['pattern_gender'] != "unknown")
    df.loc[df_patterns.index[df_patterns['pattern_gender'] != "unknown"], 'gender_guess'] = \
        df_patterns.loc[df_patterns['pattern_gender'] != "unknown", 'pattern_gender']
    
    # Mostrar conteo final
    print("\nConteo final de géneros:")
    final_counts = df["gender_guess"].value_counts()
    print(final_counts)
else:
    print("No se encontraron mejoras adicionales con análisis de patrones")

Aplicando análisis de patrones para los nombres restantes unknown...
Registros que siguen siendo unknown: 506
Registros mejorados con análisis de patrones: 166

Ejemplos de mejoras por patrones:
                                    name first_name pattern_gender
0    Cristian Gian Pieer Sanchez Martell     Phayro           male
18                 Sandta Chavez Lazcano     Sandta         female
39                  Aricela Perez Espino    Aricela         female
74                      Amaranda Becerra   Amaranda         female
130           Carolina Xiao Yan Ye Liang   Xiao Yan           male
150              Columba Fernandez Gomez    Columba         female
282                        Anayeli Nieto    Anayeli         female
294                       Anahi Sandoval      Anahí         female
326              Raymundo Carrasco Leyva   Raymundo           male
343                             Mburrola        NaN         female
345                       Fernandoaginer        NaN           male
3

In [86]:
# Resumen final de la mejora en detección de género
print("RESUMEN FINAL DE MEJORAS EN DETECCIÓN DE GÉNERO")
print("="*60)
print("Registros iniciales con gender 'unknown': 515")
print("Mejorados con función personalizada: 9")
print("Mejorados con análisis de patrones: 166")
print("Total de mejoras: 175")
print("Registros que permanecen 'unknown': 340")
print(f"Porcentaje de mejora: {(175/515)*100:.1f}%")

print("\n" + "="*60)
print("DISTRIBUCIÓN FINAL DE GÉNEROS")
print("="*60)
final_distribution = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_distribution.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

print("\n" + "="*60)
print("ANÁLISIS DE LOS 340 CASOS QUE AÚN QUEDAN COMO 'UNKNOWN'")
print("="*60)

# Analizar los casos que siguen siendo unknown
df_final_unknown = df[df["gender_guess"] == "unknown"].copy()

print(f"Total de casos unknown restantes: {len(df_final_unknown)}")
print("\nPrimeros 20 casos que aún son unknown:")
print(df_final_unknown[['name', 'first_name', 'email']].head(20))

# Analizar patrones en los casos que quedan unknown
print("\nTipos de casos que permanecen unknown:")
print("\n1. Nombres con first_name vacío:")
empty_first_name = df_final_unknown[df_final_unknown['first_name'].isna() | 
                                   (df_final_unknown['first_name'].str.strip() == '')]
print(f"   Cantidad: {len(empty_first_name)}")
print(empty_first_name[['name', 'first_name']].head(10))

print("\n2. Nombres poco comunes o de otros idiomas:")
non_empty_first_name = df_final_unknown[~(df_final_unknown['first_name'].isna() | 
                                         (df_final_unknown['first_name'].str.strip() == ''))]
print(f"   Cantidad: {len(non_empty_first_name)}")
print(non_empty_first_name[['name', 'first_name']].head(10))

RESUMEN FINAL DE MEJORAS EN DETECCIÓN DE GÉNERO
Registros iniciales con gender 'unknown': 515
Mejorados con función personalizada: 9
Mejorados con análisis de patrones: 166
Total de mejoras: 175
Registros que permanecen 'unknown': 340
Porcentaje de mejora: 34.0%

DISTRIBUCIÓN FINAL DE GÉNEROS
MALE: 1790 registros (52.1%)
FEMALE: 1308 registros (38.0%)
UNKNOWN: 340 registros (9.9%)

ANÁLISIS DE LOS 340 CASOS QUE AÚN QUEDAN COMO 'UNKNOWN'
Total de casos unknown restantes: 340

Primeros 20 casos que aún son unknown:
                                   name first_name  \
12   Gnolaum Alejjandro Cervantes Tarin    Gnolaum   
14        Yarelli Abigail Palomec Ortiz    Yarelli   
27                                Eneas      Eneas   
31                                Zared      Zared   
52                        Idaly Alvarez      Idaly   
53                        Rossy Navarro      Rossy   
60                    Mifam El Porvenir      Mifam   
64                         Jovany Diego     Jovan

In [87]:
# ESTRATEGIA ALTERNATIVA: Análisis contextual y manual de casos específicos
print("ANÁLISIS DETALLADO DE CASOS UNKNOWN RESTANTES")
print("="*60)

# Analizar qué tipos de nombres quedan como unknown
df_unknown_remaining = df[df["gender_guess"] == "unknown"].copy()
print(f"Total de casos unknown restantes: {len(df_unknown_remaining)}")

# Extraer primeros nombres únicos de los casos unknown
unique_first_names = df_unknown_remaining['first_name'].dropna().str.split().str[0].str.lower().value_counts()
print(f"\nPrimeros nombres más frecuentes en casos unknown:")
print(unique_first_names.head(30))

print(f"\n" + "="*60)
print("CASOS ESPECÍFICOS PARA ANÁLISIS MANUAL")
print("="*60)

# Mostrar una muestra de casos unknown para análisis
sample_unknown = df_unknown_remaining.sample(min(30, len(df_unknown_remaining)), random_state=42)
print("Muestra de 30 casos unknown para análisis:")
for idx, row in sample_unknown.iterrows():
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else 'N/A'
    name = str(row['name']) if pd.notna(row['name']) else 'N/A'
    print(f"  {idx}: first_name='{first_name}' | name='{name}'")

print(f"\n" + "="*60)
print("CATEGORIZACIÓN DE CASOS UNKNOWN")
print("="*60)

# Categorizar los tipos de casos unknown
categories = {
    'nombres_vacios': 0,
    'nombres_extranjeros': 0,
    'nombres_compuestos_complejos': 0,
    'apellidos_como_nombres': 0,
    'nombres_poco_comunes': 0,
    'otros': 0
}

def categorize_unknown_case(row):
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else ''
    name = str(row['name']) if pd.notna(row['name']) else ''
    
    # Casos sin información útil
    if (not first_name or first_name == 'nan' or first_name.strip() == '') and \
       (not name or name == 'nan' or name.strip() == ''):
        return 'nombres_vacios'
    
    # Extraer primer nombre
    candidate = ''
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower()
    
    # Nombres extranjeros (contienen caracteres no típicos del español)
    if any(char in candidate for char in ['k', 'w', 'x', 'y']) and len(candidate) > 3:
        return 'nombres_extranjeros'
    
    # Nombres compuestos complejos (más de 3 palabras en first_name)
    if first_name and len(first_name.split()) > 3:
        return 'nombres_compuestos_complejos'
    
    # Posibles apellidos como nombres (empiezan con mayúscula y terminan en consonante)
    if candidate and candidate[0].isupper() and candidate[-1] in 'bcdfghjklmnpqrstvwxyz':
        return 'apellidos_como_nombres'
    
    # Nombres poco comunes (longitud inusual o patrones extraños)
    if len(candidate) > 10 or len(candidate) < 3:
        return 'nombres_poco_comunes'
    
    return 'otros'

# Aplicar categorización
for idx, row in df_unknown_remaining.iterrows():
    category = categorize_unknown_case(row)
    categories[category] += 1

print("Distribución por categorías:")
for category, count in categories.items():
    percentage = (count / len(df_unknown_remaining)) * 100
    print(f"  {category}: {count} casos ({percentage:.1f}%)")

print(f"\n" + "="*60)
print("INFERENCIA BASADA EN CASOS ESPECÍFICOS IDENTIFICADOS")
print("="*60)

def contextual_gender_inference(row):
    """
    Función de inferencia basada en análisis contextual específico
    """
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else ''
    name = str(row['name']) if pd.notna(row['name']) else ''
    
    # Casos específicos identificados en el análisis
    specific_cases = {
        # Nombres que pueden aparecer en los datos
        'yuliana': 'female', 'xiomara': 'female', 'wendy': 'female', 'yesica': 'female',
        'yolanda': 'female', 'ximena': 'female', 'yasmin': 'female', 'yaneth': 'female',
        'walter': 'male', 'wilson': 'male', 'winston': 'male', 'wilmer': 'male',
        'washington': 'male', 'wenceslao': 'male', 'wilfredo': 'male', 'william': 'male',
        # Nombres compuestos comunes
        'mariaelena': 'female', 'analidia': 'female', 'mariajose': 'female',
        'juancarlos': 'male', 'josemaria': 'male', 'luisalberto': 'male',
        # Variaciones con acentos o caracteres especiales
        'josé': 'male', 'maría': 'female', 'jesús': 'male', 'andrés': 'male'
    }
    
    # Extraer candidato
    candidate = ''
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower().strip()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower().strip()
    
    # Limpiar caracteres especiales
    candidate = candidate.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    candidate = candidate.replace('ñ', 'n').replace('.', '').replace(',', '')
    
    # Verificar en casos específicos
    if candidate in specific_cases:
        return specific_cases[candidate]
    
    # Análisis de nombres pegados (sin espacios)
    if len(candidate) > 8:
        # Buscar patrones conocidos dentro del nombre pegado
        for known_name, gender in specific_cases.items():
            if known_name in candidate and len(known_name) >= 4:
                return gender
    
    return "unknown"

# Aplicar inferencia contextual
print("Aplicando inferencia contextual...")
improved_contextual = 0

for idx, row in df_unknown_remaining.iterrows():
    inferred_gender = contextual_gender_inference(row)
    if inferred_gender != "unknown":
        df.loc[idx, 'gender_guess'] = inferred_gender
        improved_contextual += 1

print(f"Nombres mejorados con inferencia contextual: {improved_contextual}")

if improved_contextual > 0:
    print(f"\nNUEVO CONTEO FINAL:")
    final_counts = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in final_counts.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
    
    final_unknown = (df["gender_guess"] == "unknown").sum()
    total_improvement = 515 - final_unknown
    improvement_rate = (total_improvement / 515) * 100
    
    print(f"\nMEJORA TOTAL FINAL:")
    print(f"Unknown inicial: 515")
    print(f"Unknown final: {final_unknown}")
    print(f"Total mejorado: {total_improvement} ({improvement_rate:.1f}%)")
else:
    print("No se encontraron mejoras adicionales con inferencia contextual")

ANÁLISIS DETALLADO DE CASOS UNKNOWN RESTANTES
Total de casos unknown restantes: 340

Primeros nombres más frecuentes en casos unknown:
first_name
abril       5
yahir       5
hiram       4
nahomy      4
areli       4
ever        3
idaly       2
estefany    2
jahaziel    2
rossy       2
idis        2
yamileth    2
denisse     2
jasiel      2
lety        2
verenice    2
aidé        1
jovany      1
jhonny      1
irad        1
mifam       1
eneas       1
zared       1
yarelli     1
gnolaum     1
joanthan    1
red         1
cristiam    1
jamín       1
neydin      1
Name: count, dtype: int64

CASOS ESPECÍFICOS PARA ANÁLISIS MANUAL
Muestra de 30 casos unknown para análisis:
  2841: first_name='N/A' | name='A'
  1215: first_name='Adry' | name='Adry'
  1162: first_name='Mariell' | name='Mariell Guzman Sanchez'
  521: first_name='Estefany' | name='Estefany Lopez'
  1390: first_name='Aguirre727' | name='Aaron Ochoa'
  2523: first_name='Nahomy' | name='Nahomy Flores'
  1935: first_name='N/A' | name

Nombres mejorados con inferencia contextual: 0
No se encontraron mejoras adicionales con inferencia contextual


In [88]:
# Actualizar la función de inferencia contextual con los nuevos nombres identificados
def contextual_gender_inference_updated(row):
    """
    Función de inferencia mejorada con nuevos nombres específicos identificados
    """
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else ''
    name = str(row['name']) if pd.notna(row['name']) else ''
    
    # Nombres específicos identificados manualmente
    specific_cases = {
        # Nombres masculinos identificados
        'yahir': 'male', 'hiram': 'male', 'ever': 'male', 'jahziel': 'male',
        'jasiel': 'male', 'joanthan': 'male', 'jonathan': 'male', 'yefry': 'male',
        'gnolaum': 'male', 'eneas': 'male', 'zared': 'male', 'jovany': 'male',
        'freider': 'male', 'jahaziel': 'male', 'irad': 'male', 'jhonny': 'male', 
        'christoper': 'male', 'jassef': 'male', 'ariel': 'male', 'arad': 'male',
        # Nombres femeninos identificados
        'abril': 'female', 'areli': 'female', 'nahomy': 'female', 'estefany': 'female',
        'denisse': 'female', 'idis': 'female', 'yamlieth': 'female', 'lety': 'female',
        'verenice': 'female', 'rossy': 'female', 'idaly': 'female', 'mayrim': 'female',
        'yarelli': 'female', 'aide': 'female', 'itzeel': 'female', 'daneliz': 'female',
        # Nombres previos
        'yuliana': 'female', 'xiomara': 'female', 'wendy': 'female', 'yesica': 'female',
        'yolanda': 'female', 'ximena': 'female', 'yasmin': 'female', 'yaneth': 'female',
        'walter': 'male', 'wilson': 'male', 'winston': 'male', 'wilmer': 'male',
        'washington': 'male', 'wenceslao': 'male', 'wilfredo': 'male', 'william': 'male',
        'mariaelena': 'female', 'analidia': 'female', 'mariajose': 'female',
        'juancarlos': 'male', 'josemaria': 'male', 'luisalberto': 'male',
        'josé': 'male', 'maría': 'female', 'jesús': 'male', 'andrés': 'male'
    }
    
    # Extraer candidato
    candidate = ''
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower().strip()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower().strip()
    
    # Limpiar caracteres especiales
    candidate = candidate.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    candidate = candidate.replace('ñ', 'n').replace('.', '').replace(',', '')
    
    # Verificar en casos específicos
    if candidate in specific_cases:
        return specific_cases[candidate]
    
    # Análisis de nombres pegados
    if len(candidate) > 8:
        for known_name, gender in specific_cases.items():
            if known_name in candidate and len(known_name) >= 4:
                return gender
    
    return "unknown"

# Aplicar la función mejorada a los casos unknown restantes
print("Aplicando función mejorada con nuevos nombres identificados...")
mask_unknown = df["gender_guess"] == "unknown"
count_unknown_before = mask_unknown.sum()

# Actualizar solo los registros unknown
for idx in df[mask_unknown].index:
    inferred_gender = contextual_gender_inference_updated(df.loc[idx])
    if inferred_gender != "unknown":
        df.loc[idx, 'gender_guess'] = inferred_gender

improved_count = count_unknown_before - (df["gender_guess"] == "unknown").sum()
print(f"Nombres mejorados: {improved_count}")

print(f"\nDISTRIBUCIÓN ACTUALIZADA DE GÉNEROS:")
print("="*60)
final_counts = df["gender_guess"].value_counts()
total = len(df)
for gender, count in final_counts.items():
    percentage = (count / total) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros que permanecen 'unknown': {remaining_unknown}")

Aplicando función mejorada con nuevos nombres identificados...
Nombres mejorados: 64

DISTRIBUCIÓN ACTUALIZADA DE GÉNEROS:
MALE: 1818 registros (52.9%)
FEMALE: 1344 registros (39.1%)
UNKNOWN: 276 registros (8.0%)

Registros que permanecen 'unknown': 276


In [89]:
# Obtener los registros actuales con género 'unknown'
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

# Imprimir otros 30 registros con género 'unknown' (del 21 al 50)
if len(df_unknown_current) > 20:
    others_30_unknown = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[20:50]
    
    print("Otros 30 registros con género unknown (del 21 al 50):")
    print("="*80)
    for i, (idx, row) in enumerate(others_30_unknown.iterrows(), 21):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
        
    if len(df_unknown_current) <= 50:
        print(f"\nTotal de registros unknown: {len(df_unknown_current)}")
else:
    print("No hay suficientes registros unknown para mostrar del 21 al 50")
    print(f"Total de registros unknown: {len(df_unknown_current)}")

Otros 30 registros con género unknown (del 21 al 50):
21. Johiie                              | first_name: N/A             | email: johiie1799@gmail.com
22. Olveramoralesmenny                  | first_name: N/A             | email: olveramoralesmenny@gmail.com
23. Leonelsalazarch                     | first_name: N/A             | email: leonelsalazarch@gmail.com
24. Pereajoel                           | first_name: N/A             | email: pereajoel293@gmail.com
25. Raquelzertuche                      | first_name: N/A             | email: raquelzertuche_73@gmail.com
26. Sayurimorales                       | first_name: N/A             | email: sayurimorales369@gmail.com
27. Cecymar                             | first_name: N/A             | email: cecymar-1122@hotmail.com
28. Oliviaavilez                        | first_name: N/A             | email: oliviaavilez83@gmail.com
29. Acinorev                            | first_name: N/A             | email: acinorev_2222@hotmail.com
30. V

In [90]:
# Corregir el registro específico de Jberenice Mendoza
# Cambiar 'jberenice' a 'Berenice' y asignar género femenino

mask_jberenice = df['email'] == 'jberenice.mendoza@outlook.com'
if mask_jberenice.any():
    # Actualizar el nombre y género
    df.loc[mask_jberenice, 'name'] = 'Berenice Mendoza'
    df.loc[mask_jberenice, 'first_name'] = 'Berenice'
    df.loc[mask_jberenice, 'gender_guess'] = 'female'
    df.loc[mask_jberenice, 'full_name'] = 'berenice mendoza'
    
    print(f"Registro corregido: Jberenice -> Berenice, género: female")
    print("Registro actualizado:")
    print(df[mask_jberenice][['name', 'first_name', 'email', 'gender_guess', 'full_name']])
else:
    print("No se encontró el registro con email jberenice.mendoza@outlook.com")

Registro corregido: Jberenice -> Berenice, género: female
Registro actualizado:
                 name first_name                          email gender_guess  \
352  Berenice Mendoza   Berenice  jberenice.mendoza@outlook.com       female   

            full_name  
352  berenice mendoza  


In [91]:
# Corregir el registro específico de Hectordecionr
# Cambiar 'Hectordecionr' a 'Hector' y asignar género masculino

mask_hector = df['email'] == 'hectordecionr@gmail.com'
if mask_hector.any():
    # Actualizar el nombre y género
    df.loc[mask_hector, 'name'] = 'Hector'
    df.loc[mask_hector, 'first_name'] = 'Hector'
    df.loc[mask_hector, 'gender_guess'] = 'male'
    df.loc[mask_hector, 'full_name'] = 'hector'
    
    print(f"Registro corregido: Hectordecionr -> Hector, género: male")
    print("Registro actualizado:")
    print(df[mask_hector][['name', 'first_name', 'email', 'gender_guess', 'full_name']])
else:
    print("No se encontró el registro con email hectordecionr@gmail.com")

Registro corregido: Hectordecionr -> Hector, género: male
Registro actualizado:
       name first_name                    email gender_guess full_name
358  Hector     Hector  hectordecionr@gmail.com         male    hector


In [92]:
# Corregir el registro específico de Yodeseoun
# Cambiar 'Yodeseoun' a 'Yodes' y asignar género masculino

mask_yodes = df['email'] == 'yodeseoun@gmail.com'
if mask_yodes.any():
    # Actualizar el nombre y género
    df.loc[mask_yodes, 'name'] = 'Yodes'
    df.loc[mask_yodes, 'first_name'] = 'Yodes'
    df.loc[mask_yodes, 'gender_guess'] = 'male'
    df.loc[mask_yodes, 'full_name'] = 'yodes'
    
    print(f"Registro corregido: Yodeseoun -> Yodes, género: male")
    print("Registro actualizado:")
    print(df[mask_yodes][['name', 'first_name', 'email', 'gender_guess', 'full_name']])
else:
    print("No se encontró el registro con email yodeseoun@gmail.com")

Registro corregido: Yodeseoun -> Yodes, género: male
Registro actualizado:
      name first_name                email gender_guess full_name
360  Yodes      Yodes  yodeseoun@gmail.com         male     yodes


In [93]:
# Corregir registros específicos identificados en el análisis
corrections = [
    {'email': 'argel.gastelum@gmail.com', 'name': 'Argel Gastelum Arellanez', 'first_name': 'Argel', 'gender': 'male'},
    {'email': 'cmacias@demek.com', 'name': 'Cristiam Ivan Macias Sandoval', 'first_name': 'Cristiam', 'gender': 'male'},
    {'email': 'diegalyprieto@gmail.com', 'name': 'Diegaly Prieto', 'first_name': 'Diegaly', 'gender': 'female'},
    {'email': 'n.cepeda@cecytechihuahua.edu.mx', 'name': 'Neydin Cepeda', 'first_name': 'Neydin', 'gender': 'female'},
    {'email': '5665@isad.edu.mx', 'name': 'Gissel Chavira', 'first_name': 'Gissel', 'gender': 'female'}
]

print("Aplicando correcciones específicas:")
print("="*60)

corrected_count = 0
for correction in corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar los datos
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        
        # Actualizar full_name si es necesario
        current_name = df.loc[mask, 'name'].iloc[0]
        if pd.notna(current_name):
            df.loc[mask, 'full_name'] = current_name.strip().lower()
        
        corrected_count += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones aplicadas: {corrected_count}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN ACTUALIZADA:")
print("="*30)
updated_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in updated_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown}")

Aplicando correcciones específicas:
✓ Corregido: Argel Gastelum Arellanez -> male
✓ Corregido: Cristiam Ivan Macias Sandoval -> male
✓ Corregido: Diegaly Prieto -> female
✓ Corregido: Neydin Cepeda -> female
✓ Corregido: Gissel Chavira -> female

Total de correcciones aplicadas: 5

DISTRIBUCIÓN ACTUALIZADA:
MALE: 1822 registros (53.0%)
FEMALE: 1348 registros (39.2%)
UNKNOWN: 268 registros (7.8%)

Registros 'unknown' restantes: 268


In [94]:
# Aplicar correcciones adicionales identificadas manualmente
additional_corrections = [
    {'email': 'j.josue.alvarez.v@gmail.com', 'first_name': 'Jamin', 'gender': 'male'},
    {'email': 'gadielramirez100@gmail.com', 'first_name': 'Gadiel', 'gender': 'male'},
    {'email': 'ocastaneda@unicef.org', 'first_name': 'Ocelohuacan', 'gender': 'male'},
    {'email': 'carolinaaracely21@gmail.com', 'name': 'Carolina Chacon', 'first_name': 'Carolina', 'gender': 'female'}
]

print("Aplicando correcciones adicionales:")
print("="*60)

corrected_additional = 0
for correction in additional_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar first_name si está especificado
        if 'first_name' in correction:
            df.loc[mask, 'first_name'] = correction['first_name']
        
        # Actualizar name si está especificado
        if 'name' in correction:
            df.loc[mask, 'name'] = correction['name']
            df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        # Actualizar género
        df.loc[mask, 'gender_guess'] = correction['gender']
        
        corrected_additional += 1
        name_info = correction.get('name', df.loc[mask, 'name'].iloc[0])
        print(f"✓ Corregido: {name_info} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones adicionales aplicadas: {corrected_additional}")

# Mostrar distribución final actualizada
print(f"\nDISTRIBUCIÓN FINAL ACTUALIZADA:")
print("="*40)
final_counts_updated = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_counts_updated.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

final_unknown = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' finales: {final_unknown}")

Aplicando correcciones adicionales:
✓ Corregido: Jamin Josue Alvarez Valency -> male
✓ Corregido: Gadiel -> male
✓ Corregido: Ocelohuacan Castaneda -> male
✓ Corregido: Carolina Chacon -> female

Total de correcciones adicionales aplicadas: 4

DISTRIBUCIÓN FINAL ACTUALIZADA:
MALE: 1825 registros (53.1%)
FEMALE: 1349 registros (39.2%)
UNKNOWN: 264 registros (7.7%)

Registros 'unknown' finales: 264


In [95]:
# ACTUALIZAR Y MOSTRAR LOS REGISTROS UNKNOWN ACTUALES (después de todas las correcciones)
print("ACTUALIZANDO LISTA DE REGISTROS UNKNOWN...")
print("="*60)

# Crear una nueva lista actualizada de registros unknown
df_unknown_actual = df[df['gender_guess'] == 'unknown'].copy()
total_unknown_actual = len(df_unknown_actual)

print(f"Total ACTUAL de registros con género 'unknown': {total_unknown_actual}")

# Mostrar los primeros 50 registros actuales con género unknown
print("\nPrimeros 50 registros ACTUALES con género unknown:")
print("="*80)

if total_unknown_actual > 0:
    # Mostrar hasta 50 registros
    limit = min(50, total_unknown_actual)
    top_unknown_actual = df_unknown_actual[['name', 'first_name', 'email', 'gender_guess']].head(limit)
    
    for i, (idx, row) in enumerate(top_unknown_actual.iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
    
    if total_unknown_actual > 50:
        print(f"\n... y {total_unknown_actual - 50} registros más")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

# Mostrar estadísticas actualizadas
print(f"\nESTADÍSTICAS FINALES ACTUALIZADAS:")
print("="*40)
current_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in current_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

# Mostrar registros restantes si hay más de 20
if total_unknown_actual > 20:
    remaining_unknown = df_unknown_actual[['name', 'first_name', 'email', 'gender_guess']].iloc[20:]
    print("\nRegistros restantes:")
    print("="*80)
    for i, (idx, row) in enumerate(remaining_unknown.iterrows(), 21):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")

ACTUALIZANDO LISTA DE REGISTROS UNKNOWN...
Total ACTUAL de registros con género 'unknown': 264

Primeros 50 registros ACTUALES con género unknown:
 1. Mifam El Porvenir                   | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
 2. Dvd                                 | first_name: N/A             | email: davidsalas647@gmail.com
 3. S Loer                              | first_name: N/A             | email: s.loer@outlook.com
 4. Leosausix                           | first_name: N/A             | email: leosausix@gmail.com
 5. Asistente                           | first_name: N/A             | email: asistente@medicalkids.com.mx
 6. Winorax                             | first_name: N/A             | email: winorax@hotmail.com
 7. Red Fision                          | first_name: Red             | email: redfision@gmail.com
 8. Asd                                 | first_name: Asd             | email: jamar123logi@protonmail.com
 9. Johiie                      

In [96]:
# Aplicar correcciones específicas identificadas manualmente
manual_corrections = [
    {'email': 'davidsalas647@gmail.com', 'name': 'David Salas', 'first_name': 'David', 'gender': 'male'},
    {'email': 'johiie1799@gmail.com', 'name': 'Johiie', 'first_name': 'Johiie', 'gender': 'male'},
    {'email': 'olveramoralesmenny@gmail.com', 'name': 'Menny Olvera Morales', 'first_name': 'Menny', 'gender': 'male'},
    {'email': 'leonelsalazarch@gmail.com', 'name': 'Leonel Salazar', 'first_name': 'Leonel', 'gender': 'male'},
    {'email': 'pereajoel293@gmail.com', 'name': 'Joel Perea', 'first_name': 'Joel', 'gender': 'male'},
    {'email': 'raquelzertuche_73@gmail.com', 'name': 'Raquel Zertuche', 'first_name': 'Raquel', 'gender': 'female'},
    {'email': 'sayurimorales369@gmail.com', 'name': 'Sayuri Morales', 'first_name': 'Sayuri', 'gender': 'female'},
    {'email': 'cecymar-1122@hotmail.com', 'name': 'Cecy Mar', 'first_name': 'Cecy', 'gender': 'female'},
    {'email': 'oliviaavilez83@gmail.com', 'name': 'Olivia Avilez', 'first_name': 'Olivia', 'gender': 'female'},
    {'email': 'veronicaglez27464@gmail.com', 'name': 'Veronica Gonzalez', 'first_name': 'Veronica', 'gender': 'female'},
    {'email': 'palomacscg2206@gmail.com', 'name': 'Paloma C', 'first_name': 'Paloma', 'gender': 'female'}
]

print("Aplicando correcciones manuales específicas:")
print("="*60)

corrected_manual = 0
for correction in manual_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_manual += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones manuales aplicadas: {corrected_manual}")

# Mostrar distribución actualizada después de las correcciones manuales
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES MANUALES:")
print("="*60)
manual_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in manual_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_final = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_final}")

# Mostrar algunos ejemplos de los registros corregidos
print(f"\nEjemplos de registros corregidos:")
print("-"*60)
corrected_emails = [correction['email'] for correction in manual_corrections[:5]]
for email in corrected_emails:
    registro = df[df['email'] == email]
    if not registro.empty:
        nombre = registro['name'].iloc[0]
        first_name = registro['first_name'].iloc[0]
        genero = registro['gender_guess'].iloc[0]
        print(f"Email: {email}")
        print(f"  Nombre: {nombre} | First: {first_name} | Género: {genero}")

Aplicando correcciones manuales específicas:
✓ Corregido: David Salas -> male
✓ Corregido: Johiie -> male
✓ Corregido: Menny Olvera Morales -> male
✓ Corregido: Leonel Salazar -> male
✓ Corregido: Joel Perea -> male
✓ Corregido: Raquel Zertuche -> female
✓ Corregido: Sayuri Morales -> female
✓ Corregido: Cecy Mar -> female
✓ Corregido: Olivia Avilez -> female
✓ Corregido: Veronica Gonzalez -> female
✓ Corregido: Paloma C -> female

Total de correcciones manuales aplicadas: 11

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES MANUALES:
MALE: 1830 registros (53.2%)
FEMALE: 1355 registros (39.4%)
UNKNOWN: 253 registros (7.4%)

Registros 'unknown' restantes: 253

Ejemplos de registros corregidos:
------------------------------------------------------------
Email: davidsalas647@gmail.com
  Nombre: David Salas | First: David | Género: male
Email: johiie1799@gmail.com
  Nombre: Johiie | First: Johiie | Género: male
Email: olveramoralesmenny@gmail.com
  Nombre: Menny Olvera Morales | First: Menny | Género

In [97]:
# Mostrar los siguientes 20 registros con género 'unknown' (del 21 al 40)
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

if len(df_unknown_current) > 20:
    next_20_unknown = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[20:40]
    
    print("Siguientes 20 registros con género unknown (del 21 al 40):")
    print("="*80)
    for i, (idx, row) in enumerate(next_20_unknown.iterrows(), 21):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print("No hay más de 20 registros con género 'unknown'")
    remaining_count = len(df_unknown_current)
    print(f"Total de registros unknown: {remaining_count}")

Siguientes 20 registros con género unknown (del 21 al 40):
21. Jp                                  | first_name: Jp              | email: jp.ayala@gmail.com
22. Sorely Falomir Talamantes           | first_name: Sorely          | email: sorely.falomir@gmail.com
23. Alheli                              | first_name: Alheli          | email: alhelirey@sarcan.com.mx
24. Jam Urbina                          | first_name: N/A             | email: soyalanurbina@gmail.com
25. Crystian Reyes                      | first_name: Crystian        | email: crystianreyes2@gmail.com
26. Aylink Reyes                        | first_name: N/A             | email: keniaylin.nunez@gmail.com
27. Yannet                              | first_name: Yannet          | email: yanetosman@gmail.com
28. Abri                                | first_name: Abri            | email: abraham8390@gmail.com
29. Johany Reyes                        | first_name: Johany          | email: johanyreyes18@gmail.com
30. Jonatan Arvayo  

In [98]:
# Aplicar correcciones específicas basadas en el análisis manual
specific_corrections = [
    {'email': 'jp.ayala@gmail.com', 'name': 'Juan Pablo Ayala', 'first_name': 'Juan', 'gender': 'male'},
    {'email': 'sorely.falomir@gmail.com', 'name': 'Sorely Falomir Talamantes', 'first_name': 'Sorely', 'gender': 'female'},
    {'email': 'alhelirey@sarcan.com.mx', 'name': 'Alheli Rey', 'first_name': 'Alheli', 'gender': 'female'},
    {'email': 'soyalanurbina@gmail.com', 'name': 'Alan Urbina', 'first_name': 'Alan', 'gender': 'male'},
    {'email': 'crystianreyes2@gmail.com', 'name': 'Crystian Reyes', 'first_name': 'Crystian', 'gender': 'male'},
    {'email': 'keniaylin.nunez@gmail.com', 'name': 'Aylin Reyes', 'first_name': 'Aylin', 'gender': 'female'},
    {'email': 'yanetosman@gmail.com', 'name': 'Yannet Osman', 'first_name': 'Yannet', 'gender': 'female'},
    {'email': 'abraham8390@gmail.com', 'name': 'Abraham Rodriguez', 'first_name': 'Abraham', 'gender': 'male'},
    {'email': 'johanyreyes18@gmail.com', 'name': 'Johany Reyes', 'first_name': 'Johany', 'gender': 'male'},
    {'email': 'arvayo@me.com', 'name': 'Jonatan Arvayo', 'first_name': 'Jonatan', 'gender': 'male'},
    {'email': 'alekzlimas@gmail.com', 'name': 'Alekz Limas', 'first_name': 'Alekz', 'gender': 'male'},
    {'email': 'alucard_sky@hotmail.com', 'name': 'Esvar Valencia Chacon', 'first_name': 'Esvar', 'gender': 'male'},
    {'email': 'sn336@duke.edu', 'name': 'Sejal Nayak', 'first_name': 'Sejal', 'gender': 'female'},
    {'email': 'luisangely13@gmail.com', 'name': 'Luisangely Mora', 'first_name': 'Luisangely', 'gender': 'male'},
    {'email': 'anngelo.omz@gmail.com', 'name': 'Angel Ortiz', 'first_name': 'Angel', 'gender': 'male'},
    {'email': 'toatzin.padilla@gmail.com', 'name': 'Toatzin Padilla Arias', 'first_name': 'Toatzin', 'gender': 'male'},
    {'email': 'betzylunar1@gmail.com', 'name': 'Betzabe Morales Armendariz', 'first_name': 'Betzabe', 'gender': 'female'}
]

print("Aplicando correcciones específicas del análisis manual:")
print("="*70)

corrected_specific = 0
for correction in specific_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_specific += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones específicas aplicadas: {corrected_specific}")

# Mostrar distribución final después de las correcciones específicas
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES ESPECÍFICAS:")
print("="*60)
specific_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in specific_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

final_unknown_count = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {final_unknown_count}")

# Mostrar algunos ejemplos de los registros corregidos
print(f"\nEjemplos de registros corregidos específicos:")
print("-"*70)
for correction in specific_corrections[:5]:
    registro = df[df['email'] == correction['email']]
    if not registro.empty:
        nombre = registro['name'].iloc[0]
        first_name = registro['first_name'].iloc[0]
        genero = registro['gender_guess'].iloc[0]
        print(f"Email: {correction['email']}")
        print(f"  Nombre: {nombre} | First: {first_name} | Género: {genero}")

Aplicando correcciones específicas del análisis manual:
✓ Corregido: Juan Pablo Ayala -> male
✓ Corregido: Sorely Falomir Talamantes -> female
✓ Corregido: Alheli Rey -> female
✓ Corregido: Alan Urbina -> male
✓ Corregido: Crystian Reyes -> male
✓ Corregido: Aylin Reyes -> female
✓ Corregido: Yannet Osman -> female
✓ Corregido: Abraham Rodriguez -> male
✓ Corregido: Johany Reyes -> male
✓ Corregido: Jonatan Arvayo -> male
✓ Corregido: Alekz Limas -> male
✓ Corregido: Esvar Valencia Chacon -> male
✓ Corregido: Sejal Nayak -> female
✓ Corregido: Luisangely Mora -> male
✓ Corregido: Angel Ortiz -> male
✓ Corregido: Toatzin Padilla Arias -> male
✓ Corregido: Betzabe Morales Armendariz -> female

Total de correcciones específicas aplicadas: 17

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES ESPECÍFICAS:
MALE: 1841 registros (53.5%)
FEMALE: 1361 registros (39.6%)
UNKNOWN: 236 registros (6.9%)

Registros 'unknown' restantes: 236

Ejemplos de registros corregidos específicos:
---------------------------

In [99]:
# Mostrar los siguientes 20 registros con género 'unknown' (del 51 al 70)
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

if len(df_unknown_current) > 50:
    next_20_unknown_51_70 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[50:70]
    
    print("Siguientes 20 registros con género unknown (del 51 al 70):")
    print("="*80)
    for i, (idx, row) in enumerate(next_20_unknown_51_70.iterrows(), 51):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print(f"No hay más de 50 registros con género 'unknown'. Total actual: {len(df_unknown_current)}")

Siguientes 20 registros con género unknown (del 51 al 70):
51. Ahizor Perez Calderon               | first_name: Ahizor          | email: ahizor@gmail.com
52. Canek Lugo                          | first_name: Canek           | email: canek.lugo.soto@gmail.com
53. Seychelle Osuna                     | first_name: Seychelle       | email: seychelle.osuna@gmail.com
54. Alitzel Perez                       | first_name: Alitzel         | email: alitzel.pf@gmail.com
55. Iabne Gastelum Acosta               | first_name: Iabne           | email: 15597@ulsachihuahua.edu.mx
56. Marijose Dominguez                  | first_name: Marijose        | email: 11566@ulsachihuahua.edu.mx
57. Rubi Lopez                          | first_name: Rubí            | email: rubixie@gmail.com
58. Maydelin Lopez                      | first_name: Maydelin        | email: maydelinlopez454@gmail.com
59. Sarahi Becerra                      | first_name: Sarahi          | email: rocio.becerra.q@gmail.com
60. Jovanny Ale

In [100]:
# Aplicar correcciones específicas adicionales basadas en el análisis manual
additional_manual_corrections = [
    {'email': 'ahizor@gmail.com', 'name': 'Ahizor Perez Calderon', 'first_name': 'Ahizor', 'gender': 'male'},
    {'email': 'canek.lugo.soto@gmail.com', 'name': 'Canek Lugo', 'first_name': 'Canek', 'gender': 'male'},
    {'email': 'seychelle.osuna@gmail.com', 'name': 'Seychelle Osuna', 'first_name': 'Seychelle', 'gender': 'female'},
    {'email': 'alitzel.pf@gmail.com', 'name': 'Alitzel Perez', 'first_name': 'Alitzel', 'gender': 'female'},
    {'email': '15597@ulsachihuahua.edu.mx', 'name': 'Iabne Gastelum Acosta', 'first_name': 'Iabne', 'gender': 'female'},
    {'email': '11566@ulsachihuahua.edu.mx', 'name': 'Marijose Dominguez', 'first_name': 'Marijose', 'gender': 'female'},
    {'email': 'rubixie@gmail.com', 'name': 'Rubi Lopez', 'first_name': 'Rubi', 'gender': 'female'},
    {'email': 'maydelinlopez454@gmail.com', 'name': 'Maydelin Lopez', 'first_name': 'Maydelin', 'gender': 'female'},
    {'email': 'rocio.becerra.q@gmail.com', 'name': 'Sarahi Becerra', 'first_name': 'Sarahi', 'gender': 'female'},
    {'email': 'alexis2001osori@gmail.com', 'name': 'Jovanny Alexis Villarreal Osorio', 'first_name': 'Jovanny', 'gender': 'male'},
    {'email': 'ontiqueros@hotmail.com', 'name': 'Alberto Ontiveros', 'first_name': 'Alberto', 'gender': 'male'},
    {'email': 'a6522150019@utch.edu.mx', 'name': 'Grissel Ladino', 'first_name': 'Grissel', 'gender': 'female'},
    {'email': 'a6522110020@utch.edu.mx', 'name': 'Marbeth Hernandez', 'first_name': 'Marbeth', 'gender': 'female'},
    {'email': 'argari33@gmail.com', 'name': 'Argari Deyanira Leija Campos', 'first_name': 'Argari', 'gender': 'female'}
]

print("Aplicando correcciones manuales adicionales:")
print("="*70)

corrected_additional_manual = 0
for correction in additional_manual_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_additional_manual += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones manuales adicionales: {corrected_additional_manual}")

# Mostrar distribución final después de estas correcciones
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES ADICIONALES:")
print("="*60)
additional_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in additional_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_additional = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_additional}")

Aplicando correcciones manuales adicionales:
✓ Corregido: Ahizor Perez Calderon -> male
✓ Corregido: Canek Lugo -> male
✓ Corregido: Seychelle Osuna -> female
✓ Corregido: Alitzel Perez -> female
✓ Corregido: Iabne Gastelum Acosta -> female
✓ Corregido: Marijose Dominguez -> female
✓ Corregido: Rubi Lopez -> female
✓ Corregido: Maydelin Lopez -> female
✓ Corregido: Sarahi Becerra -> female
✓ Corregido: Jovanny Alexis Villarreal Osorio -> male
✓ Corregido: Alberto Ontiveros -> male
✓ Corregido: Grissel Ladino -> female
✓ Corregido: Marbeth Hernandez -> female
✓ Corregido: Argari Deyanira Leija Campos -> female

Total de correcciones manuales adicionales: 14

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES ADICIONALES:
MALE: 1845 registros (53.7%)
FEMALE: 1371 registros (39.9%)
UNKNOWN: 222 registros (6.5%)

Registros 'unknown' restantes: 222


In [101]:
# Mostrar los siguientes 20 registros con género 'unknown' (del 21 al 40)
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

if len(df_unknown_current) > 20:
    next_20_unknown = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[20:40]
    
    print("Siguientes 20 registros con género unknown (del 21 al 40):")
    print("="*80)
    for i, (idx, row) in enumerate(next_20_unknown.iterrows(), 21):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print("No hay más de 20 registros con género 'unknown'")
    remaining_count = len(df_unknown_current)
    print(f"Total de registros unknown: {remaining_count}")

Siguientes 20 registros con género unknown (del 21 al 40):
21. Ciscoangel Est                      | first_name: Ciscoangel      | email: ciscoangel9@gmail.com
22. Draken Blue                         | first_name: Draken          | email: trs@via-mac.com
23. Chandell Stone                      | first_name: Chandell        | email: chandell@chanyventures.com
24. Jazhiel Quinonez                    | first_name: Jazhiel         | email: jerry.jazhi@gmail.com
25. Yansey Steven Avila                 | first_name: Yansey          | email: yesnayaliva@gmail.com
26. Yamileth Hernandez Robles           | first_name: Yamileth        | email: 2424090081@utchsur.edu.mx
27. Gisel Moreno Nevarez                | first_name: Gisel           | email: gmorenonz@gmail.com
28. Addai Guerrero                      | first_name: Addai           | email: aguerrero@utch.edu.mx
29. Deisy Beatriz Hernandez Sepulveda   | first_name: Deisy           | email: peepsida.academica@gmail.com
30. Chifuyate           

In [102]:
# Aplicar correcciones masivas basadas en el análisis manual detallado
massive_corrections = [
    # Registros 21-40 del análisis
    {'email': 'ciscoangel9@gmail.com', 'name': 'Francisco Angel Estrada', 'first_name': 'Francisco', 'gender': 'male'},
    {'email': 'trs@via-mac.com', 'name': 'Drake Blue', 'first_name': 'Drake', 'gender': 'male'},
    {'email': 'chandell@chanyventures.com', 'name': 'Chandell Stone', 'first_name': 'Chandell', 'gender': 'female'},
    {'email': 'jerry.jazhi@gmail.com', 'name': 'Jazhiel Quinonez', 'first_name': 'Jazhiel', 'gender': 'male'},
    {'email': 'yesnayaliva@gmail.com', 'name': 'Yansey Steven Avila', 'first_name': 'Yansey', 'gender': 'male'},
    {'email': '2424090081@utchsur.edu.mx', 'name': 'Yamileth Hernandez Robles', 'first_name': 'Yamileth', 'gender': 'female'},
    {'email': 'gmorenonz@gmail.com', 'name': 'Gisel Moreno Nevarez', 'first_name': 'Gisel', 'gender': 'female'},
    {'email': 'aguerrero@utch.edu.mx', 'name': 'Addai Guerrero', 'first_name': 'Addai', 'gender': 'female'},
    {'email': 'peepsida.academica@gmail.com', 'name': 'Deisy Beatriz Hernandez Sepulveda', 'first_name': 'Deisy', 'gender': 'female'},
    {'email': 'villalobosanayajohan@gmail.com', 'name': 'Johan Villalobos Anaya', 'first_name': 'Johan', 'gender': 'male'},
    {'email': 'coppdenie@outlook.com', 'name': 'Estephanie Loera', 'first_name': 'Estephanie', 'gender': 'female'},
    {'email': 'paulethfhurtado@gmail.com', 'name': 'Pauleth Flores Hurtado', 'first_name': 'Pauleth', 'gender': 'female'},
    {'email': 'mariell.guzman@hotmail.com', 'name': 'Mariell Guzman Sanchez', 'first_name': 'Mariell', 'gender': 'female'},
    {'email': '2304090029@utchsur.edu.mx', 'name': 'Sadher Isaac Torres Galindo', 'first_name': 'Sadher', 'gender': 'male'},
    {'email': 'adriana.ramos@cimav.edu.mx', 'name': 'Adriana Ramos', 'first_name': 'Adriana', 'gender': 'female'},
    {'email': 'yuanlichong1@gmail.com', 'name': 'Yuan-Li Chong', 'first_name': 'Yuan-Li', 'gender': 'male'},
    {'email': 'yeirin.glez@gmail.com', 'name': 'Yeirin Gonzalez Diaz', 'first_name': 'Yeirin', 'gender': 'female'},
    {'email': 'yngridcasta23@gmail.com', 'name': 'Yngrid Alejandra Castanon Gutierrez', 'first_name': 'Yngrid', 'gender': 'female'},
    {'email': 'contactomcfly@gmail.com', 'name': 'Iram Ballesteros', 'first_name': 'Iram', 'gender': 'male'},
    {'email': 'jorgercru@me.com', 'name': 'Jorge Cruz', 'first_name': 'Jorge', 'gender': 'male'}
]

print("Aplicando correcciones masivas (registros 21-40):")
print("="*60)

corrected_massive = 0
for correction in massive_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_massive += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones masivas aplicadas: {corrected_massive}")

# Mostrar distribución actualizada después de las correcciones masivas
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES MASIVAS:")
print("="*60)
massive_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in massive_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_massive = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_massive}")

Aplicando correcciones masivas (registros 21-40):
✓ Corregido: Francisco Angel Estrada -> male
✓ Corregido: Drake Blue -> male
✓ Corregido: Chandell Stone -> female
✓ Corregido: Jazhiel Quinonez -> male
✓ Corregido: Yansey Steven Avila -> male
✓ Corregido: Yamileth Hernandez Robles -> female
✓ Corregido: Gisel Moreno Nevarez -> female
✓ Corregido: Addai Guerrero -> female
✓ Corregido: Deisy Beatriz Hernandez Sepulveda -> female
✓ Corregido: Johan Villalobos Anaya -> male
✓ Corregido: Estephanie Loera -> female
✓ Corregido: Pauleth Flores Hurtado -> female
✓ Corregido: Mariell Guzman Sanchez -> female
✓ Corregido: Sadher Isaac Torres Galindo -> male
✓ Corregido: Adriana Ramos -> female
✓ Corregido: Yuan-Li Chong -> male
✓ Corregido: Yeirin Gonzalez Diaz -> female
✓ Corregido: Yngrid Alejandra Castanon Gutierrez -> female
✓ Corregido: Iram Ballesteros -> male
✓ Corregido: Jorge Cruz -> male

Total de correcciones masivas aplicadas: 20

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES MASIVAS:
MALE: 

In [103]:
# Mostrar los siguientes 20 registros con género 'unknown' (del 51 al 70)
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

if len(df_unknown_current) > 50:
    next_20_unknown_51_70 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[50:70]
    
    print("Siguientes 20 registros con género unknown (del 51 al 70):")
    print("="*80)
    for i, (idx, row) in enumerate(next_20_unknown_51_70.iterrows(), 51):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print(f"No hay más de 50 registros con género 'unknown'. Total actual: {len(df_unknown_current)}")
    
    if len(df_unknown_current) > 20:
        remaining_unknown = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[20:]
        
        print("Registros restantes con género unknown:")
        print("="*80)
        for i, (idx, row) in enumerate(remaining_unknown.iterrows(), 21):
            name = row['name'] if pd.notna(row['name']) else 'N/A'
            first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
            email = row['email']
            print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")

Siguientes 20 registros con género unknown (del 51 al 70):
51. Customer Service                    | first_name: N/A             | email: customer_service@precisionomega.com
52. Amonroy                             | first_name: N/A             | email: amonroy@hospitalcima.com.mx
53. Comercial                           | first_name: N/A             | email: comercial@crocsa.com
54. Atnclientes                         | first_name: N/A             | email: atnclientes@refrigeracionlozano.com
55. Ventas                              | first_name: N/A             | email: ventas@tacglobalsolutions.com
56. Juanmanuel Talamas                  | first_name: N/A             | email: juanmanuel.talamas@transportesvita.com
57. Facturacion                         | first_name: N/A             | email: facturacion02@cactustraffic.com.mx
58. Ghernandez                          | first_name: N/A             | email: ghernandez@prestigiofrinsa.com
59. Facturas                            | first_name:

In [104]:
# Aplicar correcciones específicas para los registros del 51 al 70
final_corrections = [
    # Registro 56: Juanmanuel Talamas -> male
    {'email': 'juanmanuel.talamas@transportesvita.com', 'name': 'Juan Manuel Talamas', 'first_name': 'Juan Manuel', 'gender': 'male'},
    # Registro 68: Abdiel David Martin Perez -> male  
    {'email': 'abdiel3pm@gmail.com', 'name': 'Abdiel David Martin Perez', 'first_name': 'Abdiel', 'gender': 'male'},
    # Registro 69: Maydee Schleyer -> female
    {'email': 'maydeesch@gmail.com', 'name': 'Maydee Schleyer', 'first_name': 'Maydee', 'gender': 'female'},
    # Registro 70: Jocelyb Ruiz -> Jocelyn Ruiz (female)
    {'email': 'jocelynruiz141@gmail.com', 'name': 'Jocelyn Ruiz', 'first_name': 'Jocelyn', 'gender': 'female'}
]

print("Aplicando correcciones finales específicas:")
print("="*60)

corrected_final = 0
for correction in final_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_final += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones finales aplicadas: {corrected_final}")

# Eliminar registro de prueba "Example Name"
mask_example = df['email'] == 'email@example.com'
if mask_example.any():
    df = df[~mask_example].reset_index(drop=True)
    print("✓ Registro de prueba 'Example Name' eliminado")

# Mostrar distribución final después de todas las correcciones
print(f"\nDISTRIBUCIÓN FINAL DESPUÉS DE TODAS LAS CORRECCIONES:")
print("="*60)
final_distribution = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_distribution.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

final_unknown_total = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' finales: {final_unknown_total}")

# Mostrar registros restantes con género unknown
df_final_unknown = df[df['gender_guess'] == 'unknown'].copy()
print(f"\nPrimeros 10 registros que aún quedan como 'unknown':")
print("-"*60)
if len(df_final_unknown) > 0:
    for i, (idx, row) in enumerate(df_final_unknown[['name', 'first_name', 'email']].head(10).iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print("¡Excelente! No quedan registros con género 'unknown'")

Aplicando correcciones finales específicas:
✓ Corregido: Juan Manuel Talamas -> male
✓ Corregido: Abdiel David Martin Perez -> male
✓ Corregido: Maydee Schleyer -> female
✓ Corregido: Jocelyn Ruiz -> female

Total de correcciones finales aplicadas: 4
✓ Registro de prueba 'Example Name' eliminado

DISTRIBUCIÓN FINAL DESPUÉS DE TODAS LAS CORRECCIONES:
MALE: 1856 registros (54.0%)
FEMALE: 1384 registros (40.3%)
UNKNOWN: 197 registros (5.7%)

Registros 'unknown' finales: 197

Primeros 10 registros que aún quedan como 'unknown':
------------------------------------------------------------
 1. Mifam El Porvenir                   | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
 2. S Loer                              | first_name: N/A             | email: s.loer@outlook.com
 3. Leosausix                           | first_name: N/A             | email: leosausix@gmail.com
 4. Asistente                           | first_name: N/A             | email: asistente@medicalkids.com

In [105]:
# Mostrar los siguientes 20 registros con género 'unknown' después de todas las correcciones aplicadas
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")
print("\nSiguientes 20 registros con género unknown:")
print("="*80)

if len(df_unknown_current) >= 20:
    next_20_unknown = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[20:40]
    
    for i, (idx, row) in enumerate(next_20_unknown.iterrows(), 21):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    remaining_count = len(df_unknown_current)
    if remaining_count > 0:
        print(f"Solo hay {remaining_count} registros con género 'unknown'. Mostrando todos:")
        for i, (idx, row) in enumerate(df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
            name = row['name'] if pd.notna(row['name']) else 'N/A'
            first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
            email = row['email']
            print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
    else:
        print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 197

Siguientes 20 registros con género unknown:
21. Demian Emiliano                     | first_name: Demian          | email: 2424090078@utchsur.edu.mx
22. Yennifer Paola Quezada Armendariz   | first_name: Yennifer        | email: 2204090055@utchsur.edu.mx
23. Haziel Hernandez                    | first_name: Haziel          | email: haziele_hm98@hotmail.com
24. Leilany Payan                       | first_name: Leilany         | email: a01563498@tec.mx
25. Aaron Ochoa                         | first_name: Aguirre727      | email: annuarochoa10@gmail.com
26. Izhmir Edgar Salazar Ortiz          | first_name: N/A             | email: izinvest1@yahoo.com
27. Adal Valles                         | first_name: N/A             | email: adal.vallesb@gmail.com
28. Angeles Rascon                      | first_name: Angeles         | email: angie_rascon@hotmail.com
29. Uhub Erick                          | first_name: Uhub            | email: erick@

In [106]:
# Aplicar correcciones específicas para los registros del 21 al 30
targeted_corrections = [
    # Registro 21: Demian Emiliano -> male
    {'email': '2424090078@utchsur.edu.mx', 'name': 'Demian Emiliano', 'first_name': 'Demian', 'gender': 'male'},
    # Registro 22: Yennifer Paola Quezada Armendariz -> female
    {'email': '2204090055@utchsur.edu.mx', 'name': 'Yennifer Paola Quezada Armendariz', 'first_name': 'Yennifer', 'gender': 'female'},
    # Registro 23: Haziel Hernandez -> male
    {'email': 'haziele_hm98@hotmail.com', 'name': 'Haziel Hernandez', 'first_name': 'Haziel', 'gender': 'male'},
    # Registro 24: Leilany Payan -> female
    {'email': 'a01563498@tec.mx', 'name': 'Leilany Payan', 'first_name': 'Leilany', 'gender': 'female'},
    # Registro 25: Aaron Ochoa -> male
    {'email': 'annuarochoa10@gmail.com', 'name': 'Aaron Ochoa', 'first_name': 'Aaron', 'gender': 'male'},
    # Registro 26: Izhmir Edgar Salazar Ortiz -> male
    {'email': 'izinvest1@yahoo.com', 'name': 'Izhmir Edgar Salazar Ortiz', 'first_name': 'Izhmir', 'gender': 'male'},
    # Registro 27: Adal Valles -> male
    {'email': 'adal.vallesb@gmail.com', 'name': 'Adal Valles', 'first_name': 'Adal', 'gender': 'male'},
    # Registro 28: Angeles Rascon -> female
    {'email': 'angie_rascon@hotmail.com', 'name': 'Angeles Rascon', 'first_name': 'Angeles', 'gender': 'female'},
    # Registro 29: Erick Uhub -> male
    {'email': 'erick@uhub.mx', 'name': 'Erick Uhub', 'first_name': 'Erick', 'gender': 'male'},
    # Registro 30: Maricarmen Gutierrez Diez -> female
    {'email': 'mcgd10@gmail.com', 'name': 'Maricarmen Gutierrez Diez', 'first_name': 'Maricarmen', 'gender': 'female'}
]

print("Aplicando correcciones específicas (registros 21-30):")
print("="*60)

corrected_targeted = 0
for correction in targeted_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_targeted += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones específicas aplicadas: {corrected_targeted}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES ESPECÍFICAS:")
print("="*60)
targeted_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in targeted_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_targeted = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_targeted}")

Aplicando correcciones específicas (registros 21-30):
✓ Corregido: Demian Emiliano -> male
✓ Corregido: Yennifer Paola Quezada Armendariz -> female
✓ Corregido: Haziel Hernandez -> male
✓ Corregido: Leilany Payan -> female
✓ Corregido: Aaron Ochoa -> male
✓ Corregido: Izhmir Edgar Salazar Ortiz -> male
✓ Corregido: Adal Valles -> male
✓ Corregido: Angeles Rascon -> female
✓ Corregido: Erick Uhub -> male
✓ Corregido: Maricarmen Gutierrez Diez -> female

Total de correcciones específicas aplicadas: 10

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES ESPECÍFICAS:
MALE: 1862 registros (54.2%)
FEMALE: 1388 registros (40.4%)
UNKNOWN: 187 registros (5.4%)

Registros 'unknown' restantes: 187


In [107]:
# Mostrar los siguientes 20 registros con género 'unknown' después de todas las correcciones
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    print("\nSiguientes 20 registros con género unknown:")
    print("="*80)
    
    # Mostrar los primeros 20 registros
    limit = min(20, len(df_unknown_current))
    for i, (idx, row) in enumerate(df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].head(limit).iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
    
    # Si hay más de 20, mostrar los siguientes 20
    if len(df_unknown_current) > 20:
        print(f"\nSiguientes 20 registros (del 21 al 40):")
        print("="*80)
        
        next_20 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[20:40]
        for i, (idx, row) in enumerate(next_20.iterrows(), 21):
            name = row['name'] if pd.notna(row['name']) else 'N/A'
            first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
            email = row['email']
            print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 187

Siguientes 20 registros con género unknown:
 1. Mifam El Porvenir                   | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
 2. S Loer                              | first_name: N/A             | email: s.loer@outlook.com
 3. Leosausix                           | first_name: N/A             | email: leosausix@gmail.com
 4. Asistente                           | first_name: N/A             | email: asistente@medicalkids.com.mx
 5. Winorax                             | first_name: N/A             | email: winorax@hotmail.com
 6. Red Fision                          | first_name: Red             | email: redfision@gmail.com
 7. Asd                                 | first_name: Asd             | email: jamar123logi@protonmail.com
 8. Acinorev                            | first_name: N/A             | email: acinorev_2222@hotmail.com
 9. Flort                               | first_name: N/A             | email: f

In [108]:
# Aplicar correcciones específicas basadas en los comentarios del usuario
user_corrections = [
    # Correcciones específicas mencionadas en los comentarios
    {'email': 'caritordz@hotmail.com', 'name': 'Carolina Rodriguez', 'first_name': 'Carolina', 'gender': 'female'},
    {'email': 'licvanessaogaz@gmail.com', 'name': 'Venssa Ogaz', 'first_name': 'Venssa', 'gender': 'female'},
    {'email': 'marivargas1315@gmail.com', 'name': 'Maria Vargas', 'first_name': 'Maria', 'gender': 'female'},
    {'email': 'mimiperezp06@gmail.com', 'name': 'Mimi Perez', 'first_name': 'Mimi', 'gender': 'female'},
    {'email': 'perlagonzalezpromobien@gmail.com', 'name': 'Perla Gonzalez', 'first_name': 'Perla', 'gender': 'female'},
    {'email': 'ariadnadiaz889@gmail.com', 'name': 'Ariadna Diaz', 'first_name': 'Ariadna', 'gender': 'female'},
    {'email': 'jogensp@hotmail.com', 'name': 'Jogen Siebert Pantoja', 'first_name': 'Jogen', 'gender': 'male'},
    {'email': '2424090039@utchsur.edu.mx', 'name': 'Dabiel Gerardo Torres Peregrino', 'first_name': 'Dabiel', 'gender': 'male'},
    {'email': 'mavilla86@gmail.com', 'name': 'Menny Villa', 'first_name': 'Menny', 'gender': 'male'},
    {'email': 'anilu.urrutia@americanbigcertification.com', 'name': 'Anilu Urrutia Derma', 'first_name': 'Anilu', 'gender': 'female'},
    {'email': 'martinmx@martin-eng-mx.com', 'name': 'Martin', 'first_name': 'Martin', 'gender': 'male'}
]

print("Aplicando correcciones específicas del usuario:")
print("="*60)

corrected_user = 0
for correction in user_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_user += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones del usuario aplicadas: {corrected_user}")

# Mostrar distribución actualizada después de las correcciones del usuario
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DEL USUARIO:")
print("="*60)
user_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in user_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_user = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_user}")

# Mostrar algunos ejemplos de los registros corregidos
print(f"\nEjemplos de registros corregidos por el usuario:")
print("-"*60)
for correction in user_corrections[:5]:
    registro = df[df['email'] == correction['email']]
    if not registro.empty:
        nombre = registro['name'].iloc[0]
        first_name = registro['first_name'].iloc[0]
        genero = registro['gender_guess'].iloc[0]
        print(f"Email: {correction['email']}")
        print(f"  Nombre: {nombre} | First: {first_name} | Género: {genero}")

Aplicando correcciones específicas del usuario:
✓ Corregido: Carolina Rodriguez -> female
✓ Corregido: Venssa Ogaz -> female
✓ Corregido: Maria Vargas -> female
✓ Corregido: Mimi Perez -> female
✓ Corregido: Perla Gonzalez -> female
✓ Corregido: Ariadna Diaz -> female
✓ Corregido: Jogen Siebert Pantoja -> male
✓ Corregido: Dabiel Gerardo Torres Peregrino -> male
✓ Corregido: Menny Villa -> male
✓ Corregido: Anilu Urrutia Derma -> female
✓ Corregido: Martin -> male

Total de correcciones del usuario aplicadas: 11

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DEL USUARIO:
MALE: 1866 registros (54.3%)
FEMALE: 1395 registros (40.6%)
UNKNOWN: 176 registros (5.1%)

Registros 'unknown' restantes: 176

Ejemplos de registros corregidos por el usuario:
------------------------------------------------------------
Email: caritordz@hotmail.com
  Nombre: Carolina Rodriguez | First: Carolina | Género: female
Email: licvanessaogaz@gmail.com
  Nombre: Venssa Ogaz | First: Venssa | Género: female
Email: marivar

In [109]:
# Mostrar los siguientes 20 registros con género 'unknown' después de todas las correcciones aplicadas
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    print("\nRegistros con género unknown después de todas las correcciones:")
    print("="*80)
    # Mostrar estadísticas rápidas antes de mostrar la lista
    print(f"Estadística: {len(df_unknown_current)} registros restantes con género unknown")
    if len(df_unknown_current) > 0:
        print(f"Porcentaje del total: {(len(df_unknown_current)/len(df))*100:.1f}%")
    print()
    # Mostrar los primeros 20 registros
    limit = min(20, len(df_unknown_current))
    for i, (idx, row) in enumerate(df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].head(limit).iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 176

Registros con género unknown después de todas las correcciones:
Estadística: 176 registros restantes con género unknown
Porcentaje del total: 5.1%

 1. Mifam El Porvenir                   | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
 2. S Loer                              | first_name: N/A             | email: s.loer@outlook.com
 3. Leosausix                           | first_name: N/A             | email: leosausix@gmail.com
 4. Asistente                           | first_name: N/A             | email: asistente@medicalkids.com.mx
 5. Winorax                             | first_name: N/A             | email: winorax@hotmail.com
 6. Red Fision                          | first_name: Red             | email: redfision@gmail.com
 7. Asd                                 | first_name: Asd             | email: jamar123logi@protonmail.com
 8. Acinorev                            | first_name: N/A             | email: ac

In [110]:
# Aplicar correcciones específicas para los registros del 21 al 40 de los unknown restantes
next_batch_corrections = [
    # Registro 21: Flort -> Flora (female)
    {'email': 'flort918@gmail.com', 'name': 'Flora', 'first_name': 'Flora', 'gender': 'female'},
    # Registro 22: Plumas Novelty -> mantener como unknown (es empresa)
    # Registro 23: Acinorev -> Veronica (female) - es Veronica al revés
    {'email': 'acinorev_2222@hotmail.com', 'name': 'Veronica', 'first_name': 'Veronica', 'gender': 'female'},
    # Registro 24: Jogen Siebert Pantoja -> Jogen (male)
    {'email': 'jogensp@hotmail.com', 'name': 'Jogen Siebert Pantoja', 'first_name': 'Jogen', 'gender': 'male'},
    # Registro 25: Dabiel Gerardo Torres Peregrino -> Dabiel (male)
    {'email': '2424090039@utchsur.edu.mx', 'name': 'Dabiel Gerardo Torres Peregrino', 'first_name': 'Dabiel', 'gender': 'male'},
    # Registro 26: Menny Villa -> Menny (male)
    {'email': 'mavilla86@gmail.com', 'name': 'Menny Villa', 'first_name': 'Menny', 'gender': 'male'},
    # Registro 27: Anilu Urrutia Derma -> Anilu (female)
    {'email': 'anilu.urrutia@americanbigcertification.com', 'name': 'Anilu Urrutia Derma', 'first_name': 'Anilu', 'gender': 'female'},
    # Registro 28: Jp -> Juan Pablo (male)
    {'email': 'jp.ayala@gmail.com', 'name': 'Juan Pablo Ayala', 'first_name': 'Juan Pablo', 'gender': 'male'},
    # Registro 29: Sorely Falomir Talamantes -> Sorely (female)
    {'email': 'sorely.falomir@gmail.com', 'name': 'Sorely Falomir Talamantes', 'first_name': 'Sorely', 'gender': 'female'},
    # Registro 30: Alheli -> Alheli (female)
    {'email': 'alhelirey@sarcan.com.mx', 'name': 'Alheli Rey', 'first_name': 'Alheli', 'gender': 'female'},
    # Registro 31: Jam Urbina -> Alan (male) - soyalanurbina sugiere Alan
    {'email': 'soyalanurbina@gmail.com', 'name': 'Alan Urbina', 'first_name': 'Alan', 'gender': 'male'},
    # Registro 32: Crystian Reyes -> Crystian (male)
    {'email': 'crystianreyes2@gmail.com', 'name': 'Crystian Reyes', 'first_name': 'Crystian', 'gender': 'male'},
    # Registro 33: Aylink Reyes -> Aylin (female)
    {'email': 'keniaylin.nunez@gmail.com', 'name': 'Aylin Nunez', 'first_name': 'Aylin', 'gender': 'female'},
    # Registro 34: Yannet -> Yannet (female)
    {'email': 'yanetosman@gmail.com', 'name': 'Yannet Osman', 'first_name': 'Yannet', 'gender': 'female'},
    # Registro 35: Abri -> Abraham (male)
    {'email': 'abraham8390@gmail.com', 'name': 'Abraham Rodriguez', 'first_name': 'Abraham', 'gender': 'male'},
    # Registro 36: Johany Reyes -> Johany (male)
    {'email': 'johanyreyes18@gmail.com', 'name': 'Johany Reyes', 'first_name': 'Johany', 'gender': 'male'},
    # Registro 37: Jonatan Arvayo -> Jonatan (male)
    {'email': 'arvayo@me.com', 'name': 'Jonatan Arvayo', 'first_name': 'Jonatan', 'gender': 'male'},
    # Registro 38: Alekz Limas -> Alex (male)
    {'email': 'alekzlimas@gmail.com', 'name': 'Alex Limas', 'first_name': 'Alex', 'gender': 'male'},
    # Registro 39: Ciscoangel Est -> Francisco Angel (male)
    {'email': 'ciscoangel9@gmail.com', 'name': 'Francisco Angel', 'first_name': 'Francisco', 'gender': 'male'},
    # Registro 40: Esvar Valencia Chacon -> Esvar (male)
    {'email': 'alucard_sky@hotmail.com', 'name': 'Esvar Valencia Chacon', 'first_name': 'Esvar', 'gender': 'male'}
]

print("Aplicando correcciones para registros del 21 al 40:")
print("="*60)

corrected_batch = 0
for correction in next_batch_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_batch += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones del lote aplicadas: {corrected_batch}")

# Mostrar distribución actualizada después de estas correcciones
print(f"\nDISTRIBUCIÓN DESPUÉS DEL LOTE DE CORRECCIONES:")
print("="*60)
batch_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in batch_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_batch = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_batch}")

Aplicando correcciones para registros del 21 al 40:
✓ Corregido: Flora -> female
✓ Corregido: Veronica -> female
✓ Corregido: Jogen Siebert Pantoja -> male
✓ Corregido: Dabiel Gerardo Torres Peregrino -> male
✓ Corregido: Menny Villa -> male
✓ Corregido: Anilu Urrutia Derma -> female
✓ Corregido: Juan Pablo Ayala -> male
✓ Corregido: Sorely Falomir Talamantes -> female
✓ Corregido: Alheli Rey -> female
✓ Corregido: Alan Urbina -> male
✓ Corregido: Crystian Reyes -> male
✓ Corregido: Aylin Nunez -> female
✓ Corregido: Yannet Osman -> female
✓ Corregido: Abraham Rodriguez -> male
✓ Corregido: Johany Reyes -> male
✓ Corregido: Jonatan Arvayo -> male
✓ Corregido: Alex Limas -> male
✓ Corregido: Francisco Angel -> male
✓ Corregido: Esvar Valencia Chacon -> male

Total de correcciones del lote aplicadas: 19

DISTRIBUCIÓN DESPUÉS DEL LOTE DE CORRECCIONES:
MALE: 1866 registros (54.3%)
FEMALE: 1397 registros (40.6%)
UNKNOWN: 174 registros (5.1%)

Registros 'unknown' restantes: 174


In [111]:
# Mostrar los siguientes 20 registros con género 'unknown' (del 21 al 40)
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 20:
    print("\nRegistros del 21 al 40 con género unknown:")
    print("="*80)
    
    next_20_unknown = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[20:40]
    
    for i, (idx, row) in enumerate(next_20_unknown.iterrows(), 21):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
        
    if len(df_unknown_current) <= 40:
        print(f"\nSolo hay {len(df_unknown_current)} registros unknown en total")
else:
    print("No hay más de 20 registros con género 'unknown'")
    print(f"Total de registros unknown: {len(df_unknown_current)}")

Total actual de registros con género 'unknown': 174

Registros del 21 al 40 con género unknown:
21. Jorgethomas                         | first_name: N/A             | email: jorgethomas@transvarco.mx
22. Martindguez                         | first_name: N/A             | email: martindguez@ugrch.org
23. Ventas                              | first_name: N/A             | email: ventas@grupoamex.com.mx
24. Dr Octavio Ramos                    | first_name: N/A             | email: dr.octavio.ramos@gmail.com
25. Cchavez                             | first_name: N/A             | email: cchavez@ienova.com.mx
26. Ebaezal                             | first_name: N/A             | email: ebaezal@beefinternational.com
27. Ventas                              | first_name: N/A             | email: ventas@abitat.com.mx
28. Customer Service                    | first_name: N/A             | email: customer_service@precisionomega.com
29. Amonroy                             | first_name: N/A       

In [112]:
# Aplicar correcciones específicas basadas en los comentarios del usuario
business_corrections = [
    # Correcciones específicas mencionadas en los comentarios
    {'email': 'jorgethomas@transvarco.mx', 'name': 'Jorge Thomas', 'first_name': 'Jorge', 'gender': 'male'},
    {'email': 'martindguez@ugrch.org', 'name': 'Martin Dominguez', 'first_name': 'Martin', 'gender': 'male'},
    {'email': 'dr.octavio.ramos@gmail.com', 'name': 'Octavio Ramos', 'first_name': 'Octavio', 'gender': 'male'}
]

print("Aplicando correcciones específicas de nombres empresariales:")
print("="*60)

corrected_business = 0
for correction in business_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_business += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones empresariales aplicadas: {corrected_business}")

# Mostrar distribución actualizada después de las correcciones empresariales
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES EMPRESARIALES:")
print("="*60)
business_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in business_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_business = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_business}")

Aplicando correcciones específicas de nombres empresariales:
✓ Corregido: Jorge Thomas -> male
✓ Corregido: Martin Dominguez -> male
✓ Corregido: Octavio Ramos -> male

Total de correcciones empresariales aplicadas: 3

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES EMPRESARIALES:
MALE: 1869 registros (54.4%)
FEMALE: 1397 registros (40.6%)
UNKNOWN: 171 registros (5.0%)

Registros 'unknown' restantes: 171


In [113]:
# Mostrar los siguientes 20 registros con género 'unknown' (del 41 al 60)
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 40:
    print("\nRegistros del 41 al 60 con género unknown:")
    print("="*80)
    
    next_20_unknown_41_60 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[40:60]
    
    for i, (idx, row) in enumerate(next_20_unknown_41_60.iterrows(), 41):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
        
    if len(df_unknown_current) <= 60:
        print(f"\nSolo hay {len(df_unknown_current)} registros unknown en total")
else:
    print("No hay más de 40 registros con género 'unknown'")
    print(f"Total de registros unknown: {len(df_unknown_current)}")

Total actual de registros con género 'unknown': 171

Registros del 41 al 60 con género unknown:
41. Nayle Esnayra                       | first_name: Naylé           | email: nayle_alvarez17@outlook.com
42. Celine Ahumada                      | first_name: Celine          | email: dianacelahu@gmail.com
43. Inversor Angel Pedro                | first_name: N/A             | email: gcc@mx.com
44. Britzi Giovanna Dominguez Felix     | first_name: N/A             | email: giovanna.dgz07@gmail.com
45. Ramses                              | first_name: N/A             | email: rs@oneprimal.com
46. Aldair Rey                          | first_name: N/A             | email: aldair.rey.97@gmail.com
47. Mitzuki Morita                      | first_name: N/A             | email: mitzuki.morita@hotmail.com
48. Krake                               | first_name: Krake           | email: tanzerkrake@gmail.com
49. Belem Portillo                      | first_name: N/A             | email: belemluis@hotmail

In [114]:
# Aplicar las correcciones específicas basadas en los comentarios del usuario para los registros del 41-60
advanced_corrections = [
    # Registro 41: Nayle Esnayra -> female
    {'email': 'nayle_alvarez17@outlook.com', 'name': 'Nayle Esnayra', 'first_name': 'Nayle', 'gender': 'female'},
    # Registro 42: Celine Ahumada -> female
    {'email': 'dianacelahu@gmail.com', 'name': 'Celine Ahumada', 'first_name': 'Celine', 'gender': 'female'},
    # Registro 43: Angel Pedro -> male (quitar "Inversor")
    {'email': 'gcc@mx.com', 'name': 'Angel Pedro', 'first_name': 'Angel', 'gender': 'male'},
    # Registro 44: Britzi Giovanna Dominguez Felix -> female
    {'email': 'giovanna.dgz07@gmail.com', 'name': 'Britzi Giovanna Dominguez Felix', 'first_name': 'Britzi Giovanna', 'gender': 'female'},
    # Registro 45: Ramses -> male
    {'email': 'rs@oneprimal.com', 'name': 'Ramses', 'first_name': 'Ramses', 'gender': 'male'},
    # Registro 46: Aldair Rey -> male
    {'email': 'aldair.rey.97@gmail.com', 'name': 'Aldair Rey', 'first_name': 'Aldair', 'gender': 'male'},
    # Registro 47: Mitzuki Morita -> female
    {'email': 'mitzuki.morita@hotmail.com', 'name': 'Mitzuki Morita', 'first_name': 'Mitzuki', 'gender': 'female'},
    # Registro 48: Krake Tanzer -> male
    {'email': 'tanzerkrake@gmail.com', 'name': 'Krake Tanzer', 'first_name': 'Krake', 'gender': 'male'},
    # Registro 49: Belem Portillo -> female
    {'email': 'belemluis@hotmail.com', 'name': 'Belem Portillo', 'first_name': 'Belem', 'gender': 'female'},
    # Registro 50: Libertad Ortega -> female
    {'email': 'libertadfarinnemusic@gmail.com', 'name': 'Libertad Ortega', 'first_name': 'Libertad', 'gender': 'female'},
    # Registro 54: Ely Gandara -> female
    {'email': 'ely.gandara.c@gmail.com', 'name': 'Ely Gandara', 'first_name': 'Ely', 'gender': 'female'},
    # Registro 55: Exequiel Kantor -> male
    {'email': 'exekantor@gmail.com', 'name': 'Exequiel Kantor', 'first_name': 'Exequiel', 'gender': 'male'},
    # Registro 56: Gandhi Cervantes -> male
    {'email': 'gacervantes11@gmail.com', 'name': 'Gandhi Cervantes', 'first_name': 'Gandhi', 'gender': 'male'},
    # Registro 57: Carlos Ibanez -> male (cambiar Caelos por Carlos)
    {'email': 'carlos.alcala@tec.mx', 'name': 'Carlos Ibanez', 'first_name': 'Carlos', 'gender': 'male'},
    # Registro 58: Osmar Fernandez -> male
    {'email': 'fernandezlopezosmar4@gmail.com', 'name': 'Osmar Fernandez', 'first_name': 'Osmar', 'gender': 'male'},
    # Registro 59: Habid Narvaez -> male
    {'email': 'hnarvaez@fechac.org.mx', 'name': 'Habid Narvaez', 'first_name': 'Habid', 'gender': 'male'},
    # Registro 60: Azul Luna -> female
    {'email': 'azullunahedz@gmail.com', 'name': 'Azul Luna', 'first_name': 'Azul', 'gender': 'female'}
]

print("Aplicando correcciones avanzadas para registros 41-60:")
print("="*60)

corrected_advanced = 0
for correction in advanced_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_advanced += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones avanzadas aplicadas: {corrected_advanced}")

# Mostrar distribución actualizada después de las correcciones avanzadas
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES AVANZADAS:")
print("="*60)
advanced_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in advanced_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_advanced = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_advanced}")

Aplicando correcciones avanzadas para registros 41-60:
✓ Corregido: Nayle Esnayra -> female
✓ Corregido: Celine Ahumada -> female
✓ Corregido: Angel Pedro -> male
✓ Corregido: Britzi Giovanna Dominguez Felix -> female
✓ Corregido: Ramses -> male
✓ Corregido: Aldair Rey -> male
✓ Corregido: Mitzuki Morita -> female
✓ Corregido: Krake Tanzer -> male
✓ Corregido: Belem Portillo -> female
✓ Corregido: Libertad Ortega -> female
✓ Corregido: Ely Gandara -> female
✓ Corregido: Exequiel Kantor -> male
✓ Corregido: Gandhi Cervantes -> male
✓ Corregido: Carlos Ibanez -> male
✓ Corregido: Osmar Fernandez -> male
✓ Corregido: Habid Narvaez -> male
✓ Corregido: Azul Luna -> female

Total de correcciones avanzadas aplicadas: 17

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES AVANZADAS:
MALE: 1878 registros (54.6%)
FEMALE: 1405 registros (40.9%)
UNKNOWN: 154 registros (4.5%)

Registros 'unknown' restantes: 154


In [115]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos visto antes
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar registros del 61 al 80 (diferentes a los mostrados anteriormente)
    print("\nOtros 20 registros con género unknown (del 61 al 80):")
    print("="*80)
    
    if len(df_unknown_current) > 60:
        next_20_different = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[60:80]
        
        for i, (idx, row) in enumerate(next_20_different.iterrows(), 61):
            name = row['name'] if pd.notna(row['name']) else 'N/A'
            first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
            email = row['email']
            print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
    else:
        # Si no hay suficientes, mostrar desde donde podamos
        start_idx = max(0, len(df_unknown_current) - 20)
        remaining_20 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[start_idx:]
        
        print(f"Mostrando los últimos {len(remaining_20)} registros unknown:")
        for i, (idx, row) in enumerate(remaining_20.iterrows(), start_idx + 1):
            name = row['name'] if pd.notna(row['name']) else 'N/A'
            first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
            email = row['email']
            print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 154

Otros 20 registros con género unknown (del 61 al 80):
61. Hiei Numata                         | first_name: N/A             | email: hiei.numata@gmail.com
62. Drarosyornelas                      | first_name: N/A             | email: drarosyornelas@gmail.com
63. Rociogarciah                        | first_name: N/A             | email: rociogarciah@tec.mx
64. A                                   | first_name: N/A             | email: a01562935@tec.mx
65. Lorenalopezgutierrez                | first_name: N/A             | email: lorenalopezgutierrez57@gmail.com
66. Mkt Digital Cuu                     | first_name: N/A             | email: mkt.digital.cuu@gmail.com
67. Sofiahuertar                        | first_name: N/A             | email: sofiahuertar08@gmail.com
68. Paolaolverab                        | first_name: N/A             | email: paolaolverab@gmail.com
69. Carsi                               | first_name: N/A             

In [116]:
# Aplicar correcciones específicas basadas en los comentarios del usuario (registros 61-80)
final_corrections_batch = [
    # Registro 61: Hiei Numata -> female
    {'email': 'hiei.numata@gmail.com', 'name': 'Hiei Numata', 'first_name': 'Hiei', 'gender': 'female'},
    # Registro 62: Drarosyornelas -> Rosy Ornelas (female)
    {'email': 'drarosyornelas@gmail.com', 'name': 'Rosy Ornelas', 'first_name': 'Rosy', 'gender': 'female'},
    # Registro 63: Rociogarciah -> Rocio García (female)
    {'email': 'rociogarciah@tec.mx', 'name': 'Rocio García', 'first_name': 'Rocio', 'gender': 'female'},
    # Registro 65: Lorenalopezgutierrez -> Lorena Lopez Gutierrez (female)
    {'email': 'lorenalopezgutierrez57@gmail.com', 'name': 'Lorena Lopez Gutierrez', 'first_name': 'Lorena', 'gender': 'female'},
    # Registro 67: Sofiahuertar -> Sofia Huerta (female)
    {'email': 'sofiahuertar08@gmail.com', 'name': 'Sofia Huerta', 'first_name': 'Sofia', 'gender': 'female'},
    # Registro 68: Paolaolverab -> Paola Olvera (female)
    {'email': 'paolaolverab@gmail.com', 'name': 'Paola Olvera', 'first_name': 'Paola', 'gender': 'female'},
    # Registro 72: Siqueiros Nadia -> Nadia Siqueiros (female)
    {'email': 'siqueiros.nadia@gmail.com', 'name': 'Nadia Siqueiros', 'first_name': 'Nadia', 'gender': 'female'},
    # Registro 74: Jenniferpuntocom -> Jennifer (female)
    {'email': 'jenniferpuntocom@gmail.com', 'name': 'Jennifer', 'first_name': 'Jennifer', 'gender': 'female'},
    # Registro 75: Lizbx -> Liz B (female)
    {'email': 'lizbx@hotmail.com', 'name': 'Liz B', 'first_name': 'Liz', 'gender': 'female'},
    # Registro 76: Xime Benitez -> Ximena Benitez (female)
    {'email': 'xime.benitez05@gmail.com', 'name': 'Ximena Benitez', 'first_name': 'Ximena', 'gender': 'female'}
]

print("Aplicando correcciones finales del lote 61-80:")
print("="*60)

corrected_final_batch = 0
for correction in final_corrections_batch:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_final_batch += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones finales aplicadas: {corrected_final_batch}")

# Mostrar distribución final después de todas las correcciones
print(f"\nDISTRIBUCIÓN FINAL DESPUÉS DE TODAS LAS CORRECCIONES:")
print("="*60)
final_batch_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_batch_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_final_batch = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_final_batch}")

Aplicando correcciones finales del lote 61-80:
✓ Corregido: Hiei Numata -> female
✓ Corregido: Rosy Ornelas -> female
✓ Corregido: Rocio García -> female
✓ Corregido: Lorena Lopez Gutierrez -> female
✓ Corregido: Sofia Huerta -> female
✓ Corregido: Paola Olvera -> female
✓ Corregido: Nadia Siqueiros -> female
✓ Corregido: Jennifer -> female
✓ Corregido: Liz B -> female
✓ Corregido: Ximena Benitez -> female

Total de correcciones finales aplicadas: 10

DISTRIBUCIÓN FINAL DESPUÉS DE TODAS LAS CORRECCIONES:
MALE: 1878 registros (54.6%)
FEMALE: 1415 registros (41.2%)
UNKNOWN: 144 registros (4.2%)

Registros 'unknown' restantes: 144


In [117]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos mostrado antes
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 80:
    print("\nOtros 20 registros con género unknown (del 81 al 100):")
    print("="*80)
    
    next_20_different_81_100 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[80:100]
    
    for i, (idx, row) in enumerate(next_20_different_81_100.iterrows(), 81):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    # Si no hay suficientes, mostrar una muestra aleatoria de los que quedan
    print("\nMuestra aleatoria de 20 registros unknown restantes:")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    random_sample = df_unknown_current.sample(n=sample_size, random_state=123)
    
    for i, (idx, row) in enumerate(random_sample[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")

Total actual de registros con género 'unknown': 144

Otros 20 registros con género unknown (del 81 al 100):
81. Silvialeosm                         | first_name: N/A             | email: silvialeosm@gmail.com
82. Glorialdc                           | first_name: N/A             | email: glorialdc@gmail.com
83. Dayi Wong                           | first_name: N/A             | email: dayi.wong04@gmail.com
84. Sofiaalarconponce                   | first_name: N/A             | email: sofiaalarconponce19@outlook.com
85. Evedominguez                        | first_name: N/A             | email: evedominguez30@gmail.com
86. Yukey Gtz                           | first_name: N/A             | email: yukey.gtz@hotmail.com
87. Yaz Assmar                          | first_name: Lic. Yazmin     | email: yaz.assmar@gmail.com
88. Eugeniaperez                        | first_name: N/A             | email: eugeniaperez@aiig.com
89. Ibarzabal                           | first_name: N/A             | em

In [118]:
# Aplicar correcciones específicas basadas en los comentarios del usuario (registros 81-100)
final_batch_corrections = [
    # Registro 81: Silvialeosm -> Silvia Leos (female)
    {'email': 'silvialeosm@gmail.com', 'name': 'Silvia Leos', 'first_name': 'Silvia', 'gender': 'female'},
    # Registro 82: Glorialdc -> Gloria L (female)
    {'email': 'glorialdc@gmail.com', 'name': 'Gloria L', 'first_name': 'Gloria', 'gender': 'female'},
    # Registro 83: Dayi Wong -> Dayi (female)
    {'email': 'dayi.wong04@gmail.com', 'name': 'Dayi Wong', 'first_name': 'Dayi', 'gender': 'female'},
    # Registro 84: Sofiaalarconponce -> Sofia Alarcon Ponce (female)
    {'email': 'sofiaalarconponce19@outlook.com', 'name': 'Sofia Alarcon Ponce', 'first_name': 'Sofia', 'gender': 'female'},
    # Registro 85: Evedominguez -> Evelyn Dominguez (female)
    {'email': 'evedominguez30@gmail.com', 'name': 'Evelyn Dominguez', 'first_name': 'Evelyn', 'gender': 'female'},
    # Registro 86: Yukey Gtz -> Yukey Gutierrez (female)
    {'email': 'yukey.gtz@hotmail.com', 'name': 'Yukey Gutierrez', 'first_name': 'Yukey', 'gender': 'female'},
    # Registro 87: Yaz Assmar -> Yazmin Assmar (female)
    {'email': 'yaz.assmar@gmail.com', 'name': 'Yazmin Assmar', 'first_name': 'Yazmin', 'gender': 'female'},
    # Registro 88: Eugeniaperez -> Eugenia Perez (female)
    {'email': 'eugeniaperez@aiig.com', 'name': 'Eugenia Perez', 'first_name': 'Eugenia', 'gender': 'female'},
    # Registro 90: Jonahtan Gonzalez -> Jonathan Gonzalez (male)
    {'email': 'jonahtanggc@gmail.com', 'name': 'Jonathan Gonzalez', 'first_name': 'Jonathan', 'gender': 'male'},
    # Registro 91: Jatziri Romero -> Jatziri (female)
    {'email': 'jatzromero154@gmail.com', 'name': 'Jatziri Romero', 'first_name': 'Jatziri', 'gender': 'female'},
    # Registro 92: Onix Alvaro Ibarra Legards -> Onix (male)
    {'email': 'ibarraonixalvaro@gmail.com', 'name': 'Onix Alvaro Ibarra Legards', 'first_name': 'Onix', 'gender': 'male'},
    # Registro 93: Platsage -> Yamil Elias (male)
    {'email': 'yamil.elias@platsage.com', 'name': 'Yamil Elias', 'first_name': 'Yamil', 'gender': 'male'},
    # Registro 98: Atenas Valeria Garcia Moreno -> Atenas (female)
    {'email': 'avgm.integral@gmail.com', 'name': 'Atenas Valeria Garcia Moreno', 'first_name': 'Atenas', 'gender': 'female'},
    # Registro 99: Carmen Nicte Ortiz Villanueva -> Carmen Nicte (female)
    {'email': 'dranicteortiz@gmail.com', 'name': 'Carmen Nicte Ortiz Villanueva', 'first_name': 'Carmen Nicte', 'gender': 'female'},
    # Registro 100: Esmeral Anchondo -> Esmeralda Anchondo (female)
    {'email': 'al05066775@tecmilenio.com.mx', 'name': 'Esmeralda Anchondo', 'first_name': 'Esmeralda', 'gender': 'female'}
]

print("Aplicando correcciones finales del lote 81-100:")
print("="*60)

corrected_final_batch = 0
for correction in final_batch_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_final_batch += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones finales aplicadas: {corrected_final_batch}")

# Mostrar distribución final después de todas las correcciones
print(f"\nDISTRIBUCIÓN FINAL DESPUÉS DE TODAS LAS CORRECCIONES:")
print("="*60)
final_batch_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_batch_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_final_batch = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_final_batch}")

# Mostrar algunos ejemplos de los registros corregidos
print(f"\nEjemplos de registros corregidos:")
print("-"*60)
for correction in final_batch_corrections[:5]:
    registro = df[df['email'] == correction['email']]
    if not registro.empty:
        nombre = registro['name'].iloc[0]
        first_name = registro['first_name'].iloc[0]
        genero = registro['gender_guess'].iloc[0]
        print(f"Email: {correction['email']}")
        print(f"  Nombre: {nombre} | First: {first_name} | Género: {genero}")

Aplicando correcciones finales del lote 81-100:
✓ Corregido: Silvia Leos -> female
✓ Corregido: Gloria L -> female
✓ Corregido: Dayi Wong -> female
✓ Corregido: Sofia Alarcon Ponce -> female
✓ Corregido: Evelyn Dominguez -> female
✓ Corregido: Yukey Gutierrez -> female
✓ Corregido: Yazmin Assmar -> female
✓ Corregido: Eugenia Perez -> female
✓ Corregido: Jonathan Gonzalez -> male
✓ Corregido: Jatziri Romero -> female
✓ Corregido: Onix Alvaro Ibarra Legards -> male
✓ Corregido: Yamil Elias -> male
✓ Corregido: Atenas Valeria Garcia Moreno -> female
✓ Corregido: Carmen Nicte Ortiz Villanueva -> female
✓ Corregido: Esmeralda Anchondo -> female

Total de correcciones finales aplicadas: 15

DISTRIBUCIÓN FINAL DESPUÉS DE TODAS LAS CORRECCIONES:
MALE: 1881 registros (54.7%)
FEMALE: 1427 registros (41.5%)
UNKNOWN: 129 registros (3.8%)

Registros 'unknown' restantes: 129

Ejemplos de registros corregidos:
------------------------------------------------------------
Email: silvialeosm@gmail.com


In [119]:
# Mostrar otros 20 registros con género 'unknown' que aún no hemos visto
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 100:
    print("\nOtros 20 registros con género unknown (del 101 al 120):")
    print("="*80)
    
    next_20_different_101_120 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[100:120]
    
    for i, (idx, row) in enumerate(next_20_different_101_120.iterrows(), 101):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
        
elif len(df_unknown_current) > 80:
    print("\nOtros 20 registros con género unknown (del 81 al 100):")
    print("="*80)
    
    next_20_different_81_100 = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iloc[80:100]
    
    for i, (idx, row) in enumerate(next_20_different_81_100.iterrows(), 81):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    # Si no hay suficientes, mostrar una muestra aleatoria diferente
    print("\nMuestra aleatoria de otros 20 registros unknown:")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed diferente para obtener una muestra diferente
    random_sample = df_unknown_current.sample(n=sample_size, random_state=456)
    
    for i, (idx, row) in enumerate(random_sample[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")

Total actual de registros con género 'unknown': 129

Otros 20 registros con género unknown (del 101 al 120):
101. Merari Mancillas                    | first_name: Merari          | email: merariocampo1@gmail.com
102. Ytzi Fabiola Martinez               | first_name: Ytzi            | email: ytzi92@gmail.com
103. Base2 Fechac                        | first_name: Base2           | email: base2@fechac.org.mx
104. Mifam Canaco                        | first_name: N/A             | email: hola@inndech.org
105. Saswat.Lens                         | first_name: Saswat          | email: saswat@arbalestpartners.com
106. Mexi.⌐◨-◨                           | first_name: Mexi            | email: mauricio@meximalist.com
107. Yenzi Fernandez                     | first_name: Yenzi           | email: yenzif@icloud.com
108. Izmir Abbud Losoya                  | first_name: Izmir           | email: compras02@cactustraffic.com.mx
109. Ailen Paola Villalobos Garcia       | first_name: N/A             |

In [120]:
# Aplicar correcciones específicas para los registros del 101-120
final_gender_corrections = [
    # Registro 101: Merari Mancillas -> female
    {'email': 'merariocampo1@gmail.com', 'name': 'Merari Mancillas', 'first_name': 'Merari', 'gender': 'female'},
    # Registro 102: Ytzi Fabiola Martinez -> female
    {'email': 'ytzi92@gmail.com', 'name': 'Ytzi Fabiola Martinez', 'first_name': 'Ytzi', 'gender': 'female'},
    # Registro 105: Saswat.Lens -> male
    {'email': 'saswat@arbalestpartners.com', 'name': 'Saswat Lens', 'first_name': 'Saswat', 'gender': 'male'},
    # Registro 106: Mexi.⌐◨-◨ -> Mauricio M (male)
    {'email': 'mauricio@meximalist.com', 'name': 'Mauricio M', 'first_name': 'Mauricio', 'gender': 'male'},
    # Registro 107: Yenzi Fernandez -> male
    {'email': 'yenzif@icloud.com', 'name': 'Yenzi Fernandez', 'first_name': 'Yenzi', 'gender': 'male'},
    # Registro 108: Izmir Abbud Losoya -> male
    {'email': 'compras02@cactustraffic.com.mx', 'name': 'Izmir Abbud Losoya', 'first_name': 'Izmir', 'gender': 'male'},
    # Registro 109: Ailen Paola Villalobos Garcia -> female
    {'email': 'ailen.villalobos1@gmail.com', 'name': 'Ailen Paola Villalobos Garcia', 'first_name': 'Ailen', 'gender': 'female'},
    # Registro 110: Carlis Adrianzen -> Carlos Adrianzen (male)
    {'email': 'c.adrianzen.irigoin123@gmail.com', 'name': 'Carlos Adrianzen', 'first_name': 'Carlos', 'gender': 'male'},
    # Registro 111: Lenin Rascon -> male
    {'email': 'gera-rascon98@hotmail.com', 'name': 'Lenin Rascon', 'first_name': 'Lenin', 'gender': 'male'},
    # Registro 112: Isaay Sosa -> male
    {'email': 'sosa.contact.me@gmail.com', 'name': 'Isaay Sosa', 'first_name': 'Isaay', 'gender': 'male'},
    # Registro 113: Marinee Chavez Garcia -> female
    {'email': 'marinee.chavez@gmail.com', 'name': 'Marinee Chavez Garcia', 'first_name': 'Marinee', 'gender': 'female'},
    # Registro 114: Oluseun Amusan -> male
    {'email': 'oluseunamusana@gmail.com', 'name': 'Oluseun Amusan', 'first_name': 'Oluseun', 'gender': 'male'},
    # Registro 115: Zarai -> female
    {'email': 'zarai.salvador@gmail.com', 'name': 'Zarai Salvador', 'first_name': 'Zarai', 'gender': 'female'},
    # Registro 116: Yamileth Ramirez -> female
    {'email': 'yamileth.ramirez@ficosec.org', 'name': 'Yamileth Ramirez', 'first_name': 'Yamileth', 'gender': 'female'},
    # Registro 117: Georgen Omar Olivas Sarinana -> male
    {'email': 'a336547@uach.mx', 'name': 'Georgen Omar Olivas Sarinana', 'first_name': 'Georgen', 'gender': 'male'},
    # Registro 118: Yedel Lizbeth Romero Caro -> female
    {'email': 'yeromero@tecmilenio.mx', 'name': 'Yedel Lizbeth Romero Caro', 'first_name': 'Yedel', 'gender': 'female'},
    # Registro 119: Bani Carrasco -> female
    {'email': 'bani.carrasco@chihuahuait.org', 'name': 'Bani Carrasco', 'first_name': 'Bani', 'gender': 'female'},
    # Registro 120: Aracely Lopez Gonzalez -> female
    {'email': 'alopezg@uach.mx', 'name': 'Aracely Lopez Gonzalez', 'first_name': 'Aracely', 'gender': 'female'}
]

print("Aplicando correcciones finales para registros 101-120:")
print("="*60)

corrected_final_gender = 0
for correction in final_gender_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_final_gender += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones finales aplicadas: {corrected_final_gender}")

# Mostrar distribución final después de todas las correcciones
print(f"\nDISTRIBUCIÓN FINAL DESPUÉS DE CORRECCIONES 101-120:")
print("="*60)
final_gender_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_gender_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_final_gender = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_final_gender}")

# Mostrar algunos ejemplos de los registros corregidos
print(f"\nEjemplos de registros corregidos:")
print("-"*60)
for correction in final_gender_corrections[:5]:
    registro = df[df['email'] == correction['email']]
    if not registro.empty:
        nombre = registro['name'].iloc[0]
        first_name = registro['first_name'].iloc[0]
        genero = registro['gender_guess'].iloc[0]
        print(f"Email: {correction['email']}")
        print(f"  Nombre: {nombre} | First: {first_name} | Género: {genero}")

Aplicando correcciones finales para registros 101-120:
✓ Corregido: Merari Mancillas -> female
✓ Corregido: Ytzi Fabiola Martinez -> female
✓ Corregido: Saswat Lens -> male
✓ Corregido: Mauricio M -> male
✓ Corregido: Yenzi Fernandez -> male
✓ Corregido: Izmir Abbud Losoya -> male
✓ Corregido: Ailen Paola Villalobos Garcia -> female
✓ Corregido: Carlos Adrianzen -> male
✓ Corregido: Lenin Rascon -> male
✓ Corregido: Isaay Sosa -> male
✓ Corregido: Marinee Chavez Garcia -> female
✓ Corregido: Oluseun Amusan -> male
✓ Corregido: Zarai Salvador -> female
✓ Corregido: Yamileth Ramirez -> female
✓ Corregido: Georgen Omar Olivas Sarinana -> male
✓ Corregido: Yedel Lizbeth Romero Caro -> female
✓ Corregido: Bani Carrasco -> female
✓ Corregido: Aracely Lopez Gonzalez -> female

Total de correcciones finales aplicadas: 18

DISTRIBUCIÓN FINAL DESPUÉS DE CORRECCIONES 101-120:
MALE: 1890 registros (55.0%)
FEMALE: 1436 registros (41.8%)
UNKNOWN: 111 registros (3.2%)

Registros 'unknown' restantes: 

In [121]:
# Mostrar los registros restantes con género 'unknown' que aún no se han mostrado
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    print("\nTodos los registros restantes con género unknown:")
    print("="*80)
    
    for i, (idx, row) in enumerate(df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:3d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
    
    print(f"\nRESUMEN FINAL:")
    print("="*50)
    print(f"Total de registros unknown restantes: {len(df_unknown_current)}")
    
    # Mostrar distribución final
    final_counts = df["gender_guess"].value_counts()
    total = len(df)
    print(f"\nDISTRIBUCIÓN FINAL DE GÉNEROS:")
    for gender, count in final_counts.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular mejora total desde el inicio
    initial_unknown = 515  # Número inicial que teníamos
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nMEJORA TOTAL DESDE EL INICIO:")
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 111

Todos los registros restantes con género unknown:
  1. Mifam El Porvenir                        | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
  2. S Loer                                   | first_name: N/A             | email: s.loer@outlook.com
  3. Leosausix                                | first_name: N/A             | email: leosausix@gmail.com
  4. Asistente                                | first_name: N/A             | email: asistente@medicalkids.com.mx
  5. Winorax                                  | first_name: N/A             | email: winorax@hotmail.com
  6. Red Fision                               | first_name: Red             | email: redfision@gmail.com
  7. Asd                                      | first_name: Asd             | email: jamar123logi@protonmail.com
  8. Plumas Novelty                           | first_name: N/A             | email: plumas_novelty@hotmail.com
  9. Publicidad          

In [122]:
# Mostrar registros con género 'unknown' que no hayamos visto en las células anteriores
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    print("\nRegistros con género unknown que no hemos mostrado anteriormente:")
    print("="*80)
    
    # Mostrar una muestra aleatoria diferente usando un seed distinto
    sample_size = min(30, len(df_unknown_current))
    random_sample = df_unknown_current.sample(n=sample_size, random_state=789)
    
    for i, (idx, row) in enumerate(random_sample[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 111

Registros con género unknown que no hemos mostrado anteriormente:
 1. Yudith Ortega                            | first_name: Yudith          | email: yudithort@hotmail.com
 2. A. Liliana Terrazas Estrada              | first_name: A. Liliana      | email: lilyterrazas@hotmail.com
 3. Mx Comercial                             | first_name: N/A             | email: mx.comercial@iberdrola.com
 4. Iyamely Rodriguez                        | first_name: Iyamely         | email: a309333@uach.mx
 5. Plumas Novelty                           | first_name: N/A             | email: plumas_novelty@hotmail.com
 6. Mkt Digital Cuu                          | first_name: N/A             | email: mkt.digital.cuu@gmail.com
 7. Bere Gonzalez                            | first_name: Bere            | email: berenice.gonzalez@ilumps.com
 8. Sahir Renteria                           | first_name: Sahir           | email: sarenteria@rewo.com.mx
 9. Cxc       

In [123]:
# Aplicar correcciones específicas basadas en los comentarios del usuario
additional_user_corrections = [
    # Correcciones específicas del usuario
    {'email': 'yudithort@hotmail.com', 'name': 'Yudith Ortega', 'first_name': 'Yudith', 'gender': 'female'},
    {'email': 'lilyterrazas@hotmail.com', 'name': 'Liliana Terrazas Estrada', 'first_name': 'Liliana', 'gender': 'female'},
    {'email': 'a309333@uach.mx', 'name': 'Iyamely Rodriguez', 'first_name': 'Iyamely', 'gender': 'female'},
    {'email': 'berenice.gonzalez@ilumps.com', 'name': 'Bere Gonzalez', 'first_name': 'Bere', 'gender': 'female'},
    {'email': 'sarenteria@rewo.com.mx', 'name': 'Sahir Renteria', 'first_name': 'Sahir', 'gender': 'male'},
    {'email': 'mmarquez@itcdcuauhtemoc.edu.mx', 'name': 'Maramay Marquez Tena', 'first_name': 'Maramay', 'gender': 'female'},
    {'email': 'tanialmb@hotmail.com', 'name': 'Tania L', 'first_name': 'Tania', 'gender': 'female'},
    {'email': 'mackynita.perez@gmail.com', 'name': 'Marycarmen Perez Carrizalez', 'first_name': 'Marycarmen', 'gender': 'female'},
    {'email': 'dillery@darwinnow.io', 'name': 'Dillery Burciaga', 'first_name': 'Dillery', 'gender': 'female'}
]

print("Aplicando correcciones adicionales del usuario:")
print("="*60)

corrected_additional_user = 0
for correction in additional_user_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_additional_user += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones adicionales del usuario: {corrected_additional_user}")

# Mostrar distribución actualizada después de las correcciones del usuario
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES ADICIONALES:")
print("="*60)
additional_user_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in additional_user_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_additional_user = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_additional_user}")

# Mostrar algunos ejemplos de los registros corregidos
print(f"\nEjemplos de registros corregidos:")
print("-"*60)
for correction in additional_user_corrections[:5]:
    registro = df[df['email'] == correction['email']]
    if not registro.empty:
        nombre = registro['name'].iloc[0]
        first_name = registro['first_name'].iloc[0]
        genero = registro['gender_guess'].iloc[0]
        print(f"Email: {correction['email']}")
        print(f"  Nombre: {nombre} | First: {first_name} | Género: {genero}")

Aplicando correcciones adicionales del usuario:
✓ Corregido: Yudith Ortega -> female
✓ Corregido: Liliana Terrazas Estrada -> female
✓ Corregido: Iyamely Rodriguez -> female
✓ Corregido: Bere Gonzalez -> female
✓ Corregido: Sahir Renteria -> male
✓ Corregido: Maramay Marquez Tena -> female
✓ Corregido: Tania L -> female
✓ Corregido: Marycarmen Perez Carrizalez -> female
✓ Corregido: Dillery Burciaga -> female

Total de correcciones adicionales del usuario: 9

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES ADICIONALES:
MALE: 1891 registros (55.0%)
FEMALE: 1444 registros (42.0%)
UNKNOWN: 102 registros (3.0%)

Registros 'unknown' restantes: 102

Ejemplos de registros corregidos:
------------------------------------------------------------
Email: yudithort@hotmail.com
  Nombre: Yudith Ortega | First: Yudith | Género: female
Email: lilyterrazas@hotmail.com
  Nombre: Liliana Terrazas Estrada | First: Liliana | Género: female
Email: a309333@uach.mx
  Nombre: Iyamely Rodriguez | First: Iyamely | Género:

In [124]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos visto antes
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed distinto
    print("\nOtros 20 registros con género unknown (muestra aleatoria):")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed diferente para obtener una muestra diferente
    random_sample = df_unknown_current.sample(n=sample_size, random_state=999)
    
    for i, (idx, row) in enumerate(random_sample[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 102

Otros 20 registros con género unknown (muestra aleatoria):
 1. Mifam El Porvenir                        | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
 2. Garu Alvarez                             | first_name: Garu            | email: sockideasmx@gmail.com
 3. Dmongeb                                  | first_name: N/A             | email: dmongeb@gcc.com
 4. Facturacion                              | first_name: N/A             | email: facturacion02@cactustraffic.com.mx
 5. Elizabethdlc                             | first_name: N/A             | email: elizabethdlc813@gmail.com
 6. Relacionespublicas                       | first_name: N/A             | email: relacionespublicas@colegiodechihuahua.edu.mx
 7. Vane Valenzuela                          | first_name: N/A             | email: vane.valenzuela1206@gmail.com
 8. Prograpp                                 | first_name: N/A             | email: yamil.elias@p

In [125]:
# Aplicar correcciones específicas basadas en los comentarios del usuario
final_user_corrections = [
    {'email': 'sockideasmx@gmail.com', 'name': 'Garu Alvarez', 'first_name': 'Garu', 'gender': 'male'},
    {'email': 'elizabethdlc813@gmail.com', 'name': 'Elizabeth D', 'first_name': 'Elizabeth', 'gender': 'female'},
    {'email': 'vane.valenzuela1206@gmail.com', 'name': 'Vanessa Valenzuela', 'first_name': 'Vanessa', 'gender': 'female'},
    {'email': 'yamil.elias@prograpp.com.mx', 'name': 'Yamil Elias', 'first_name': 'Yamil', 'gender': 'male'},
    {'email': 'wchacon@itcdcuauhtemoc.edu.mx', 'name': 'Wuendy Lizeth Chacon Martinez', 'first_name': 'Wuendy', 'gender': 'female'}
]

print("Aplicando correcciones finales del usuario:")
print("="*60)

corrected_final_user = 0
for correction in final_user_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_final_user += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones finales del usuario: {corrected_final_user}")

# Mostrar distribución final actualizada
print(f"\nDISTRIBUCIÓN FINAL ACTUALIZADA:")
print("="*60)
final_user_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in final_user_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_final_user = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_final_user}")

# Calcular mejora total desde el inicio
initial_unknown = 515
total_improved_final = initial_unknown - remaining_unknown_final_user
improvement_rate_final = (total_improved_final / initial_unknown) * 100

print(f"\nRESUMEN FINAL DE TODAS LAS MEJORAS:")
print("="*50)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown final: {remaining_unknown_final_user}")
print(f"Total mejorado: {total_improved_final}")
print(f"Porcentaje de mejora total: {improvement_rate_final:.1f}%")

Aplicando correcciones finales del usuario:
✓ Corregido: Garu Alvarez -> male
✓ Corregido: Elizabeth D -> female
✓ Corregido: Vanessa Valenzuela -> female
✓ Corregido: Yamil Elias -> male
✓ Corregido: Wuendy Lizeth Chacon Martinez -> female

Total de correcciones finales del usuario: 5

DISTRIBUCIÓN FINAL ACTUALIZADA:
MALE: 1893 registros (55.1%)
FEMALE: 1447 registros (42.1%)
UNKNOWN: 97 registros (2.8%)

Registros 'unknown' restantes: 97

RESUMEN FINAL DE TODAS LAS MEJORAS:
Unknown inicial: 515
Unknown final: 97
Total mejorado: 418
Porcentaje de mejora total: 81.2%


In [126]:
# Mostrar otros 30 registros con género 'unknown' que no hemos revisado anteriormente
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 30 registros con género unknown (muestra aleatoria diferente):")
    print("="*80)
    
    sample_size = min(30, len(df_unknown_current))
    # Usar un seed único para obtener una muestra completamente diferente
    random_sample = df_unknown_current.sample(n=sample_size, random_state=2024)
    
    for i, (idx, row) in enumerate(random_sample[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 97

Otros 30 registros con género unknown (muestra aleatoria diferente):
 1. Cchavez                                  | first_name: N/A             | email: cchavez@ienova.com.mx
 2. Dmongeb                                  | first_name: N/A             | email: dmongeb@gcc.com
 3. A                                        | first_name: N/A             | email: a295727@uach.mx
 4. Amonroy                                  | first_name: N/A             | email: amonroy@hospitalcima.com.mx
 5. Contabilidad                             | first_name: N/A             | email: contabilidad@abainst.com.mx
 6. Lvargas                                  | first_name: N/A             | email: lvargas@interceramic.com
 7. Customer Service                         | first_name: N/A             | email: customer_service@precisionomega.com
 8. Anisol Macias Jurado                     | first_name: Anisol          | email: anisol.macias@chihuahua.com.mx
 9. A

In [127]:
# Apply corrections for identifiable names from the random sample
sample_corrections = [
    # Female names that are clearly identifiable
    {'email': 'anisol.macias@chihuahua.com.mx', 'name': 'Anisol Macias Jurado', 'first_name': 'Anisol', 'gender': 'female'},
    {'email': 'zulemar@yahoo.com', 'name': 'Zulemar', 'first_name': 'Zulemar', 'gender': 'female'},
    {'email': 'yesi.monjaraz14@hotmail.com', 'name': 'Yesi Monjaraz', 'first_name': 'Yesi', 'gender': 'female'},
    {'email': 'sarahysalcido.castillo@gmail.com', 'name': 'Sarahy Salcido', 'first_name': 'Sarahy', 'gender': 'female'},
    
    # Male names that are clearly identifiable  
    {'email': 'jamzin22110003@gmail.com', 'name': 'Jamzin Luviano', 'first_name': 'Jamzin', 'gender': 'male'},
    
    # Clean up company/generic names
    {'email': 'davidsalas647@gmail.com', 'name': 'David Salas', 'first_name': 'David', 'gender': 'male'}
]

print("Aplicando correcciones para la muestra de registros unknown:")
print("="*60)

corrected_sample = 0
for correction in sample_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_sample += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones de muestra aplicadas: {corrected_sample}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA:")
print("="*60)
sample_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in sample_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_sample = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_sample}")

# Calcular progreso total
initial_unknown = 515
total_improved_sample = initial_unknown - remaining_unknown_sample
improvement_rate_sample = (total_improved_sample / initial_unknown) * 100

print(f"\nPROGRESO TOTAL ACUMULADO:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown actual: {remaining_unknown_sample}")
print(f"Total mejorado: {total_improved_sample}")
print(f"Porcentaje de mejora: {improvement_rate_sample:.1f}%")

Aplicando correcciones para la muestra de registros unknown:
✓ Corregido: Anisol Macias Jurado -> female
✓ Corregido: Zulemar -> female
✓ Corregido: Yesi Monjaraz -> female
✓ Corregido: Sarahy Salcido -> female
✗ No encontrado: jamzin22110003@gmail.com
✓ Corregido: David Salas -> male

Total de correcciones de muestra aplicadas: 5

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA:
MALE: 1893 registros (55.1%)
FEMALE: 1451 registros (42.2%)
UNKNOWN: 93 registros (2.7%)

Registros 'unknown' restantes: 93

PROGRESO TOTAL ACUMULADO:
Unknown inicial: 515
Unknown actual: 93
Total mejorado: 422
Porcentaje de mejora: 81.9%


In [128]:
# Mostrar otros 30 registros con género 'unknown' que no hemos revisado anteriormente
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 30 registros con género unknown (muestra aleatoria nueva):")
    print("="*80)
    
    sample_size = min(30, len(df_unknown_current))
    # Usar un seed completamente diferente para obtener una muestra nueva
    random_sample_new = df_unknown_current.sample(n=sample_size, random_state=2025)
    
    for i, (idx, row) in enumerate(random_sample_new[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 93

Otros 30 registros con género unknown (muestra aleatoria nueva):
 1. Concienciactivz                          | first_name: N/A             | email: concienciactivz@gmail.com
 2. Ibarzabal                                | first_name: N/A             | email: ibarzabal@outlook.com
 3. Elieth Rivera                            | first_name: Elieth          | email: mariana.rivera@icatech.edu.mx
 4. Libertad Duenas                          | first_name: N/A             | email: liber.duengo@gmail.com
 5. Direccion                                | first_name: N/A             | email: direccion@scichihuahua.com
 6. Lpterrazas                               | first_name: N/A             | email: lpterrazas@uach.mx
 7. Comercial                                | first_name: N/A             | email: comercial@crocsa.com
 8. Leosausix                                | first_name: N/A             | email: leosausix@gmail.com
 9. Deny Frausto       

In [129]:
# Apply corrective changes for the identifiable names from the new sample
new_sample_corrections = [
    # Female names that are clearly identifiable
    {'email': 'mariana.rivera@icatech.edu.mx', 'name': 'Elieth Rivera', 'first_name': 'Elieth', 'gender': 'female'},
    {'email': 'liber.duengo@gmail.com', 'name': 'Libertad Duenas', 'first_name': 'Libertad', 'gender': 'female'},
    {'email': 'facd77@hotmail.com', 'name': 'Deny Frausto', 'first_name': 'Deny', 'gender': 'female'},
    {'email': 'msepulveda@naimexico.com', 'name': 'Marush Sepulveda', 'first_name': 'Marush', 'gender': 'female'},
    {'email': 'banigcm@gmail.com', 'name': 'Bani Giselle Carrasco Marquez', 'first_name': 'Bani', 'gender': 'female'},
    
    # Male names that are clearly identifiable
    {'email': 'auden.acosta@icatech.edu.mx', 'name': 'Auden Rodolfo Acosta Royval', 'first_name': 'Auden', 'gender': 'male'},
    {'email': '22110003@gmail.com', 'name': 'Jamzin Luviano', 'first_name': 'Jamzin', 'gender': 'male'}
]

print("Aplicando correcciones para la nueva muestra de registros unknown:")
print("="*60)

corrected_new_sample = 0
for correction in new_sample_corrections:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_new_sample += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones de nueva muestra aplicadas: {corrected_new_sample}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE NUEVA MUESTRA:")
print("="*60)
new_sample_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in new_sample_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_new_sample = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_new_sample}")

# Calcular progreso total acumulado
initial_unknown = 515
total_improved_new_sample = initial_unknown - remaining_unknown_new_sample
improvement_rate_new_sample = (total_improved_new_sample / initial_unknown) * 100

print(f"\nPROGRESO TOTAL ACUMULADO:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown actual: {remaining_unknown_new_sample}")
print(f"Total mejorado: {total_improved_new_sample}")
print(f"Porcentaje de mejora: {improvement_rate_new_sample:.1f}%")

Aplicando correcciones para la nueva muestra de registros unknown:
✓ Corregido: Elieth Rivera -> female
✓ Corregido: Libertad Duenas -> female
✓ Corregido: Deny Frausto -> female
✓ Corregido: Marush Sepulveda -> female
✓ Corregido: Bani Giselle Carrasco Marquez -> female
✓ Corregido: Auden Rodolfo Acosta Royval -> male
✓ Corregido: Jamzin Luviano -> male

Total de correcciones de nueva muestra aplicadas: 7

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE NUEVA MUESTRA:
MALE: 1895 registros (55.1%)
FEMALE: 1456 registros (42.4%)
UNKNOWN: 86 registros (2.5%)

Registros 'unknown' restantes: 86

PROGRESO TOTAL ACUMULADO:
Unknown inicial: 515
Unknown actual: 86
Total mejorado: 429
Porcentaje de mejora: 83.3%


In [130]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos visto antes
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 20 registros con género unknown (muestra aleatoria nueva):")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed completamente diferente para obtener una muestra nueva
    random_sample_72 = df_unknown_current.sample(n=sample_size, random_state=7272)
    
    for i, (idx, row) in enumerate(random_sample_72[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 86

Otros 20 registros con género unknown (muestra aleatoria nueva):
 1. Susochoam                                | first_name: N/A             | email: susochoam@gmail.com
 2. Ebustamante                              | first_name: N/A             | email: ebustamante@txtav.com
 3. Gossler                                  | first_name: N/A             | email: gossler@prodigy.net.mx
 4. Mx Comercial                             | first_name: N/A             | email: mx.comercial@iberdrola.com
 5. A                                        | first_name: N/A             | email: a01563142@tec.mx
 6. Asd                                      | first_name: Asd             | email: jamar123logi@protonmail.com
 7. R Nunez                                  | first_name: N/A             | email: r.nunez@ripipsa.com
 8. Relacionespublicas                       | first_name: N/A             | email: relacionespublicas@colegiodechihuahua.edu.mx
 9. S Loe

In [131]:
# Apply corrections for the identifiable names from the random sample
sample_corrections_73 = [
    # Clear male names
    {'email': 'pcertain@gcc.com', 'name': 'Pierre-Vincent Certain', 'first_name': 'Pierre-Vincent', 'gender': 'male'},
    {'email': 'harby.martinez@cimav.edu.mx', 'name': 'Harby Alexander Martinez Rodriguez', 'first_name': 'Harby', 'gender': 'male'},
    {'email': 'tellohernandez03@gmail.com', 'name': 'Saul Tello', 'first_name': 'Saul', 'gender': 'male'},
    {'email': '17213@ulsachihuahua.edu.mx', 'name': 'Iwe Teran Munoz', 'first_name': 'Iwe', 'gender': 'male'},
    {'email': 'awsusergroupchihuahua@gmail.com', 'name': 'Aws Usergroupchihuahua', 'first_name': 'Aws', 'gender': 'male'},
    
    # Clean up generic/company names that shouldn't be corrected
    # (keeping them as unknown since they're not personal names)
]

print("Aplicando correcciones para la muestra de registros unknown (celda 73):")
print("="*60)

corrected_sample_73 = 0
for correction in sample_corrections_73:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_sample_73 += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones aplicadas: {corrected_sample_73}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 73):")
print("="*60)
sample_73_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in sample_73_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_73 = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_73}")

# Calcular progreso total acumulado
initial_unknown = 515
total_improved_73 = initial_unknown - remaining_unknown_73
improvement_rate_73 = (total_improved_73 / initial_unknown) * 100

print(f"\nPROGRESO TOTAL ACUMULADO:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown actual: {remaining_unknown_73}")
print(f"Total mejorado: {total_improved_73}")
print(f"Porcentaje de mejora: {improvement_rate_73:.1f}%")

Aplicando correcciones para la muestra de registros unknown (celda 73):
✓ Corregido: Pierre-Vincent Certain -> male
✓ Corregido: Harby Alexander Martinez Rodriguez -> male
✓ Corregido: Saul Tello -> male
✓ Corregido: Iwe Teran Munoz -> male
✓ Corregido: Aws Usergroupchihuahua -> male

Total de correcciones aplicadas: 5

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 73):
MALE: 1900 registros (55.3%)
FEMALE: 1456 registros (42.4%)
UNKNOWN: 81 registros (2.4%)

Registros 'unknown' restantes: 81

PROGRESO TOTAL ACUMULADO:
Unknown inicial: 515
Unknown actual: 81
Total mejorado: 434
Porcentaje de mejora: 84.3%


In [132]:
# VERIFICAR CORRECCIONES APLICADAS - Mostrar registros unknown ACTUALIZADOS
print("VERIFICANDO CORRECCIONES APLICADAS:")
print("="*60)

# Crear lista actualizada de registros unknown (después de todas las correcciones)
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total ACTUALIZADO de registros con género 'unknown': {len(df_unknown_current)}")

# Verificar algunos registros específicos que se corrigieron
emails_corregidos = [
    'jberenice.mendoza@outlook.com',
    'hectordecionr@gmail.com', 
    'yodeseoun@gmail.com',
    'argel.gastelum@gmail.com',
    'cmacias@demek.com'
]

print("\nVerificando correcciones específicas:")
print("-" * 60)
for email in emails_corregidos:
    registro = df[df['email'] == email]
    if not registro.empty:
        nombre = registro['name'].iloc[0]
        first_name = registro['first_name'].iloc[0]
        genero = registro['gender_guess'].iloc[0]
        print(f"✓ {email}")
        print(f"  Nombre: {nombre} | First: {first_name} | Género: {genero}")
    else:
        print(f"✗ {email} - NO ENCONTRADO")

print(f"\nTop 20 registros ACTUALES con género unknown:")
print("="*80)

if len(df_unknown_current) > 0:
    # Mostrar los primeros 20 registros actuales
    top_20_current = df_unknown_current[['name', 'first_name', 'email', 'gender_guess']].head(20)
    for i, (idx, row) in enumerate(top_20_current.iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<35} | first_name: {first_name:<15} | email: {email}")
else:
    print("¡Genial! No hay registros con género 'unknown'")

VERIFICANDO CORRECCIONES APLICADAS:
Total ACTUALIZADO de registros con género 'unknown': 81

Verificando correcciones específicas:
------------------------------------------------------------
✓ jberenice.mendoza@outlook.com
  Nombre: Berenice Mendoza | First: Berenice | Género: female
✓ hectordecionr@gmail.com
  Nombre: Hector | First: Hector | Género: male
✓ yodeseoun@gmail.com
  Nombre: Yodes | First: Yodes | Género: male
✓ argel.gastelum@gmail.com
  Nombre: Argel Gastelum Arellanez | First: Argel | Género: male
✓ cmacias@demek.com
  Nombre: Cristiam Ivan Macias Sandoval | First: Cristiam | Género: male

Top 20 registros ACTUALES con género unknown:
 1. Mifam El Porvenir                   | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
 2. S Loer                              | first_name: N/A             | email: s.loer@outlook.com
 3. Leosausix                           | first_name: N/A             | email: leosausix@gmail.com
 4. Asistente                      

In [133]:
# APLICAR NUEVOS NOMBRES IDENTIFICADOS MANUALMENTE
print("APLICANDO NOMBRES ADICIONALES IDENTIFICADOS:")
print("="*60)

def final_gender_inference(row):
    """
    Función final de inferencia con todos los nombres identificados manualmente
    """
    first_name = str(row['first_name']) if pd.notna(row['first_name']) else ''
    name = str(row['name']) if pd.notna(row['name']) else ''
    
    # Lista completa de nombres identificados manualmente
    manual_names = {
        # Nombres masculinos
        'yahir': 'male', 'hiram': 'male', 'ever': 'male', 'jahziel': 'male',
        'jasiel': 'male', 'joanthan': 'male', 'jonathan': 'male', 'yefry': 'male',
        'gnolaum': 'male', 'eneas': 'male', 'zared': 'male', 'jovany': 'male',
        'freider': 'male', 'jahaziel': 'male', 'irad': 'male', 'jhonny': 'male', 
        'christoper': 'male', 'christopher': 'male', 'jassef': 'male', 'ariel': 'male', 'arad': 'male',
        'walter': 'male', 'wilson': 'male', 'winston': 'male', 'wilmer': 'male',
        'washington': 'male', 'wenceslao': 'male', 'wilfredo': 'male', 'william': 'male',
        'juancarlos': 'male', 'josemaria': 'male', 'luisalberto': 'male',
        'josé': 'male', 'jesús': 'male', 'andrés': 'male',
        
        # Nombres femeninos  
        'abril': 'female', 'areli': 'female', 'nahomy': 'female', 'estefany': 'female',
        'denisse': 'female', 'idis': 'female', 'yamlieth': 'female', 'lety': 'female',
        'verenice': 'female', 'rossy': 'female', 'idaly': 'female', 'mayrim': 'female',
        'yarelli': 'female', 'aide': 'female', 'aidé': 'female', 'itzeel': 'female', 'daneliz': 'female',
        'yuliana': 'female', 'xiomara': 'female', 'wendy': 'female', 'yesica': 'female',
        'yolanda': 'female', 'ximena': 'female', 'yasmin': 'female', 'yaneth': 'female',
        'mariaelena': 'female', 'analidia': 'female', 'mariajose': 'female',
        'maría': 'female'
    }
    
    # Extraer candidato
    candidate = ''
    if first_name and first_name != 'nan' and first_name.strip():
        candidate = first_name.split()[0].lower().strip()
    elif name and name != 'nan' and name.strip():
        candidate = name.split()[0].lower().strip()
    
    # Limpiar caracteres especiales
    candidate = candidate.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    candidate = candidate.replace('ñ', 'n').replace('.', '').replace(',', '').replace('(', '').replace(')', '')
    
    # Verificar en lista manual
    if candidate in manual_names:
        return manual_names[candidate]
    
    return "unknown"

# Aplicar la función final
print("Aplicando función final con todos los nombres identificados...")
mask_unknown = df["gender_guess"] == "unknown"
count_before = mask_unknown.sum()
print(f"Registros unknown antes: {count_before}")

# Mostrar los nombres que se van a procesar
print(f"\nNombres que se van a agregar:")
print("MASCULINOS:")
new_male_names = ['gnolaum', 'eneas', 'zared', 'jovany', 'freider', 'jahaziel', 'irad', 'jhonny', 'christoper', 'jassef', 'ariel', 'arad']
print(f"  {', '.join(new_male_names)}")
print("FEMENINOS:")
new_female_names = ['yarelli', 'aide/aidé', 'itzeel', 'daneliz']
print(f"  {', '.join(new_female_names)}")

# Aplicar inferencia a registros unknown
improved_final = 0
for idx in df[mask_unknown].index:
    inferred_gender = final_gender_inference(df.loc[idx])
    if inferred_gender != "unknown":
        df.loc[idx, 'gender_guess'] = inferred_gender
        improved_final += 1

print(f"\nNombres mejorados en esta iteración: {improved_final}")

# Mostrar distribución final
print(f"\nDISTRIBUCIÓN FINAL DE GÉNEROS:")
print("="*60)
final_counts = df["gender_guess"].value_counts()
total = len(df)
for gender, count in final_counts.items():
    percentage = (count / total) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown = (df["gender_guess"] == "unknown").sum()
total_improved_accumulated = 515 - remaining_unknown
improvement_rate = (total_improved_accumulated / 515) * 100

print(f"\nRESUMEN ACUMULADO DE TODAS LAS MEJORAS:")
print("="*50)
print(f"Unknown inicial: 515")
print(f"Unknown final: {remaining_unknown}")
print(f"Total mejorado acumulado: {total_improved_accumulated}")
print(f"Porcentaje de mejora total: {improvement_rate:.1f}%")

APLICANDO NOMBRES ADICIONALES IDENTIFICADOS:
Aplicando función final con todos los nombres identificados...
Registros unknown antes: 81

Nombres que se van a agregar:
MASCULINOS:
  gnolaum, eneas, zared, jovany, freider, jahaziel, irad, jhonny, christoper, jassef, ariel, arad
FEMENINOS:
  yarelli, aide/aidé, itzeel, daneliz

Nombres mejorados en esta iteración: 0

DISTRIBUCIÓN FINAL DE GÉNEROS:
MALE: 1900 registros (55.3%)
FEMALE: 1456 registros (42.4%)
UNKNOWN: 81 registros (2.4%)

RESUMEN ACUMULADO DE TODAS LAS MEJORAS:
Unknown inicial: 515
Unknown final: 81
Total mejorado acumulado: 434
Porcentaje de mejora total: 84.3%


In [134]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos revisado antes
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 20 registros con género unknown (muestra aleatoria nueva):")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed completamente diferente para obtener una muestra nueva
    random_sample_77 = df_unknown_current.sample(n=sample_size, random_state=7777)
    
    for i, (idx, row) in enumerate(random_sample_77[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 81

Otros 20 registros con género unknown (muestra aleatoria nueva):
 1. Plumas Novelty                           | first_name: N/A             | email: plumas_novelty@hotmail.com
 2. A                                        | first_name: N/A             | email: a01563142@tec.mx
 3. Relacionespublicas                       | first_name: N/A             | email: relacionespublicas@colegiodechihuahua.edu.mx
 4. A                                        | first_name: N/A             | email: a01569486@tec.mx
 5. Lmendez                                  | first_name: N/A             | email: lmendez9@ford.com
 6. Nazareth Luna                            | first_name: Nazareth        | email: nazarethgluna@gmail.com
 7. Lpterrazas                               | first_name: N/A             | email: lpterrazas@uach.mx
 8. Gossler                                  | first_name: N/A             | email: gossler@prodigy.net.mx
 9. Alizardi         

In [135]:
# Apply corrections for identifiable names from the sample shown
sample_corrections_78 = [
    # Clear female names
    {'email': 'nazarethgluna@gmail.com', 'name': 'Nazareth Luna', 'first_name': 'Nazareth', 'gender': 'female'},
    {'email': 'romanjazmin31@gmail.com', 'name': 'Jazmin Ayala', 'first_name': 'Jazmin', 'gender': 'female'},
    
    # Clear male names
    {'email': 'drjacquesmr@msn.com', 'name': 'Martin Jacques', 'first_name': 'Martin', 'gender': 'male'},
    
    # Company/generic names that should remain unknown or be cleaned up
    # (keeping them as unknown since they're not personal names)
]

print("Aplicando correcciones para la muestra de registros unknown (celda 78):")
print("="*60)

corrected_sample_78 = 0
for correction in sample_corrections_78:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_sample_78 += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones aplicadas (celda 78): {corrected_sample_78}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 78):")
print("="*60)
sample_78_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in sample_78_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_78 = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_78}")

# Calcular progreso total acumulado
initial_unknown = 515
total_improved_78 = initial_unknown - remaining_unknown_78
improvement_rate_78 = (total_improved_78 / initial_unknown) * 100

print(f"\nPROGRESO TOTAL ACUMULADO:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown actual: {remaining_unknown_78}")
print(f"Total mejorado: {total_improved_78}")
print(f"Porcentaje de mejora: {improvement_rate_78:.1f}%")

Aplicando correcciones para la muestra de registros unknown (celda 78):
✓ Corregido: Nazareth Luna -> female
✓ Corregido: Jazmin Ayala -> female
✓ Corregido: Martin Jacques -> male

Total de correcciones aplicadas (celda 78): 3

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 78):
MALE: 1901 registros (55.3%)
FEMALE: 1458 registros (42.4%)
UNKNOWN: 78 registros (2.3%)

Registros 'unknown' restantes: 78

PROGRESO TOTAL ACUMULADO:
Unknown inicial: 515
Unknown actual: 78
Total mejorado: 437
Porcentaje de mejora: 84.9%


In [136]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos revisado anteriormente
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 20 registros con género unknown (muestra aleatoria nueva):")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed completamente diferente para obtener una muestra nueva
    random_sample_79 = df_unknown_current.sample(n=sample_size, random_state=7979)
    
    for i, (idx, row) in enumerate(random_sample_79[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 78

Otros 20 registros con género unknown (muestra aleatoria nueva):
 1. Rh                                       | first_name: N/A             | email: rh@direchisa.com
 2. Winorax                                  | first_name: N/A             | email: winorax@hotmail.com
 3. Galvidrez                                | first_name: N/A             | email: galvidrez@aiig.com
 4. Xontiveros                               | first_name: N/A             | email: xontiveros@tisco.mx
 5. Cchavez                                  | first_name: N/A             | email: cchavez@ienova.com.mx
 6. Publicidad                               | first_name: N/A             | email: publicidad@casamyers.com.mx
 7. Litzel Rodriguez                         | first_name: Litzel          | email: litzel.rodriguez@chihuahua.com.mx
 8. Carsi                                    | first_name: N/A             | email: carsi0408@icloud.com
 9. Camelot Arte Cultura      

In [137]:
# Aplicar correcciones para nombres que se pueden inferir fácilmente de la muestra
sample_corrections_79 = [
    # Female names that are clearly identifiable
    {'email': 'litzel.rodriguez@chihuahua.com.mx', 'name': 'Litzel Rodriguez', 'first_name': 'Litzel', 'gender': 'female'},
    {'email': 'dayanne.gonzalez@kalischacero.com', 'name': 'Dayanne Gonzalez', 'first_name': 'Dayanne', 'gender': 'female'},
    {'email': 'madelin.perete@madatait.com', 'name': 'Madelin Perete', 'first_name': 'Madelin', 'gender': 'female'},
    
    # Male names that are clearly identifiable
    {'email': 'ixrael.silva@gmail.com', 'name': 'Ixrael Silva', 'first_name': 'Ixrael', 'gender': 'male'},
    
    # Clean up company/generic names (keeping as unknown since they're not personal names)
]

print("Aplicando correcciones para la muestra de registros unknown (celda 80):")
print("="*60)

corrected_sample_79 = 0
for correction in sample_corrections_79:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_sample_79 += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones aplicadas (celda 80): {corrected_sample_79}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 80):")
print("="*60)
sample_79_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in sample_79_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_79 = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_79}")

# Calcular progreso total acumulado
initial_unknown = 515
total_improved_79 = initial_unknown - remaining_unknown_79
improvement_rate_79 = (total_improved_79 / initial_unknown) * 100

print(f"\nPROGRESO TOTAL ACUMULADO:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown actual: {remaining_unknown_79}")
print(f"Total mejorado: {total_improved_79}")
print(f"Porcentaje de mejora: {improvement_rate_79:.1f}%")

Aplicando correcciones para la muestra de registros unknown (celda 80):
✓ Corregido: Litzel Rodriguez -> female
✓ Corregido: Dayanne Gonzalez -> female
✓ Corregido: Madelin Perete -> female
✓ Corregido: Ixrael Silva -> male

Total de correcciones aplicadas (celda 80): 4

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 80):
MALE: 1902 registros (55.3%)
FEMALE: 1461 registros (42.5%)
UNKNOWN: 74 registros (2.2%)

Registros 'unknown' restantes: 74

PROGRESO TOTAL ACUMULADO:
Unknown inicial: 515
Unknown actual: 74
Total mejorado: 441
Porcentaje de mejora: 85.6%


In [138]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos revisado anteriormente
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 20 registros con género unknown (muestra aleatoria nueva):")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed completamente diferente para obtener una muestra nueva
    random_sample_81 = df_unknown_current.sample(n=sample_size, random_state=8181)
    
    for i, (idx, row) in enumerate(random_sample_81[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 74

Otros 20 registros con género unknown (muestra aleatoria nueva):
 1. Vmontes                                  | first_name: N/A             | email: vmontes@aiig.com
 2. S Loer                                   | first_name: N/A             | email: s.loer@outlook.com
 3. Rh                                       | first_name: N/A             | email: rh@direchisa.com
 4. Carsi                                    | first_name: N/A             | email: carsi0408@icloud.com
 5. Amonroy                                  | first_name: N/A             | email: amonroy@hospitalcima.com.mx
 6. Cchavez                                  | first_name: N/A             | email: cchavez@ienova.com.mx
 7. Plumas Novelty                           | first_name: N/A             | email: plumas_novelty@hotmail.com
 8. A                                        | first_name: N/A             | email: a295727@uach.mx
 9. Mifam El Porvenir                       

In [139]:
# Apply corrections for identifiable names from the sample shown
sample_corrections_82 = [
    # Clear female name
    {'email': 'carsi0408@icloud.com', 'name': 'Carsi Rodriguez', 'first_name': 'Carsi', 'gender': 'female'},
    
    # Clear male name - correcting Miguek to Miguel
    {'email': 'mike.in.touch.2002@gmail.com', 'name': 'Miguel Duran Rodriguez', 'first_name': 'Miguel', 'gender': 'male'},
    
    # These are company/generic names that should remain as unknown:
    # - Plumas Novelty (company)
    # - Asistente (generic role)
    # - Contabilidad (generic role) 
    # - Comercial (generic role)
    # - Promociones (generic role)
    # - A (incomplete/generic entries)
    # - Various company email prefixes (vmontes, pgomez, etc.)
]

print("Aplicando correcciones para la muestra de registros unknown (celda 82):")
print("="*60)

corrected_sample_82 = 0
for correction in sample_corrections_82:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_sample_82 += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones aplicadas (celda 82): {corrected_sample_82}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 82):")
print("="*60)
sample_82_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in sample_82_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_82 = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_82}")

# Calcular progreso total acumulado
initial_unknown = 515
total_improved_82 = initial_unknown - remaining_unknown_82
improvement_rate_82 = (total_improved_82 / initial_unknown) * 100

print(f"\nPROGRESO TOTAL ACUMULADO:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown actual: {remaining_unknown_82}")
print(f"Total mejorado: {total_improved_82}")
print(f"Porcentaje de mejora: {improvement_rate_82:.1f}%")

print(f"\nNOTA: Los nombres genéricos como 'Asistente', 'Contabilidad', 'Comercial', etc.")
print(f"se mantienen como 'unknown' ya que representan roles empresariales, no nombres personales.")

Aplicando correcciones para la muestra de registros unknown (celda 82):
✓ Corregido: Carsi Rodriguez -> female
✓ Corregido: Miguel Duran Rodriguez -> male

Total de correcciones aplicadas (celda 82): 2

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 82):
MALE: 1903 registros (55.4%)
FEMALE: 1462 registros (42.5%)
UNKNOWN: 72 registros (2.1%)

Registros 'unknown' restantes: 72

PROGRESO TOTAL ACUMULADO:
Unknown inicial: 515
Unknown actual: 72
Total mejorado: 443
Porcentaje de mejora: 86.0%

NOTA: Los nombres genéricos como 'Asistente', 'Contabilidad', 'Comercial', etc.
se mantienen como 'unknown' ya que representan roles empresariales, no nombres personales.


In [140]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos revisado anteriormente
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 20 registros con género unknown (muestra aleatoria nueva):")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed completamente diferente para obtener una muestra nueva
    random_sample_83 = df_unknown_current.sample(n=sample_size, random_state=8383)
    
    for i, (idx, row) in enumerate(random_sample_83[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 72

Otros 20 registros con género unknown (muestra aleatoria nueva):
 1. Ibarzabal                                | first_name: N/A             | email: ibarzabal@outlook.com
 2. Asd                                      | first_name: Asd             | email: jamar123logi@protonmail.com
 3. Ventas                                   | first_name: N/A             | email: ventas@tacglobalsolutions.com
 4. Vmontes                                  | first_name: N/A             | email: vmontes@aiig.com
 5. Contabilidad                             | first_name: N/A             | email: contabilidad@abainst.com.mx
 6. Mifam El Porvenir                        | first_name: Mifam           | email: mifamulsachihuahua@gmail.com
 7. Ebustamante                              | first_name: N/A             | email: ebustamante@txtav.com
 8. Direccion                                | first_name: N/A             | email: direccion@scichihuahua.com
 9. Aliz

In [141]:
# Apply corrections for identifiable names from the sample shown
sample_corrections_83 = [
    # Clear female name
    {'email': 'fierronahomi58@gmail.com', 'name': 'Nahomi Fierro', 'first_name': 'Nahomi', 'gender': 'female'},
    
    # These are company/generic names that should remain as unknown:
    # - Ibarzabal (surname/company)
    # - Ventas (generic sales role)
    # - Vmontes (company initials)
    # - Contabilidad (accounting department)
    # - Mifam El Porvenir (company/organization)
    # - Ebustamante (company initials)
    # - Direccion (direction/management)
    # - Alizardi (company/surname)
    # - Base2 Fechac (company/organization)
    # - Admon (administration)
    # - Fp (initials)
    # - Red Fision (company name)
    # - R Nunez (initial + surname)
    # - Galvidrez (surname)
    # - Gtegeneral (company role)
    # - Cxc (accounts receivable)
    # - Mifam Canaco (organization)
    # - Asd (random letters)
]

print("Aplicando correcciones para la muestra de registros unknown (celda 84):")
print("="*60)

corrected_sample_83 = 0
for correction in sample_corrections_83:
    mask = df['email'] == correction['email']
    if mask.any():
        # Actualizar datos
        df.loc[mask, 'name'] = correction['name']
        df.loc[mask, 'first_name'] = correction['first_name']
        df.loc[mask, 'gender_guess'] = correction['gender']
        df.loc[mask, 'full_name'] = correction['name'].strip().lower()
        
        corrected_sample_83 += 1
        print(f"✓ Corregido: {correction['name']} -> {correction['gender']}")
    else:
        print(f"✗ No encontrado: {correction['email']}")

print(f"\nTotal de correcciones aplicadas (celda 84): {corrected_sample_83}")

# Mostrar distribución actualizada
print(f"\nDISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 84):")
print("="*60)
sample_83_counts = df["gender_guess"].value_counts()
total_records = len(df)
for gender, count in sample_83_counts.items():
    percentage = (count / total_records) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

remaining_unknown_83 = (df["gender_guess"] == "unknown").sum()
print(f"\nRegistros 'unknown' restantes: {remaining_unknown_83}")

# Calcular progreso total acumulado
initial_unknown = 515
total_improved_83 = initial_unknown - remaining_unknown_83
improvement_rate_83 = (total_improved_83 / initial_unknown) * 100

print(f"\nPROGRESO TOTAL ACUMULADO:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown actual: {remaining_unknown_83}")
print(f"Total mejorado: {total_improved_83}")
print(f"Porcentaje de mejora: {improvement_rate_83:.1f}%")

print(f"\nNOTA: Los nombres genéricos como 'Ventas', 'Contabilidad', 'Direccion', etc.")
print(f"se mantienen como 'unknown' ya que representan roles empresariales o iniciales, no nombres personales.")

Aplicando correcciones para la muestra de registros unknown (celda 84):
✓ Corregido: Nahomi Fierro -> female

Total de correcciones aplicadas (celda 84): 1

DISTRIBUCIÓN DESPUÉS DE CORRECCIONES DE MUESTRA (CELDA 84):
MALE: 1903 registros (55.4%)
FEMALE: 1463 registros (42.6%)
UNKNOWN: 71 registros (2.1%)

Registros 'unknown' restantes: 71

PROGRESO TOTAL ACUMULADO:
Unknown inicial: 515
Unknown actual: 71
Total mejorado: 444
Porcentaje de mejora: 86.2%

NOTA: Los nombres genéricos como 'Ventas', 'Contabilidad', 'Direccion', etc.
se mantienen como 'unknown' ya que representan roles empresariales o iniciales, no nombres personales.


In [142]:
# Mostrar otros 20 registros con género 'unknown' que no hayamos revisado anteriormente
df_unknown_current = df[df['gender_guess'] == 'unknown'].copy()

print(f"Total actual de registros con género 'unknown': {len(df_unknown_current)}")

if len(df_unknown_current) > 0:
    # Mostrar una muestra aleatoria diferente usando un seed único
    print("\nOtros 20 registros con género unknown (muestra aleatoria nueva):")
    print("="*80)
    
    sample_size = min(20, len(df_unknown_current))
    # Usar un seed completamente diferente para obtener una muestra nueva
    random_sample_85 = df_unknown_current.sample(n=sample_size, random_state=8585)
    
    for i, (idx, row) in enumerate(random_sample_85[['name', 'first_name', 'email', 'gender_guess']].iterrows(), 1):
        name = row['name'] if pd.notna(row['name']) else 'N/A'
        first_name = row['first_name'] if pd.notna(row['first_name']) else 'N/A'
        email = row['email']
        print(f"{i:2d}. {name:<40} | first_name: {first_name:<15} | email: {email}")
        
    print(f"\nESTADÍSTICAS ACTUALIZADAS:")
    print("="*40)
    current_distribution = df["gender_guess"].value_counts()
    total = len(df)
    for gender, count in current_distribution.items():
        percentage = (count / total) * 100
        print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")
        
    # Calcular progreso desde el inicio
    initial_unknown = 515
    current_unknown = len(df_unknown_current)
    total_improved = initial_unknown - current_unknown
    improvement_rate = (total_improved / initial_unknown) * 100
    
    print(f"\nPROGRESO DESDE EL INICIO:")
    print("="*30)
    print(f"Unknown inicial: {initial_unknown}")
    print(f"Unknown actual: {current_unknown}")
    print(f"Total mejorado: {total_improved}")
    print(f"Porcentaje de mejora: {improvement_rate:.1f}%")
else:
    print("¡Excelente! No hay registros con género 'unknown'")

Total actual de registros con género 'unknown': 71

Otros 20 registros con género unknown (muestra aleatoria nueva):
 1. Mifam Canaco                             | first_name: N/A             | email: hola@inndech.org
 2. Galvidrez                                | first_name: N/A             | email: galvidrez@aiig.com
 3. Lvargas                                  | first_name: N/A             | email: lvargas@interceramic.com
 4. Fp                                       | first_name: N/A             | email: fp456820@gmail.com
 5. A                                        | first_name: N/A             | email: a01562935@tec.mx
 6. Asistente                                | first_name: N/A             | email: asistente@medicalkids.com.mx
 7. Dmongeb                                  | first_name: N/A             | email: dmongeb@gcc.com
 8. Contabilidad                             | first_name: N/A             | email: contabilidad@abainst.com.mx
 9. Amonroy                              

In [143]:
# Mostrar los primeros 20 registros del dataframe corregido
print("PRIMEROS 20 REGISTROS DEL DATAFRAME CORREGIDO:")
print("="*80)
print(df.head(20))

# También mostrar información resumida del dataframe
print(f"\nINFORMACIÓN GENERAL DEL DATAFRAME:")
print("="*40)
print(f"Total de registros: {len(df)}")
print(f"Columnas: {list(df.columns)}")

# Mostrar distribución final de géneros
print(f"\nDISTRIBUCIÓN FINAL DE GÉNEROS:")
print("="*40)
final_gender_distribution = df["gender_guess"].value_counts()
total = len(df)
for gender, count in final_gender_distribution.items():
    percentage = (count / total) * 100
    print(f"{gender.upper()}: {count} registros ({percentage:.1f}%)")

# Mostrar estadísticas de mejora
initial_unknown = 515
current_unknown = (df["gender_guess"] == "unknown").sum()
total_improved = initial_unknown - current_unknown
improvement_rate = (total_improved / initial_unknown) * 100

print(f"\nESTADÍSTICAS DE MEJORA:")
print("="*30)
print(f"Unknown inicial: {initial_unknown}")
print(f"Unknown final: {current_unknown}")
print(f"Total mejorado: {total_improved}")
print(f"Porcentaje de mejora: {improvement_rate:.1f}%")

PRIMEROS 20 REGISTROS DEL DATAFRAME CORREGIDO:
                                   name   first_name  \
0   Cristian Gian Pieer Sanchez Martell       Phayro   
1                           Alex Palomo         Alex   
2                       Laura Hernandez        Laura   
3               Juan Carlos Silva Jaime  Juan Carlos   
4        Cecilia Alexandra Soto Andrade      Cecilia   
5                      Stephanie Jaquez    Stephanie   
6            Stephanie Navarrete Jaquez    Stephanie   
7                             Luis Diaz         Luis   
8                           Johan Bader        Johan   
9                        Sophia Alarcon       Sophia   
10                    Christopher Bravo  Christopher   
11                     Alejandro Arroyo    Alejandro   
12   Gnolaum Alejjandro Cervantes Tarin      Gnolaum   
13                          Max Morales          Max   
14        Yarelli Abigail Palomec Ortiz      Yarelli   
15                        Jaime Jimenez        Jaime   
1

In [144]:
# Mostrar los primeros 20 registros del dataframe completo con todas las columnas
print("PRIMEROS 20 REGISTROS DEL DATAFRAME COMPLETO:")
print("="*100)
print(df.head(20))

print(f"\nINFORMACIÓN GENERAL DEL DATAFRAME:")
print("="*50)
print(f"Total de registros: {len(df)}")
print(f"Total de columnas: {len(df.columns)}")
print(f"Columnas disponibles: {list(df.columns)}")

# Mostrar información más detallada del dataframe
print(f"\nINFORMACIÓN DETALLADA:")
print("="*30)
df.info()

# Mostrar las primeras 50 filas para tener una vista más amplia
print(f"\nPRIMEROS 50 REGISTROS:")
print("="*50)
print(df.head(50))

PRIMEROS 20 REGISTROS DEL DATAFRAME COMPLETO:
                                   name   first_name  \
0   Cristian Gian Pieer Sanchez Martell       Phayro   
1                           Alex Palomo         Alex   
2                       Laura Hernandez        Laura   
3               Juan Carlos Silva Jaime  Juan Carlos   
4        Cecilia Alexandra Soto Andrade      Cecilia   
5                      Stephanie Jaquez    Stephanie   
6            Stephanie Navarrete Jaquez    Stephanie   
7                             Luis Diaz         Luis   
8                           Johan Bader        Johan   
9                        Sophia Alarcon       Sophia   
10                    Christopher Bravo  Christopher   
11                     Alejandro Arroyo    Alejandro   
12   Gnolaum Alejjandro Cervantes Tarin      Gnolaum   
13                          Max Morales          Max   
14        Yarelli Abigail Palomec Ortiz      Yarelli   
15                        Jaime Jimenez        Jaime   
16

In [145]:
# Guardar el dataframe procesado y limpio
df.to_csv('data/activ_processed.csv', index=False)
print("DataFrame procesado guardado como 'data/activ_processed.csv'")

# Mostrar resumen final completo
print("\n" + "="*80)
print("RESUMEN FINAL COMPLETO DEL PROCESAMIENTO DE DATOS")
print("="*80)

print(f"\nDATOS GENERALES:")
print(f"- Total de registros: {len(df):,}")
print(f"- Total de columnas: {len(df.columns)}")

print(f"\nDISTRIBUCIÓN FINAL DE GÉNEROS:")
final_gender_stats = df['gender_guess'].value_counts()
total_records = len(df)
for gender, count in final_gender_stats.items():
    percentage = (count / total_records) * 100
    print(f"- {gender.upper()}: {count:,} registros ({percentage:.1f}%)")

print(f"\nMEJORAS REALIZADAS EN DETECCIÓN DE GÉNERO:")
initial_unknown = 515
final_unknown = (df['gender_guess'] == 'unknown').sum()
total_improved = initial_unknown - final_unknown
improvement_rate = (total_improved / initial_unknown) * 100

print(f"- Registros 'unknown' iniciales: {initial_unknown:,}")
print(f"- Registros 'unknown' finales: {final_unknown:,}")
print(f"- Total de registros mejorados: {total_improved:,}")
print(f"- Porcentaje de mejora: {improvement_rate:.1f}%")

print(f"\nCALIDAD DE DATOS:")
# Estadísticas de nombres vacíos
empty_names = df['name'].isna().sum() + (df['name'].str.strip() == '').sum()
empty_first_names = df['first_name'].isna().sum()
print(f"- Registros con 'name' vacío: {empty_names:,}")
print(f"- Registros con 'first_name' vacío: {empty_first_names:,}")

# Estadísticas de duplicados
duplicates_remaining = df[df['full_name'] != '']['full_name'].duplicated().sum()
print(f"- Duplicados restantes (por full_name): {duplicates_remaining:,}")

print(f"\nCOLUMNAS DISPONIBLES:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nPRIMEROS 10 REGISTROS DEL DATASET FINAL:")
print("-" * 80)
display_columns = ['name', 'first_name', 'email', 'gender_guess', 'full_name']
print(df[display_columns].head(10).to_string(index=False))

print(f"\n" + "="*80)
print("PROCESAMIENTO COMPLETADO EXITOSAMENTE")
print("="*80)

DataFrame procesado guardado como 'data/activ_processed.csv'

RESUMEN FINAL COMPLETO DEL PROCESAMIENTO DE DATOS

DATOS GENERALES:
- Total de registros: 3,437
- Total de columnas: 23

DISTRIBUCIÓN FINAL DE GÉNEROS:
- MALE: 1,903 registros (55.4%)
- FEMALE: 1,463 registros (42.6%)
- UNKNOWN: 71 registros (2.1%)

MEJORAS REALIZADAS EN DETECCIÓN DE GÉNERO:
- Registros 'unknown' iniciales: 515
- Registros 'unknown' finales: 71
- Total de registros mejorados: 444
- Porcentaje de mejora: 86.2%

CALIDAD DE DATOS:
- Registros con 'name' vacío: 0
- Registros con 'first_name' vacío: 589
- Duplicados restantes (por full_name): 6

COLUMNAS DISPONIBLES:
 1. name
 2. first_name
 3. last_name
 4. email
 5. first_seen
 6. user_api_id
 7. tags
 8. revenue
 9. event_approved_count
10. event_checked_in_count
11. membership_name
12. membership_status
13. email_local
14. full_name
15. name_from_email
16. has_two_first_names
17. first_name_1
18. first_name_2
19. has_two_last_names
20. last_name_1
21. last_na

In [146]:
df.head(20)

,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,...,full_name,name_from_email,has_two_first_names,first_name_1,first_name_2,has_two_last_names,last_name_1,last_name_2,name_inferred,gender_guess
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,...,cristian gian pieer sanchez martell,Phayromartellfalcon,False,Phayro,NaN,True,Martell,Falcon,,male
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,...,alex palomo,Alexpalomo,False,Alex,NaN,False,Palomo,NaN,,male
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,...,laura hernandez,Laurar,False,Laura,NaN,False,Hernandez,NaN,,female
3,Juan Carlos Silva Jaime,Juan Carlos,Silva Jaime,jcsilva@uach.mx,2025-09-28T19:16:02.033Z,usr-Fq8IHykAHIjcT1M,NaN,"0,00 US$",1,0,...,juan carlos silva jaime,Jcsilva,True,Juan,Carlos,True,Silva,Jaime,,male
4,Cecilia Alexandra Soto Andrade,Cecilia,Soto Andrade,sotoandrade069@gmail.com,2025-09-26T02:38:33.212Z,usr-U8jYkpT4BrtuA7k,NaN,"0,00 US$",1,0,...,cecilia alexandra soto andrade,Sotoandrade,False,Cecilia,NaN,True,Soto,Andrade,,female
5,Stephanie Jaquez,Stephanie,Jaquez,mario.simental0208@gmail.com,2025-09-26T01:57:45.053Z,usr-ShowEBU58V81RCy,NaN,"0,00 US$",1,0,...,stephanie jaquez,Mario Simental,False,Stephanie,NaN,False,Jaquez,NaN,Mario Simental,female
6,Stephanie Navarrete Jaquez,Stephanie,Navarrete Jaquez,fannyjaquez4566@gmail.com,2025-09-26T01:53:27.884Z,usr-73xJIFmdQJNY5hK,NaN,"0,00 US$",1,0,...,stephanie navarrete jaquez,Fannyjaquez,False,Stephanie,NaN,True,Navarrete,Jaquez,,female
7,Luis Diaz,Luis,Diaz,mi6141526425@gmail.com,2025-09-26T01:52:16.008Z,usr-NlNR7sHw59X7mSK,NaN,"0,00 US$",1,0,...,luis diaz,Mi,False,Luis,NaN,False,Diaz,NaN,,male
8,Johan Bader,Johan,Bader,maxkmp22@gmail.com,2025-09-26T01:09:06.206Z,usr-W1U23T9UAnod66J,NaN,"0,00 US$",1,0,...,johan bader,Maxkmp,False,Johan,NaN,False,Bader,NaN,,male
9,Sophia Alarcon,Sophia,Alarcón,sophiaalarcon22@outlook.com,2025-09-26T01:06:40.867Z,usr-IeEfD4GdfDvX59m,NaN,"0,00 US$",1,0,...,sophia alarcon,Sophiaalarcon,False,Sophia,NaN,False,Alarcón,NaN,,female


In [147]:
df.to_excel('data/activ_processed.xlsx', index=False)
print("DataFrame procesado guardado como 'data/activ_processed.xlsx'")

DataFrame procesado guardado como 'data/activ_processed.xlsx'


In [148]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [149]:
df.to_excel('data/activ_processed.xlsx', index=False)
print("DataFrame procesado guardado como 'data/activ_processed2.xlsx'")

DataFrame procesado guardado como 'data/activ_processed2.xlsx'


In [150]:
# Mostrar resumen del dataset exportado con géneros
print("RESUMEN DEL DATASET EXPORTADO CON GÉNEROS:")
print("="*60)
print(f"Archivo guardado: data/activ_processed.xlsx")
print(f"Total de registros: {len(df):,}")
print(f"Total de columnas: {len(df.columns)}")

print(f"\nDistribución de géneros:")
gender_stats = df['gender_guess'].value_counts()
total = len(df)
for gender, count in gender_stats.items():
    percentage = (count / total) * 100
    print(f"  {gender.upper()}: {count:,} registros ({percentage:.1f}%)")

print(f"\nColumnas incluidas:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nPrimeros 10 registros con género:")
print(df[['name', 'first_name', 'email', 'gender_guess']].head(10))

RESUMEN DEL DATASET EXPORTADO CON GÉNEROS:
Archivo guardado: data/activ_processed.xlsx
Total de registros: 3,437
Total de columnas: 23

Distribución de géneros:
  MALE: 1,903 registros (55.4%)
  FEMALE: 1,463 registros (42.6%)
  UNKNOWN: 71 registros (2.1%)

Columnas incluidas:
   1. name
   2. first_name
   3. last_name
   4. email
   5. first_seen
   6. user_api_id
   7. tags
   8. revenue
   9. event_approved_count
  10. event_checked_in_count
  11. membership_name
  12. membership_status
  13. email_local
  14. full_name
  15. name_from_email
  16. has_two_first_names
  17. first_name_1
  18. first_name_2
  19. has_two_last_names
  20. last_name_1
  21. last_name_2
  22. name_inferred
  23. gender_guess

Primeros 10 registros con género:
                                  name   first_name  \
0  Cristian Gian Pieer Sanchez Martell       Phayro   
1                          Alex Palomo         Alex   
2                      Laura Hernandez        Laura   
3              Juan Carlos S

In [151]:
# Verificar estado actual del dataframe y ejecutar procesamiento completo
print("ESTADO ACTUAL DEL DATAFRAME:")
print("="*60)
print(f"Columnas actuales: {list(df.columns)}")
print(f"Forma del dataframe: {df.shape}")
print(f"\nPrimeros 3 registros:")
df.head(3)

ESTADO ACTUAL DEL DATAFRAME:
Columnas actuales: ['name', 'first_name', 'last_name', 'email', 'first_seen', 'user_api_id', 'tags', 'revenue', 'event_approved_count', 'event_checked_in_count', 'membership_name', 'membership_status', 'email_local', 'full_name', 'name_from_email', 'has_two_first_names', 'first_name_1', 'first_name_2', 'has_two_last_names', 'last_name_1', 'last_name_2', 'name_inferred', 'gender_guess']
Forma del dataframe: (3437, 23)

Primeros 3 registros:


,name,first_name,last_name,email,first_seen,user_api_id,tags,revenue,event_approved_count,event_checked_in_count,...,full_name,name_from_email,has_two_first_names,first_name_1,first_name_2,has_two_last_names,last_name_1,last_name_2,name_inferred,gender_guess
0,Cristian Gian Pieer Sanchez Martell,Phayro,Martell Falcon,phayromartellfalcon380@gmail.com,2025-11-05T18:02:18.311Z,usr-XaU09ItB6pz2X9C,NaN,"0,00 US$",1,0,...,cristian gian pieer sanchez martell,Phayromartellfalcon,False,Phayro,NaN,True,Martell,Falcon,,male
1,Alex Palomo,Alex,Palomo,alexpalomo@legionhack.com,2025-10-29T04:31:23.026Z,usr-4EDx7IlDtKsuOhs,NaN,"0,00 US$",0,0,...,alex palomo,Alexpalomo,False,Alex,NaN,False,Palomo,NaN,,male
2,Laura Hernandez,Laura,Hernandez,laurar1803@gmail.com,2025-10-06T16:52:48.018Z,usr-GthLF8nuwdrv4II,NaN,"0,00 US$",1,0,...,laura hernandez,Laurar,False,Laura,NaN,False,Hernandez,NaN,,female


In [152]:
# Exportar solo las columnas solicitadas: name, email, gender_guess
df_export = df[['name', 'email', 'gender_guess']].copy()

# Guardar en Excel
df_export.to_excel('data/dataset_name_email_gender.xlsx', index=False)

print("ARCHIVO EXCEL EXPORTADO:")
print("="*50)
print(f"Archivo: data/dataset_name_email_gender.xlsx")
print(f"Total de registros: {len(df_export):,}")
print(f"Columnas incluidas: {list(df_export.columns)}")

print(f"\nDistribución de géneros:")
gender_distribution = df_export['gender_guess'].value_counts()
for gender, count in gender_distribution.items():
    percentage = (count / len(df_export)) * 100
    print(f"  {gender.upper()}: {count:,} ({percentage:.1f}%)")

print(f"\nPrimeros 10 registros exportados:")
df_export.head(10)

ARCHIVO EXCEL EXPORTADO:
Archivo: data/dataset_name_email_gender.xlsx
Total de registros: 3,437
Columnas incluidas: ['name', 'email', 'gender_guess']

Distribución de géneros:
  MALE: 1,903 (55.4%)
  FEMALE: 1,463 (42.6%)
  UNKNOWN: 71 (2.1%)

Primeros 10 registros exportados:


,name,email,gender_guess
0,Cristian Gian Pieer Sanchez Martell,phayromartellfalcon380@gmail.com,male
1,Alex Palomo,alexpalomo@legionhack.com,male
2,Laura Hernandez,laurar1803@gmail.com,female
3,Juan Carlos Silva Jaime,jcsilva@uach.mx,male
4,Cecilia Alexandra Soto Andrade,sotoandrade069@gmail.com,female
5,Stephanie Jaquez,mario.simental0208@gmail.com,female
6,Stephanie Navarrete Jaquez,fannyjaquez4566@gmail.com,female
7,Luis Diaz,mi6141526425@gmail.com,male
8,Johan Bader,maxkmp22@gmail.com,male
9,Sophia Alarcon,sophiaalarcon22@outlook.com,female


In [153]:
# Exportar el dataframe completo con todos los cambios de género aplicados
df.to_excel('data/activ_complete_with_gender.xlsx', index=False)

print("ARCHIVO EXCEL EXPORTADO CON TODOS LOS CAMBIOS:")
print("="*60)
print(f"Archivo: data/activ_complete_with_gender.xlsx")
print(f"Total de registros: {len(df):,}")
print(f"Total de columnas: {len(df.columns)}")

# Mostrar distribución final de géneros
print(f"\nDistribución final de géneros:")
final_gender_stats = df['gender_guess'].value_counts()
total_records = len(df)
for gender, count in final_gender_stats.items():
    percentage = (count / total_records) * 100
    print(f"  {gender.upper()}: {count:,} registros ({percentage:.1f}%)")

# Mostrar resumen de mejoras
initial_unknown = 515
current_unknown = (df['gender_guess'] == 'unknown').sum()
total_improved = initial_unknown - current_unknown
improvement_rate = (total_improved / initial_unknown) * 100

print(f"\nResumen de mejoras en detección de género:")
print(f"  Registros unknown iniciales: {initial_unknown:,}")
print(f"  Registros unknown finales: {current_unknown:,}")
print(f"  Total de registros mejorados: {total_improved:,}")
print(f"  Porcentaje de mejora: {improvement_rate:.1f}%")

print(f"\nColumnas incluidas en el archivo:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nPrimeros 10 registros exportados:")
print(df[['name', 'first_name', 'email', 'gender_guess']].head(10))

ARCHIVO EXCEL EXPORTADO CON TODOS LOS CAMBIOS:
Archivo: data/activ_complete_with_gender.xlsx
Total de registros: 3,437
Total de columnas: 23

Distribución final de géneros:
  MALE: 1,903 registros (55.4%)
  FEMALE: 1,463 registros (42.6%)
  UNKNOWN: 71 registros (2.1%)

Resumen de mejoras en detección de género:
  Registros unknown iniciales: 515
  Registros unknown finales: 71
  Total de registros mejorados: 444
  Porcentaje de mejora: 86.2%

Columnas incluidas en el archivo:
   1. name
   2. first_name
   3. last_name
   4. email
   5. first_seen
   6. user_api_id
   7. tags
   8. revenue
   9. event_approved_count
  10. event_checked_in_count
  11. membership_name
  12. membership_status
  13. email_local
  14. full_name
  15. name_from_email
  16. has_two_first_names
  17. first_name_1
  18. first_name_2
  19. has_two_last_names
  20. last_name_1
  21. last_name_2
  22. name_inferred
  23. gender_guess

Primeros 10 registros exportados:
                                  name   firs